In [ ]:
# Clear all variables
%reset -f


**--- Part I ---**

**0. System check**

In [ ]:
# check system: 
import sys
print(sys.executable)
!where python          # Windows shell command, works with '!' in notebooks
!python --version      # Check Python version
!pip list              # Check installed packages


In [ ]:
import os

# --- Set project root directory automatically ---
project_root = os.path.abspath(".")
os.chdir(project_root)

print("Current Working Directory:", os.getcwd())

# OPTIONAL: Uncomment and customize this if running locally on  own machine
# os.chdir(r"C:\Users\YourName\Documents\Data and code")
# print("Manually set Working Directory:", os.getcwd())


**1. Cleaning data**

In [ ]:
# ✅ Running locally in VS Code (skip this cell if using Google Colab)

import pandas as pd

# ✅ Step 1: Define the path to the local CSV file
file_path = "Document_Data_Download-2025-06-16.csv"

# ✅ Step 2: Load the dataset using recommended settings
data = pd.read_csv(
    file_path,
    encoding="ISO-8859-1",   # Matches the file’s encoding (use 'utf-8' if this fails)
    quotechar='"',           # Properly parses quoted fields with commas
    low_memory=False         # Reads entire file to infer data types correctly
)

# ✅ Step 3: Sanity checks after import
print("✅ Data loaded successfully")
print("Shape:", data.shape)                  # (rows, columns)
print("Columns:", data.columns.tolist())     # List of column names
data.head()                                  # Display the first few rows


In [ ]:

# ✅ Step 3: Preview data
print("✅ Dataset loaded successfully!")
print(f"Shape: {data.shape}") # see number of rows and number of variables
display(data.head())



In [ ]:

# ✅ Generate column summary
column_summary = data.describe(include='all').transpose()
column_summary["null_count"] = data.isnull().sum()
column_summary["data_type"] = data.dtypes

# ✅ Display summary table
column_summary = column_summary.reset_index().rename(columns={"index": "Variable"})
column_summary.head(20)  # You can adjust this number to see more



In [ ]:
# Create a fullinfo-style missing data summary
missing_summary = pd.DataFrame({
    "Variable": data.columns,
    "Missing Count": data.isnull().sum().values,
    "Non-Missing Count": data.notnull().sum().values,
    "Missing %": (data.isnull().mean() * 100).round(2)
})

# Sort by most missing
missing_summary = missing_summary.sort_values(by="Missing %", ascending=False)

# Display the result
missing_summary
# missing_summary.head(20)  # Show top 20

#  Key Reasons for Missing Data in Specific Columns
# Variable	Why It's Often Missing
# Hazard (95%)	Only tagged when a document explicitly addresses climate hazards (e.g., floods, droughts). Most policies are not disaster-risk-focused.
# Framework (94%)	Only applies to overarching “framework” laws. Most documents are not categorized as frameworks (see definition in CCLW methodology).
# Collection Description(s) & Title(s)	These apply only to documents grouped in special CPR/CCLW collections (e.g., EU climate directives). Most policies are not part of such curated sets.
# Author & Author Type (70%)	Often unavailable in source PDFs or not yet extracted. Metadata enrichment is ongoing.
# Document Variant (63%)	Populated only for translations, amendments, or alternative versions — not for original laws.
# Instrument (47%)	Tagged only when a document clearly specifies policy tools (e.g., regulation, investment). Many documents are unclassified here.
# Keyword (44%)	Optional manual tags — not exhaustive. Many records don't include them.
# Sector (36%)	Policies that are cross-sectoral or general may lack specific sector tags.
# Topic/Response (31%)	These depend on manual classification into mitigation, adaptation, etc. Older or unprocessed records may be missing this.


In [ ]:
# Check for exact duplicate rows
duplicate_count = data.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")
data[data.duplicated()]


In [ ]:
print("Sector Distribution:")
print(data["Sector"].value_counts(dropna=False))


In [ ]:
print("\nTopic/Response Distribution:")
print(data["Topic/Response"].value_counts(dropna=False))


**2. Dealing with time and event**

**2.1. --- --- Key modification**

In [ ]:
# Part 1: safe_parse()
from dateutil import parser

def safe_parse(date_str):
    try:
        return parser.parse(date_str, dayfirst=False)
    except:
        try:
            return parser.parse(date_str, dayfirst=True)
        except:
            return None

def extract_sorted_events(row):
    types = row.get("Full timeline of events (types)")
    dates = row.get("Full timeline of events (dates)")
    
    if pd.isna(types) or pd.isna(dates):
        return None
    
    type_list = [t.strip() for t in str(types).split(";")]
    date_list = [d.strip() for d in str(dates).split(";")]
    
    if len(type_list) != len(date_list):
        return None  # mismatched length
    
    # Safely parse dates
    paired = [(typ, safe_parse(d)) for typ, d in zip(type_list, date_list)]
    
    # Remove any with unparseable dates
    paired = [x for x in paired if x[1] is not None]
    
    # Sort by date
    return sorted(paired, key=lambda x: x[1])

# Apply to DataFrame
data["event_timeline"] = data.apply(extract_sorted_events, axis=1)

# Example: View first result
data["event_timeline"].dropna().iloc[0]


In [ ]:
def order_after(df, move_col, after_col):
    cols = list(df.columns)
    cols.remove(move_col)
    insert_at = cols.index(after_col) + 1
    cols.insert(insert_at, move_col)
    return df[cols]

# Apply it like this
data = order_after(data, "event_timeline", "Full timeline of events (dates)")


In [ ]:
# Create additional variables based on the cleaned and sorted event_timeline

# 1. Number of amendments
data["num_amendments"] = data["event_timeline"].apply(
    lambda x: sum(1 for t, _ in x if t == "Amended") if x else 0
)

# 2. Date when the law was first passed/approved
data["first_passed_date"] = data["event_timeline"].apply(
    lambda x: next((d for t, d in x if t == "Passed/Approved"), None) if x else None
)

# 3. Date of most recent event (e.g., latest amendment or update)
data["last_event_date"] = data["event_timeline"].apply(
    lambda x: x[-1][1] if x else None
)

# 4. Policy duration in years (from first to last event)
from numpy import nan
data["policy_duration_years"] = data.apply(
    lambda row: (row["last_event_date"] - row["first_passed_date"]).days / 365.25
    if row["last_event_date"] and row["first_passed_date"] else nan,
    axis=1
)

# Optional: Preview the new columns
data[["first_passed_date", "last_event_date", "num_amendments", "policy_duration_years"]].head()


In [ ]:
# Order the extra variable after event
def order_multiple_after(df, new_cols, after_col):
    cols = list(df.columns)
    for col in new_cols:
        if col in cols:
            cols.remove(col)
    insert_at = cols.index(after_col) + 1
    for col in reversed(new_cols):  # insert in reverse to maintain order
        cols.insert(insert_at, col)
    return df[cols]

# Columns you created
new_cols = ["num_amendments", "first_passed_date", "last_event_date", "policy_duration_years"]

# Reorder them after 'event_timeline'
data = order_multiple_after(data, new_cols, "event_timeline")


**2.2. --- --- Important filter - Full timeline of events (types)**

In [ ]:
# Count before filtering
original_count = data.shape[0]

# Apply filter
exclude_keywords = ["Other", "Repealed", "Superseded"]
data = data[~data["Full timeline of events (types)"]
            .fillna("")
            .str.contains('|'.join(exclude_keywords), case=False)]

# Reset index
data = data.reset_index(drop=True)

# Count after filtering
remaining_count = data.shape[0]
removed_count = original_count - remaining_count

# Print results
print(f"Rows removed: {removed_count}")
print(f"Rows remaining: {remaining_count}")


**3. Cleaning & Paste**

In [ ]:
import re
import string

# Step 1: Define a cleaning function
def clean_text(text):
    if pd.isna(text):
        return ""
    text = re.sub(r"<.*?>", " ", str(text))  # Remove HTML
    text = text.lower()  # Lowercase
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = re.sub(r"\s+", " ", text).strip()  # Normalize whitespace
    return text

# Step 2: Apply to Family Summary (fallback to Document Title if missing)
data["text_for_indicator"] = data["Family Summary"].fillna(data["Document Title"])
data["text_clean"] = data["text_for_indicator"].apply(clean_text)

# Show preview of cleaned text
data[["text_for_indicator", "text_clean"]].head(5)


**--- Part II ---**

## [4]. [Ambition] with un-supervise methods

Justification for Ambition Indicator Method

Because we do not have labeled training data for "high-ambition" vs. "low-ambition" policies, and the policy documents follow a structured format, we use a rule-based keyword approach. This method allows us to transparently score each policy based on clearly defined terms (e.g., net zero, just transition), which can be easily replicated or refined. Our approach is inspired by methods used in recent studies such as the Climate Policy Radar's CPR indicators.


## 4.1. --- --- keywords scoring + weighting

**[a] --- --- --- Main codes**

In [ ]:
# Step 1: Define expanded ambition keyword list (called version 3)
# dictionary from Net zero tracker
ambition_keywords = [
    "net zero", "carbon neutrality", "zero emissions", "carbon neutral", 
    "climate neutrality", "green transition", "energy transition",
    "long-term strategy", "roadmap", "vision", "trajectory", "future-proof", 
    "system transformation", "transformational change",
    "reduce emissions", "emissions reduction", "reduction target", 
    "national target", "carbon pricing", "emissions trading",
    "goal", "objective", "target", "aim", "commit", "pledge", 
    "prioritize", "aspire", "intend", "will reduce", "intends to reduce",
    "just transition", "equity", "inclusive", "social justice",
    "100% renewable", "clean energy", "phase out coal", "climate-resilient", 
    "green economy", "resilience building", "adaptive capacity", 
    "paris agreement", "ndc", "cop26", "cop27", "global climate action",
    "zero carbon", "carbon negative", "net negative", "climate positive",
    "science-based target", "1.5°c target", "1.5 degree", "baseline year", "emissions baseline",
    "absolute emissions", "emissions intensity", "business as usual", "bau",
    "reporting mechanism", "accountability", "review mechanism", "measures and steps", 
    "implementation plan", "executive accountability", "executive pay linkage",
    "carbon credit", "offsets", "removals target", "carbon removal", 
    "nature-based solutions", "ccs", "beccs", "dac", "cdr",
    "scope 1", "scope 2", "scope 3", 
    "territorial emissions", "consumption emissions", 
    "international aviation", "international shipping"
]

# Step 2: Count number of matching ambition keywords per Family Summary
def count_ambition_keywords(text):
    if pd.isna(text):
        return 0
    text = str(text).lower()
    return sum(1 for word in ambition_keywords if word in text)

data["ambition_score_raw"] = data["Family Summary"].apply(count_ambition_keywords)

# Step 3: Define ambition weights by Document Type
ambition_weights = {
    "Vision": 1.5,
    "Long-Term Strategy": 1.5,
    "Roadmap": 1.5,
    "Strategy": 1.0,
    "Plan": 1.0,
    "Action Plan": 1.0,
    "Policy": 1.0,
    "Law": 0.9,
    "Act": 0.9,
    "Regulation": 0.9,
    "Press Release": 0.7,
    "Report": 0.7,
    "Progress Report": 0.7,
    "Summary": 0.7,
    "Submission": 0.5,
    "Communication": 0.5,
    "National Communication": 0.5
}

def get_ambition_weight(doc_type):
    if pd.isna(doc_type):
        return 1.0
    for key in ambition_weights:
        if key.lower() in doc_type.lower():
            return ambition_weights[key]
    return 1.0  # fallback/default

data["ambition_weight"] = data["Document Type"].apply(get_ambition_weight)

# Step 4: Compute final adjusted score
data["ambition_score_adjusted"] = data["ambition_score_raw"] * data["ambition_weight"]

# Step 5: Reorder ambition columns after policy_duration_years
ambition_cols = ["ambition_score_raw", "ambition_weight", "ambition_score_adjusted"]
data = order_multiple_after(data, ambition_cols, "policy_duration_years")

# Step 6: Preview results
data[ambition_cols + ["Document Type", "Family Summary"]].head(10)



In [ ]:
# Step 1: Define expanded ambition keyword list (Key ver 3) - with normalization
# dictionary from Net zero tracker
ambition_keywords = [
    "net zero", "carbon neutrality", "zero emissions", "carbon neutral", 
    "climate neutrality", "green transition", "energy transition",
    "long-term strategy", "roadmap", "vision", "trajectory", "future-proof", 
    "system transformation", "transformational change",
    "reduce emissions", "emissions reduction", "reduction target", 
    "national target", "carbon pricing", "emissions trading",
    "goal", "objective", "target", "aim", "commit", "pledge", 
    "prioritize", "aspire", "intend", "will reduce", "intends to reduce",
    "just transition", "equity", "inclusive", "social justice",
    "100% renewable", "clean energy", "phase out coal", "climate-resilient", 
    "green economy", "resilience building", "adaptive capacity", 
    "paris agreement", "ndc", "cop26", "cop27", "global climate action",
    "zero carbon", "carbon negative", "net negative", "climate positive",
    "science-based target", "1.5°c target", "1.5 degree", "baseline year", "emissions baseline",
    "absolute emissions", "emissions intensity", "business as usual", "bau",
    "reporting mechanism", "accountability", "review mechanism", "measures and steps", 
    "implementation plan", "executive accountability", "executive pay linkage",
    "carbon credit", "offsets", "removals target", "carbon removal", 
    "nature-based solutions", "ccs", "beccs", "dac", "cdr",
    "scope 1", "scope 2", "scope 3", 
    "territorial emissions", "consumption emissions", 
    "international aviation", "international shipping"
]

# Step 2: Count number of matching ambition keywords per Family Summary
def count_ambition_keywords(text):
    if pd.isna(text):
        return 0
    text = str(text).lower()
    return sum(1 for word in ambition_keywords if word in text)

data["ambition_score_raw"] = data["Family Summary"].apply(count_ambition_keywords)

# Step 3: Define ambition weights by Document Type
ambition_weights = {
    "Vision": 1.5,
    "Long-Term Strategy": 1.5,
    "Roadmap": 1.5,
    "Strategy": 1.0,
    "Plan": 1.0,
    "Action Plan": 1.0,
    "Policy": 1.0,
    "Law": 0.9,
    "Act": 0.9,
    "Regulation": 0.9,
    "Press Release": 0.7,
    "Report": 0.7,
    "Progress Report": 0.7,
    "Summary": 0.7,
    "Submission": 0.5,
    "Communication": 0.5,
    "National Communication": 0.5
}

def get_ambition_weight(doc_type):
    if pd.isna(doc_type):
        return 1.0
    for key in ambition_weights:
        if key.lower() in str(doc_type).lower():
            return ambition_weights[key]
    return 1.0

data["ambition_weight"] = data["Document Type"].apply(get_ambition_weight)

# Step 4: Compute final adjusted score
data["ambition_score_adjusted"] = data["ambition_score_raw"] * data["ambition_weight"]

# Step 5: Normalize by Family Summary length
data["family_summary_length"] = data["Family Summary"].astype(str).str.split().str.len()
data["family_summary_length"] = data["family_summary_length"].replace(0, pd.NA)
data["ambition_score_normalized"] = data["ambition_score_adjusted"] / data["family_summary_length"]

# Step 6: Reorder ambition columns after policy_duration_years
ambition_cols = [
    "ambition_score_raw", "ambition_weight", "ambition_score_adjusted",
    "family_summary_length", "ambition_score_normalized"
]
data = order_multiple_after(data, ambition_cols, "policy_duration_years")


# Local display version
print(data[ambition_cols + ["Document Type", "Family Summary"]].head(10))



In [ ]:
# Key version 4: Based on CPR ICLR 2024 target definitions (enhancement to Version 3)
# CPR = Climate Policy Radar
# ICLR 2024 — the International Conference on Learning Representations, one of the top global conferences for AI and machine learning research.
# original work by the Climate Policy Radar team (they’re the authors)
# CPR uses and builds upon the Climate Change Laws of the World (CCLW) database, 
# which is hosted by the Grantham Research Institute at the London School of Economics (LSE).
# The CPR dataset includes documents from CCLW, along with additional documents from the UNFCCC and other national sources.
#climate-laws.org pages (under document details) mention CPR for machine learning and NLP-enhanced exploration of the same policy texts.
# The paper: IDENTIFYING CLIMATE TARGETS IN NATIONAL LAWS AND POLICIES USING MACHINE LEARNING 
# presents machine learning method to extract Net Zero, Reduction, and Other targets
# outlines how they trained machine learning models to automatically detect climate targets
# Sharing pretrained language models (e.g. BERT, RoBERTa).

ambition_keywords_v4 = ambition_keywords + [
    # GHG-specific
    "co2", "co2e", "methane", "ghg", "greenhouse gas",
    
    # Sectoral
    "energy efficiency", "transport emissions", "agriculture", "forestry",
    "deforestation", "reforestation", "land use", "waste", "water use", "industrial emissions",
    
    # Quantified action words
    "halve", "double", "increase by", "reduce by", "cut by", "scale up", "achieve",
    
    # Temporal phrases (best used with regex later)
    "by 2030", "by 2050", "in 5 years", "in the next decade"
]
ambition_keywords_v4 = list(set(ambition_keywords_v4))  # deduplicate

def count_keywords_v4(text):
    if pd.isna(text):
        return 0
    text = str(text).lower()
    return sum(1 for word in ambition_keywords_v4 if word in text)

# Create additional columns
data["ambition_score_raw_v4"] = data["Family Summary"].apply(count_keywords_v4)
data["ambition_score_adjusted_v4"] = data["ambition_score_raw_v4"] * data["ambition_weight"]
data["ambition_score_normalized_v4"] = data["ambition_score_adjusted_v4"] / data["family_summary_length"]


**[b] --- --- --- Aggregate to country-year data**

In [ ]:
# Step 1: Extract year from "First event in timeline" safely
data["document_year"] = pd.to_datetime(data["First event in timeline"], errors='coerce').dt.year

# Step 2: Drop rows with missing or invalid year or country
data = data.dropna(subset=["document_year", "Geographies"])
data["document_year"] = data["document_year"].astype(int)


In [ ]:
# Step 1: Assign document_year 
def extract_year(row):
    if pd.notna(row["first_passed_date"]):
        return pd.to_datetime(row["first_passed_date"]).year
    elif pd.notna(row["last_event_date"]):
        return pd.to_datetime(row["last_event_date"]).year
    else:
        return None

data["document_year"] = data.apply(extract_year, axis=1)
data = data.dropna(subset=["document_year", "Geographies"])
data["document_year"] = data["document_year"].astype(int)

# Step 2: Add binary indicator for high-weight documents (weight ≥ 1.0)
data["is_high_weight"] = (data["ambition_weight"] >= 1.0).astype(int)

# Step 3: Aggregate by country-year
ambition_panel = (
    data
    .groupby(["Geographies", "document_year"])
    .agg(
        num_docs=("Family Summary", "count"),
        mean_raw=("ambition_score_raw", "mean"),
        mean_weight=("ambition_weight", "mean"),
        share_high_weight_docs=("is_high_weight", "mean"),  # mean of 0/1 = proportion
        mean_adjusted=("ambition_score_adjusted", "mean"),
        total_adjusted=("ambition_score_adjusted", "sum")
    )
    .reset_index()
    .sort_values(["Geographies", "document_year"])
)

# Optional: round for clarity
ambition_panel[["mean_raw", "mean_weight", "share_high_weight_docs", "mean_adjusted"]] = \
    ambition_panel[["mean_raw", "mean_weight", "share_high_weight_docs", "mean_adjusted"]].round(3)

# Preview
ambition_panel.head(10)


In [ ]:
# === (a) Add ISO3 code from original data ===
iso_map = data[["Geographies", "Geography ISOs"]].dropna().drop_duplicates()
ambition_panel = ambition_panel.merge(iso_map, on="Geographies", how="left")

# === (b) Reorder columns and rename for clarity ===
ambition_panel.rename(columns={
    "Geography ISOs": "iso3",
    "document_year": "year"
}, inplace=True)

# === (c) Move iso3 and year to front
cols = ["iso3", "year"] + [col for col in ambition_panel.columns if col not in ["iso3", "year", "Geographies"]]
ambition_panel = ambition_panel[cols]

# === (d) Save to CSV
ambition_panel.to_csv("ambition_keywordscore.csv", index=False)
print("✅ Saved to: ambition_keywordscore.csv")

# === (e) Preview
ambition_panel.head(10)


In [ ]:
# Full Aggregation Code for v4 Ambition Variables (update version of version 3)

# === (a) Reuse year and ISO preparation ===
def extract_year(row):
    if pd.notna(row["first_passed_date"]):
        return pd.to_datetime(row["first_passed_date"]).year
    elif pd.notna(row["last_event_date"]):
        return pd.to_datetime(row["last_event_date"]).year
    else:
        return None

data["document_year"] = data.apply(extract_year, axis=1)
data = data.dropna(subset=["document_year", "Geographies"])
data["document_year"] = data["document_year"].astype(int)
data["is_high_weight"] = (data["ambition_weight"] >= 1.0).astype(int)

# === (b) Group and aggregate v4 indicators ===
ambition_panel_v4 = (
    data
    .groupby(["Geographies", "document_year"])
    .agg(
        num_docs=("Family Summary", "count"),
        mean_raw_v4=("ambition_score_raw_v4", "mean"),
        mean_weight=("ambition_weight", "mean"),
        share_high_weight_docs=("is_high_weight", "mean"),
        mean_adjusted_v4=("ambition_score_adjusted_v4", "mean"),
        total_adjusted_v4=("ambition_score_adjusted_v4", "sum"),
        mean_normalized_v4=("ambition_score_normalized_v4", "mean")
    )
    .reset_index()
    .sort_values(["Geographies", "document_year"])
)

# === (c) Add ISO codes from the full data
iso_map = data[["Geographies", "Geography ISOs"]].dropna().drop_duplicates()
ambition_panel_v4 = ambition_panel_v4.merge(iso_map, on="Geographies", how="left")

# === (d) Rename and reorder columns
ambition_panel_v4.rename(columns={
    "Geography ISOs": "iso3",
    "document_year": "year"
}, inplace=True)

cols_ordered = ["iso3", "year"] + [col for col in ambition_panel_v4.columns if col not in ["iso3", "year", "Geographies"]]
ambition_panel_v4 = ambition_panel_v4[cols_ordered]

# === (e) Round numerical values for presentation
cols_to_round = [col for col in ambition_panel_v4.columns if col.startswith("mean_") or col.startswith("total_") or col.startswith("share_")]
ambition_panel_v4[cols_to_round] = ambition_panel_v4[cols_to_round].round(3)

# === (f) Export to CSV
ambition_panel_v4.to_csv("ambition_keywordscore_enhance.csv", index=False)
print("✅ Saved to: ambition_keywordscore_enhance.csv")

# Preview
ambition_panel_v4.head(10)


**[c] --- --- --- Statistics, Trends**

In [ ]:
# Summary statistic (not aggregate)
import pandas as pd

# List of ambition-related variables
ambition_vars = ["ambition_score_raw", "ambition_weight", "ambition_score_adjusted"]

# Separate numeric and categorical
numeric_vars = [col for col in ambition_vars if pd.api.types.is_numeric_dtype(data[col])]
categorical_vars = [col for col in ambition_vars if col not in numeric_vars]

# 1️⃣ Stata-style summary for numeric variables
numeric_summary = data[numeric_vars].agg(['count', 'mean', 'std', 'min', 'max']).T
numeric_summary = numeric_summary.rename(columns={
    "count": "N",
    "mean": "Mean",
    "std": "Std. Dev.",
    "min": "Min",
    "max": "Max"
})
numeric_summary = numeric_summary.round(2)

# Print numeric summary
print("Stata-style Summary Statistics for Numeric Variables:\n")
print(numeric_summary)

# 2️⃣ Tabulation for categorical variables (none expected, but for completeness)
for col in categorical_vars:
    print(f"\nStata-style Tabulation for: {col}")
    tab = data[col].value_counts(dropna=False).reset_index()
    tab.columns = [col, "Frequency"]
    tab["Percent"] = (tab["Frequency"] / tab["Frequency"].sum() * 100).round(2)
    print(tab)


In [ ]:
# statistics for a specific year (aggregate country-year)
# Filter to 2016 only
ambition_2016 = ambition_panel[ambition_panel["document_year"] == 2016]

# Stata-style summary
summary_2016 = ambition_2016[[
    "num_docs", 
    "mean_raw", 
    "mean_weight", 
    "share_high_weight_docs", 
    "mean_adjusted", 
    "total_adjusted"
]].agg(['count', 'mean', 'std', 'min', 'max']).T

summary_2016 = summary_2016.rename(columns={
    "count": "N", 
    "mean": "Mean", 
    "std": "Std. Dev.", 
    "min": "Min", 
    "max": "Max"
}).round(2)

print("Stata-style Summary Statistics for 2016:\n")
print(summary_2016)


In [ ]:
# Histogram of [mean_adjusted] ambition scores across countries in 2016
import matplotlib.pyplot as plt


plt.figure(figsize=(8, 5))
plt.hist(ambition_2016["mean_adjusted"], bins=15, edgecolor='black')
plt.title("Distribution of Mean Adjusted Ambition Scores Across Countries (2016)")
plt.xlabel("Mean Adjusted Ambition Score (per document)")
plt.ylabel("Number of Countries")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Replace with one of: "num_docs", "mean_raw", "mean_weight", "share_high_weight_docs"
var_to_plot = "num_docs"

# Histogram of selected ambition variable across countries in 2016
plt.figure(figsize=(8, 5))
plt.hist(ambition_2016[var_to_plot], bins=15, edgecolor='black', color='skyblue')
plt.title(f"Distribution of {var_to_plot.replace('_', ' ').title()} Across Countries (2016)")
plt.xlabel(var_to_plot.replace('_', ' ').title())
plt.ylabel("Number of Countries")
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:
# matplot across countries in 2016
import matplotlib.pyplot as plt

# Merge ISO codes into 2016 data (if not already included)
# Assuming ambition_panel doesn't yet include "Geography ISOs", merge it in:
ambition_2016 = ambition_2016.merge(
    data[["Geographies", "Geography ISOs"]].drop_duplicates(),
    on="Geographies",
    how="left"
)

# Sort by ambition score for cleaner visualization
ambition_2016_sorted = ambition_2016.sort_values("mean_adjusted", ascending=False)

# Plot bar chart
plt.figure(figsize=(14, 6))
plt.bar(ambition_2016_sorted["Geography ISOs"], ambition_2016_sorted["mean_adjusted"], color='skyblue', edgecolor='black')
plt.title("Mean Adjusted Ambition Scores Across Countries (2016)")
plt.xlabel("Country (ISO Code)")
plt.ylabel("Mean Adjusted Ambition Score (per document)")
plt.xticks(rotation=90)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
#  Bar Chart of 2016 Ambition Scores by ISO Code
import matplotlib.pyplot as plt
import numpy as np

# Create some spacing between bars
x = np.arange(len(ambition_2016_sorted))  # numeric positions for ISO codes
width = 0.6  # narrower bars

# Set figure
plt.figure(figsize=(16, 6))

# Plot bar chart
plt.bar(x, ambition_2016_sorted["mean_adjusted"], width=width, color='skyblue', edgecolor='black')

# X-axis with ISO codes, small font, spaced labels
plt.xticks(ticks=x, labels=ambition_2016_sorted["Geography ISOs"], rotation=90, fontsize=8)

plt.title("Mean Adjusted Ambition Scores Across Countries (2016)", fontsize=14)
plt.xlabel("Country (ISO Code)", fontsize=11)
plt.ylabel("Mean Adjusted Ambition Score (per document)", fontsize=11)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
# 2. Line Plot of Global Ambition Trend Over Time
# Step 1: Compute yearly global average ambition
global_trend = (
    ambition_panel
    .groupby("document_year")
    .agg(global_avg_ambition=("mean_adjusted", "mean"))
    .reset_index()
)

# Step 2: Plot the trend
plt.figure(figsize=(8, 5))
plt.plot(global_trend["document_year"], global_trend["global_avg_ambition"], marker='o')
plt.title("Global Average Ambition Score Over Time")
plt.xlabel("Year")
plt.ylabel("Avg. Adjusted Ambition Score (per document)")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Step 1: Create yearly global averages and totals
global_trend = (
    ambition_panel
    .groupby("document_year")
    .agg(
        mean_adjusted=("mean_adjusted", "mean"),
        total_adjusted=("total_adjusted", "sum")
    )
    .reset_index()
)

# Step 2: Restrict to 1990 onward
global_trend = global_trend[global_trend["document_year"] >= 1990]

# Step 3: Identify the peak year in mean_adjusted
peak_year = global_trend.loc[global_trend["mean_adjusted"].idxmax(), "document_year"]
peak_value = global_trend["mean_adjusted"].max()

# Step 4: Plot both lines
plt.figure(figsize=(10, 6))
plt.plot(global_trend["document_year"], global_trend["mean_adjusted"], label="Mean Adjusted Ambition", marker='o')
plt.plot(global_trend["document_year"], global_trend["total_adjusted"], label="Total Adjusted Ambition", linestyle='--', marker='x', color='orange')

# Step 5: Annotate the Kyoto peak
plt.axvline(x=peak_year, color='red', linestyle='--', label=f'Peak Year ({peak_year})')
plt.text(peak_year + 0.5, peak_value + 0.1, f'Kyoto peak: {peak_year}', color='red')

# Final touches
plt.title("Global Ambition Scores Over Time (1990–Present)")
plt.xlabel("Year")
plt.ylabel("Ambition Score")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
#3. xtline-style Panel Plot: Selected Countries Over Time
# Step 1: Filter to selected countries
selected_countries = ["India", "Brazil", "Germany", "United Kingdom", "Kenya"]
filtered = ambition_panel[ambition_panel["Geographies"].isin(selected_countries)]

# Step 2: Plot multiple lines (like xtline in Stata)
plt.figure(figsize=(10, 6))
for country in selected_countries:
    subset = filtered[filtered["Geographies"] == country]
    plt.plot(subset["document_year"], subset["mean_adjusted"], label=country, marker='o')

plt.title("Mean Adjusted Ambition Score Over Time by Country")
plt.xlabel("Year")
plt.ylabel("Mean Adjusted Ambition Score")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Top 10 and Bottom 10 Countries by Mean Ambition Score (year == 2016)

# Merge ISO codes (if needed)
ambition_2016 = ambition_panel[ambition_panel["document_year"] == 2016].merge(
    data[["Geographies", "Geography ISOs"]].drop_duplicates(),
    on="Geographies",
    how="left"
)

# Top 10 countries
top10 = ambition_2016.sort_values("mean_adjusted", ascending=False).head(10)
print("Top 10 Countries by Mean Ambition Score (2016):")
print(top10[["Geography ISOs", "Geographies", "mean_adjusted"]])

# Bottom 10 countries
bottom10 = ambition_2016.sort_values("mean_adjusted", ascending=True).head(10)
print("\nBottom 10 Countries by Mean Ambition Score (2016):")
print(bottom10[["Geography ISOs", "Geographies", "mean_adjusted"]])


In [ ]:
# 2. Bar Plot of total_adjusted Ambition (2016)

import matplotlib.pyplot as plt

# Sort by total ambition score
ambition_2016_sorted = ambition_2016.sort_values("total_adjusted", ascending=False)

# Bar plot: total adjusted ambition
plt.figure(figsize=(16, 6))
plt.bar(ambition_2016_sorted["Geography ISOs"], ambition_2016_sorted["total_adjusted"], color='salmon', edgecolor='black')
plt.title("Total Adjusted Ambition Scores Across Countries (2016)")
plt.xlabel("Country (ISO Code)")
plt.ylabel("Total Adjusted Ambition Score")
plt.xticks(rotation=90, fontsize=8)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


## 4.2. --- --- Topic Modeling (e.g., LDA or BERTopic)**

In [ ]:
# Skip (not run)
# Install in local terminal (here is miniconda) - not need if using colab or cloud based
conda create -n bertopic_env python=3.10 -y
conda activate bertopic_env
pip install bertopic[all]

# Once installed, register the environment with Jupyter:
pip install ipykernel
python -m ipykernel install --user --name=bertopic_env --display-name "Python (BERTopic)"
conda activate bertopic_env

# Open Jupyter Notebook (VS Code or Jupyter Lab)
# Click the kernel selector in the top right
# Choose: Python (BERTopic)


In [ ]:
!pip install ipykernel
!python -m ipykernel install --user --name=bertopic_env --display-name "Python (BERTopic)"
!conda activate bertopic_env

In [ ]:

# install in bash:
conda install -p "c:\Users\AppData\Local\miniconda3\envs\bertopic_env" ipykernel --update-deps --force-reinstall


In [ ]:
from bertopic import BERTopic
model = BERTopic()
print("✓ BERTopic is working!")


In [ ]:
# 1️⃣ Import packages
# pip install bertopic (install in terminal - miniconda)
import pandas as pd
import re, string
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

# 2️⃣ Load policy document dataset
data = pd.read_csv("Document_Data_Download-2025-06-16.csv")


**[b] --- --- --- Standard version (fixed & reproducible)**

In [ ]:
!pip install nltk
!pip install BERTopic

In [ ]:
# need to change the kernel to BERtopic
from bertopic import BERTopic
model = BERTopic()
print("✓ BERTopic is working!")


In [ ]:
# 📦 Step 0: Import required libraries
import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords
from nltk import download
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from hdbscan import HDBSCAN

# 🧠 Step 1: Set up preprocessing
download("stopwords")
stop_words = set(stopwords.words("english"))

# 2️⃣ Load policy document dataset
data = pd.read_csv("Document_Data_Download-2025-06-16.csv")


# STEP 3: Clean-up function
def clean_text(text):
    if pd.isna(text):
        return ""
    text = re.sub(r"<.*?>", " ", str(text))  # remove HTML tags
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
    words = text.split()
    words = [w for w in words if w not in stop_words]  # remove stopwords
    text = " ".join(words)  # reassemble
    text = re.sub(r"\s+", " ", text).strip()  # normalize whitespace
    return text

def enrich_text(row):
    base = row["Family Summary"] if pd.notna(row["Family Summary"]) else row.get("Document Title", "")
    topic = f"Topic: {row['Topic/Response']}" if pd.notna(row.get("Topic/Response")) else ""
    timeline = f"Events: {row['Full timeline of event types']}" if pd.notna(row.get("Full timeline of event types")) else ""
    return f"{base}. {topic}. {timeline}"

# 👷 Step 2: Apply text processing to dataset
data["text_for_topic_model"] = data.apply(enrich_text, axis=1)
data["text_clean"] = data["text_for_topic_model"].apply(clean_text)
# Check sample cleaned text
data["text_clean"].dropna().sample(3, random_state=42).tolist()


In [ ]:
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from bertopic import BERTopic

# Set reproducible components
vectorizer_model = CountVectorizer(stop_words="english")
umap_model = UMAP(random_state=42)
hdbscan_model = HDBSCAN(prediction_data=True)  # Enables probability outputs

# Create the list of cleaned documents
docs = data["text_clean"].tolist()

# Fit BERTopic with probability calculation enabled
topic_model = BERTopic(
    calculate_probabilities=True,
    vectorizer_model=vectorizer_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    verbose=True
)

topics, probs = topic_model.fit_transform(docs)


In [ ]:
# Reduce to 10 interpretable topics
topic_model = topic_model.reduce_topics(docs, nr_topics=10)

# Recompute distributions after reduction
topics, probs = topic_model.transform(docs)

# Save top topic and top probability
data["topic"] = topics
data["topic_prob"] = [max(p) if isinstance(p, (list, np.ndarray)) else 0 for p in probs]


In [ ]:
# Topics based on review: [0, 3, 4, 5]
ambition_topics = [0, 3, 4, 5]

data["ambition_score_topic"] = [
    sum([p[i] for i in ambition_topics]) if isinstance(p, (list, np.ndarray)) else 0
    for p in probs
]

def label_ambition(score, threshold_high=0.6, threshold_med=0.3):
    if score >= threshold_high:
        return "High"
    elif score >= threshold_med:
        return "Medium"
    else:
        return "Low"

data["ambition_label"] = data["ambition_score_topic"].apply(label_ambition)
print(data["ambition_score_topic"].describe())
print(data["ambition_label"].value_counts())


In [ ]:
# Save as a new CSV file with timestamp or version label
data_with_topics = data.copy()

# Save to CSV (change path if needed)
data_with_topics.to_csv("data_with_bertopic_ambition.csv", index=False)

print("✅ Saved to 'data_with_bertopic_ambition.csv'")


**[b2] --- --- --- >>> Aggregate to country-year data**

In [ ]:
# === Step 1: Assign document_year using First event in timeline, then overwrite ===
data["document_year"] = pd.to_datetime(data["First event in timeline"], errors="coerce").dt.year

def overwrite_year(row):
    if pd.notna(row.get("first_passed_date")):
        return pd.to_datetime(row["first_passed_date"], errors="coerce").year
    elif pd.notna(row.get("last_event_date")):
        return pd.to_datetime(row["last_event_date"], errors="coerce").year
    else:
        return row["document_year"]

data["document_year"] = data.apply(overwrite_year, axis=1)

# === Step 2: Drop rows missing year or geography ===
data = data.dropna(subset=["document_year", "Geographies"])
data["document_year"] = data["document_year"].astype(int)

# === Step 3: Aggregate BERTopic results by country-year ===
bert_panel = (
    data
    .groupby(["Geographies", "document_year"])
    .agg(
        num_docs=("Family Summary", "count"),
        mean_topic_score=("ambition_score_topic", "mean"),
        share_high_topic=("ambition_label", lambda x: (x == "High").mean()),
        share_medium_topic=("ambition_label", lambda x: (x == "Medium").mean()),
        share_low_topic=("ambition_label", lambda x: (x == "Low").mean()),
        avg_topic_prob=("topic_prob", "mean")
    )
    .reset_index()
    .sort_values(["Geographies", "document_year"])
)

# === Step 4: Round for clarity ===
cols_to_round = ["mean_topic_score", "share_high_topic", "share_medium_topic", "share_low_topic", "avg_topic_prob"]
bert_panel[cols_to_round] = bert_panel[cols_to_round].round(3)

# === Step 5 (Optional): Add ISO3 codes ===
iso_map = data[["Geographies", "Geography ISOs"]].dropna().drop_duplicates()
bert_panel = bert_panel.merge(iso_map, on="Geographies", how="left")

# === Step 6: Reorder columns ===
bert_panel.rename(columns={"Geography ISOs": "iso3", "document_year": "year"}, inplace=True)
cols = ["iso3", "year"] + [c for c in bert_panel.columns if c not in ["iso3", "year", "Geographies"]]
bert_panel = bert_panel[cols]

# === Step 7: Save to CSV ===
bert_panel.to_csv("bert_topic_panel.csv", index=False)
print("✅ Saved to 'bert_topic_panel.csv'")

# === Step 8: Preview ===
bert_panel.head(10)


**[b3] --- --- --- >>> Summary statistics**

In [ ]:
# Summary Statistics
import pandas as pd

# List of BERTopic-related variables
bert_vars = ["ambition_score_topic", "target_score_CPRstyle", "ambition_label", "target_type_topic"]

# Separate numeric and categorical
numeric_vars = [col for col in bert_vars if pd.api.types.is_numeric_dtype(data[col])]
categorical_vars = [col for col in bert_vars if col not in numeric_vars]

# Numeric: summary stats
numeric_summary = data[numeric_vars].agg(['count', 'mean', 'std', 'min', 'max']).T
numeric_summary = numeric_summary.rename(columns={
    "count": "N", "mean": "Mean", "std": "Std. Dev.", "min": "Min", "max": "Max"
}).round(2)

print("📊 Stata-style Summary for Numeric BERTopic Variables:\n")
print(numeric_summary)

# Categorical: tabulation
for col in categorical_vars:
    print(f"\n🧩 Stata-style Tabulation for: {col}")
    tab = data[col].value_counts(dropna=False).reset_index()
    tab.columns = [col, "Frequency"]
    tab["Percent"] = (tab["Frequency"] / tab["Frequency"].sum() * 100).round(2)
    print(tab)


In [ ]:
# Filter BERTopic-only panel to 2016
bert_panel_2016 = bert_panel[bert_panel["document_year"] == 2016].copy()

# Stata-style summary for BERTopic ambition variables
summary_2016_bert = bert_panel_2016[[
    "mean_topic_score", "share_high_topic", 
    "share_netzero", "share_reduction", "share_other", "avg_target_score"
]].agg(['count', 'mean', 'std', 'min', 'max']).T

summary_2016_bert = summary_2016_bert.rename(columns={
    "count": "N", "mean": "Mean", "std": "Std. Dev.", "min": "Min", "max": "Max"
}).round(2)

print("\n📊 Stata-style Summary for BERTopic Panel Variables in 2016:\n")
print(summary_2016_bert)


In [ ]:
import matplotlib.pyplot as plt

# Plot: Distribution of mean_topic_score across countries in 2016
plt.figure(figsize=(9, 5))
plt.hist(bert_panel_2016["mean_topic_score"], bins=15, edgecolor='black', color='skyblue')

plt.title("Distribution of Mean BERTopic Ambition Scores Across Countries (2016)", fontsize=13)
plt.xlabel("Mean Topic-Based Ambition Score (per document)", fontsize=11)
plt.ylabel("Number of Countries", fontsize=11)

plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:
#  Bar Chart by ISO Country Code (2016)
# Merge ISO if needed
bert_panel_2016 = bert_panel_2016.merge(
    data[["Geographies", "Geography ISOs"]].drop_duplicates(),
    on="Geographies",
    how="left"
)

# Sort
bert_panel_2016_sorted = bert_panel_2016.sort_values("mean_topic_score", ascending=False)

# Plot
import numpy as np

x = np.arange(len(bert_panel_2016_sorted))
plt.figure(figsize=(16, 6))
plt.bar(x, bert_panel_2016_sorted["mean_topic_score"], width=0.6, color='skyblue', edgecolor='black')
plt.xticks(ticks=x, labels=bert_panel_2016_sorted["Geography ISOs"], rotation=90, fontsize=8)
plt.title("Mean Topic-Based Ambition Scores Across Countries (2016)")
plt.xlabel("Country (ISO Code)")
plt.ylabel("Topic-Based Ambition Score (Mean per Document)")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
# Global trend from BERTopic scores
global_trend_topic = (
    ambition_panel
    .groupby("document_year")
    .agg(global_avg_topic=("mean_topic_score", "mean"))
    .reset_index()
)

plt.figure(figsize=(8, 5))
plt.plot(global_trend_topic["document_year"], global_trend_topic["global_avg_topic"], marker='o')
plt.title("Global Average BERTopic Ambition Score Over Time")
plt.xlabel("Year")
plt.ylabel("Avg. Topic-Based Ambition Score")
plt.grid(True)
plt.tight_layout()
plt.show()


## [5]. [Policy Action] Semi-Supervised Machine Learning Classification - Weak Labeling

I apply weak labeling to generate initial binary labels (`1 = Climate Policy Action`, `0 = Not Action`) using heuristics based on:
- Timeline status ("Passed/Approved")
- Document Type (e.g., Law, Strategy, Plan, etc.)
- Summary verbs indicating action (e.g., implement, enforce, allocate)

This allows us to build an initial labeled dataset without full manual annotation.


**[a] --- --- --- Extract the gold test set & remain weak training set** 

In [ ]:
# 📦 Step 1: Import required libraries
import pandas as pd
from sklearn.model_selection import train_test_split

# 📄 Step 2: Load dataset (adjust path if needed)
file_path = "Document_Data_Download-2025-06-16.csv"  # Or  correct relative path
df = pd.read_csv(file_path, encoding="ISO-8859-1", quotechar='"', low_memory=False)

# ✅ Step 3: Filter for rows with non-null Family Summary and Document Type
df_filtered = df[df["Family Summary"].notnull() & df["Document Type"].notnull()].copy()

# ✅ Step 4: Drop rare Document Types (fewer than 5 docs) to ensure valid stratification
value_counts = df_filtered["Document Type"].value_counts()
valid_types = value_counts[value_counts >= 5].index
df_stratified = df_filtered[df_filtered["Document Type"].isin(valid_types)].copy()


In [ ]:

# ✅ Step 5: Stratified sampling (200 samples by Document Type)
gold_set, _ = train_test_split(
    df_stratified,
    stratify=df_stratified["Document Type"],
    test_size=(len(df_stratified) - 200),
    random_state=42
)

# ✅ Step 6: Save to CSV (optional: preview first few rows)
gold_set.to_csv("gold_policy_action_sample_200.csv", index=False)
print("✅ Gold set saved to 'gold_policy_action_sample_200.csv'")
display(gold_set.head())


In [ ]:
# 📦 Step 1: Filter remaining samples (exclude the gold test set)
# Make sure both `df_stratified` and `gold_set` exist from earlier

df_weak_train = df_stratified[~df_stratified.index.isin(gold_set.index)].copy()

# ✅ Step 2: Save to CSV
df_weak_train.to_csv("weak_train_policy_action.csv", index=False)
print("✅ Weak supervision training set saved as 'weak_train_policy_action.csv'")
display(df_weak_train.head())
# 

In [ ]:
# Sample change:
# Full 8974 => removing missing in [Family Summary] [Document Type], remain: 8922
# filter rare, then extract 200 testing set, remain 8722



**[b1] --- --- --- [not use] Apply weak supversing (or rule - relax condidtion)**

In [ ]:
!pip install snorkel


In [ ]:
# ✅ Step 1: Load full weak training dataset (keep all columns)
import pandas as pd

df_train = pd.read_csv("weak_train_policy_action.csv")

# Filter only rows with valid Family Summary (retain all columns)
df_train = df_train[df_train["Family Summary"].notnull()].reset_index(drop=True)


In [ ]:
from snorkel.labeling import labeling_function

ABSTAIN = -1  # manually define it ( labeling functions will work without needing to import ABSTAIN.)
ACTION = 1
NOT_ACTION = 0


In [ ]:
# Step 3: Define Labeling Functions (LFs)

# LF1: Passed/Approved mentioned
@labeling_function()
def lf_passed_approved(x):
    return ACTION if "passed/approved" in str(x["Full timeline of events (types)"]).lower() else ABSTAIN

# LF2: Strong action document types
@labeling_function()
def lf_strong_doc_type(x):
    strong_types = ["law", "act", "decree", "regulation", "executive order",
                    "plan", "strategy", "action plan", "framework", "policy", "roadmap"]
    return ACTION if any(t in str(x["Document Type"]).lower() for t in strong_types) else ABSTAIN

# LF3: Weak doc types (non-action)
@labeling_function()
def lf_weak_doc_type(x):
    weak_types = ["submission", "inventory", "report", "press release", "communication", "ndc"]
    return NOT_ACTION if any(t in str(x["Document Type"]).lower() for t in weak_types) else ABSTAIN

# LF4: Action-related verbs in Family Summary
@labeling_function()
def lf_action_keywords(x):
    verbs = ["implement", "enforce", "regulate", "support", "establish", "provide",
             "launch", "apply", "administer", "monitor", "execute", "fund", "finance",
             "deliver", "operate", "incentivize", "reporting", "compliance", "mechanism"]
    text = str(x["Family Summary"]).lower()
    return ACTION if any(word in text for word in verbs) else ABSTAIN



In [ ]:
#  Step 4: Apply LFs using Snorkel
from snorkel.labeling import PandasLFApplier

lfs = [lf_passed_approved, lf_strong_doc_type, lf_weak_doc_type, lf_action_keywords]
applier = PandasLFApplier(lfs)

# IMPORTANT: apply to full df with all original columns preserved
L_train = applier.apply(df=df_train)

# inspect label matrix
import numpy as np
print("Label matrix shape:", L_train.shape)


In [ ]:
# ✅ Step 5: Train Snorkel LabelModel and Add New Columns
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100)

# Add predicted hard and soft labels to full original DataFrame
df_train["policy_action"] = label_model.predict(L=L_train)
df_train["action_label_prob"] = label_model.predict_proba(L=L_train)[:, 1]


In [ ]:
# ✅ Step 6: Save Full Dataset for Future ML Use
# All original columns + policy_action + action_label_prob
df_train.to_csv("weak_labeled_policy_action_dataset.csv", index=False)
print("✅ Saved complete dataset with weak labels as 'weak_labeled_policy_action_dataset.csv'")


**[b2] --- --- --- [not use] Apply weak supversing (and rule - stricter condidtion) Ver 1**

In [ ]:
# ✅ Step 1: Load full weak training dataset (keep all columns)
import pandas as pd

df_train = pd.read_csv("weak_train_policy_action.csv")

# Filter only rows with valid Family Summary (retain all columns)
df_train = df_train[df_train["Family Summary"].notnull()].reset_index(drop=True)


In [ ]:
# ✅ Step 2: Define strict AND logic-based labeling function
from snorkel.labeling import labeling_function

ABSTAIN = -1
ACTION = 1
NOT_ACTION = 0

@labeling_function()
def lf_strict_policy_action(x):
    # 1. Check Passed/Approved
    status_check = "passed/approved" in str(x["Full timeline of events (types)"]).lower()
    
    # 2. Check strong document types
    strong_types = ["law", "act", "decree", "regulation", "executive order",
                    "plan", "strategy", "action plan", "framework", "policy", "roadmap"]
    doc_type = str(x["Document Type"]).lower()
    doc_type_check = any(t in doc_type for t in strong_types)

    # 3. Check weak document types (should NOT be any)
    weak_types = ["submission", "inventory", "report", "press release", "communication", "ndc"]
    if any(w in doc_type for w in weak_types):
        return NOT_ACTION

    # 4. Check Family Summary for action verbs
    verbs = ["implement", "enforce", "regulate", "support", "establish", "provide",
             "launch", "apply", "administer", "monitor", "execute", "fund", "finance",
             "deliver", "operate", "incentivize", "reporting", "compliance", "mechanism,
             "adopt", 
             "]
    summary = str(x["Family Summary"]).lower()
    text_check = any(v in summary for v in verbs)

    # Final decision: all three required checks must pass
    if status_check and doc_type_check and text_check:
        return ACTION
    else:
        return ABSTAIN


In [ ]:
# ✅ Step 3: Apply strict LF using Snorkel
from snorkel.labeling import PandasLFApplier

lfs = [lf_strict_policy_action]
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

# Inspect label matrix
import numpy as np
print("Label matrix shape:", L_train.shape)


In [ ]:
# ✅ Step 4: Train Snorkel LabelModel and Generate Labels (Skipped - dont use)
# We have created a strict, rule-based labeling function (lf_strict_policy_action) that implements a fully deterministic AND condition: 
# Passed/Approved
# AND Document Type is strong
# AND not weak
# AND Family Summary contains action verb
# The Snorkel LabelModel is helpful only when:
# We have multiple noisy labeling functions
# We want Snorkel to learn how reliable each LF is
# We want to combine votes when LFs conflict or abstain
# Why Skipping Snorkel’s LabelModel Is OK
# Snorkel is a tool to combine multiple noisy labels
# But our rule is already high-precision and low-noise
# So we are using deterministic weak labeling
# That still counts as weak supervision — just a rule-based variant (which is valid and widely used).

# Weak supervision means using imperfect, heuristic, or rule-based labels instead of (or in addition to)
# manually annotated ground truth to train or bootstrap a machine learning model.

from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100)

df_train["policy_action"] = label_model.predict(L=L_train)
df_train["action_label_prob"] = label_model.predict_proba(L=L_train)[:, 1]


In [ ]:
# ✅ Apply the rule directly to label all rows
df_train["policy_action"] = df_train.apply(lf_strict_policy_action, axis=1)


In [ ]:
df_train["policy_action"] = df_train["policy_action"].replace(-1, 0)


In [ ]:
# ✅ Step 5: Save full dataset for future use
df_train.to_csv("weak_labeled_policy_action_dataset_strict.csv", index=False)
print("✅ Saved strict weak-labeled dataset as 'weak_labeled_policy_action_dataset_strict.csv'")


**[b3] --- --- --- [Used] Apply weak supversing (and rule - stricter condidtion) Ver 2**

i. now exclude :  "plan", "strategy", "roadmap"

ii. should check again the document type, may miss several important type: EU regulation, Act...

In [ ]:
# ✅ Step 1: Load full weak training dataset (keep all columns)
import pandas as pd

df_train = pd.read_csv("weak_train_policy_action.csv")

# Filter only rows with valid Family Summary (retain all columns)
df_train = df_train[df_train["Family Summary"].notnull()].reset_index(drop=True)


In [ ]:
# ✅ Step 2: Define strict AND logic-based labeling function
from snorkel.labeling import labeling_function

ABSTAIN = -1
ACTION = 1
NOT_ACTION = 0

@labeling_function()
def lf_strict_policy_action(x):
    # 1. Check Passed/Approved
    status_check = "passed/approved" in str(x["Full timeline of events (types)"]).lower()
    
    # 2. Check strong document types
    # now exclude :  "plan", "strategy", "roadmap"
    strong_types = ["law", "act", "decree", "regulation", "executive order", "order",
                    "action plan", "framework", "policy", "directive", "ordinance",
                    "eu directive", "decree law", "eu regulation", "royal decree"]
    doc_type = str(x["Document Type"]).lower()
    doc_type_check = any(t in doc_type for t in strong_types)

    # 3. Check weak document types (should NOT be any)
    weak_types = ["submission", "inventory", "report", "press release", "communication", "ndc"]
    if any(w in doc_type for w in weak_types):
        return NOT_ACTION

    # 4. Check Family Summary for action verbs
    verbs = ["implement", "enforce", "regulate", "support", "establish", "provide",
             "launch", "apply", "administer", "monitor", "execute", "fund", "finance",
             "deliver", "operate", "incentivize", "reporting", "compliance",
             "adopt","reduce", "protect", "mitigate", "build capacity", "empower", "increase resilience"]
    summary = str(x["Family Summary"]).lower()
    text_check = any(v in summary for v in verbs)

    # Final decision: all three required checks must pass
    if status_check and doc_type_check and text_check:
        return ACTION
    else:
        return ABSTAIN


In [ ]:
# ✅ Apply the rule directly to label all rows
df_train["policy_action"] = df_train.apply(lf_strict_policy_action, axis=1)


In [ ]:
df_train["policy_action"] = df_train["policy_action"].replace(-1, 0)


In [ ]:
# ✅ Step 5: Save full dataset for future use
df_train.to_csv("weak_labeled_policy_action_dataset_strict2.csv", index=False)
print("✅ Saved strict weak-labeled dataset as 'weak_labeled_policy_action_dataset_strict2.csv'")


## [6]. [Policy Action] Classification using standard Machine learning

## 6.1. Random forest

In [ ]:
# === (a) Import Libraries ===
import pandas as pd
import numpy as np
import xgboost as xgb
import re
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    accuracy_score, roc_auc_score, confusion_matrix,
    mean_squared_error, brier_score_loss
)

# === (b) Load Data ===
train = pd.read_csv("weak_labeled_policy_action_dataset_strict2.csv", encoding="ISO-8859-1")
test = pd.read_csv("classified_climate_policy_actions.csv", encoding="ISO-8859-1")


**[a] --- --- --- Dealing the time_event (single time: event_year_first and event_year_last)**

In [ ]:
from dateutil import parser

def safe_parse_year(text):
    if pd.isna(text):
        return np.nan
    try:
        return parser.parse(str(text), dayfirst=False).year
    except:
        try:
            return parser.parse(str(text), dayfirst=True).year
        except:
            return np.nan

train["event_year_first"] = train["First event in timeline"].apply(safe_parse_year)
train["event_year_last"] = train["Last event in timeline"].apply(safe_parse_year)

test["event_year_first"] = test["First event in timeline"].apply(safe_parse_year)
test["event_year_last"] = test["Last event in timeline"].apply(safe_parse_year)


In [ ]:
# === Check types and summary stats ===
print("🔍 Column types:\n", train[["event_year_first", "event_year_last"]].dtypes)

print("\n📊 Descriptive stats for train:")
print(train[["event_year_first", "event_year_last"]].describe())

print("\n📊 Descriptive stats for test:")
print(test[["event_year_first", "event_year_last"]].describe())

# === Check value counts (frequency) ===
print("\n🗓 Top 10 most frequent 'event_year_first':")
print(train["event_year_first"].value_counts(dropna=False).sort_index().tail(10))

print("\n🗓 Top 10 most frequent 'event_year_last':")
print(train["event_year_last"].value_counts(dropna=False).sort_index().tail(10))

# Show first 20 rows for event_year_first and event_year_last in training set
print("📄 First 20 rows (Training Data):")
print(train[["event_year_first", "event_year_last"]].head(20))

# Show first 20 rows for event_year_first and event_year_last in testing set
print("\n📄 First 20 rows (Testing Data):")
print(test[["event_year_first", "event_year_last"]].head(20))


**[b] --- --- --- Dealing with multiple time event within 1 variable**

In [ ]:
from dateutil import parser

def safe_parse(date_str):
    try:
        return parser.parse(date_str, dayfirst=False)
    except:
        try:
            return parser.parse(date_str, dayfirst=True)
        except:
            return None

def extract_sorted_events(row):
    types = row.get("Full timeline of events (types)")
    dates = row.get("Full timeline of events (dates)")
    
    if pd.isna(types) or pd.isna(dates):
        return None
    
    type_list = [t.strip() for t in str(types).split(";")]
    date_list = [d.strip() for d in str(dates).split(";")]
    
    if len(type_list) != len(date_list):
        return None  # mismatched length
    
    paired = [(t, safe_parse(d)) for t, d in zip(type_list, date_list)]
    paired = [x for x in paired if x[1] is not None]  # drop invalid
    return sorted(paired, key=lambda x: x[1])

train["event_timeline"] = train.apply(extract_sorted_events, axis=1)
test["event_timeline"] = test.apply(extract_sorted_events, axis=1)


In [ ]:
def order_after(df, move_col, after_col):
    cols = list(df.columns)
    cols.remove(move_col)
    insert_at = cols.index(after_col) + 1
    cols.insert(insert_at, move_col)
    return df[cols]

train = order_after(train, "event_timeline", "Full timeline of events (dates)")
test = order_after(test, "event_timeline", "Full timeline of events (dates)")


In [ ]:
import numpy as np

for df in [train, test]:
    # 1. Count of amendments
    df["num_amendments"] = df["event_timeline"].apply(
        lambda x: sum(1 for t, _ in x if t == "Amended") if x else 0
    )

    # 2. First "Passed/Approved" date
    df["first_passed_date"] = df["event_timeline"].apply(
        lambda x: next((d for t, d in x if t == "Passed/Approved"), None) if x else None
    )

    # 3. Last event date (regardless of type)
    df["last_event_date"] = df["event_timeline"].apply(
        lambda x: x[-1][1] if x else None
    )

    # 4. Duration in years
    df["policy_duration_years"] = df.apply(
        lambda row: (row["last_event_date"] - row["first_passed_date"]).days / 365.25
        if row["last_event_date"] and row["first_passed_date"] else np.nan,
        axis=1
    )

print(train[["first_passed_date", "last_event_date", "num_amendments", "policy_duration_years"]].head(10))


In [ ]:
# Convert to datetime if not already
for df in [train, test]:
    df["first_passed_date"] = pd.to_datetime(df["first_passed_date"], errors="coerce")
    df["last_event_date"] = pd.to_datetime(df["last_event_date"], errors="coerce")

    # Extract year after conversion
    df["event_year_passed"] = df["first_passed_date"].dt.year
    df["event_year_last_event"] = df["last_event_date"].dt.year

# ✅ Preview
print("🔍 Sample extracted years:")
print(train[["first_passed_date", "event_year_passed", "last_event_date", "event_year_last_event"]].dropna().head())


**[c] --- --- --- Extra cleaning**

In [ ]:

# === (d) Define target and features ===
target = "policy_action"
exclude_cols = [
    "Document ID", "Document Title", "Family ID", "Family Title",
    "Full timeline of events (types)", target
]
feature_cols = [col for col in train.columns if col not in exclude_cols]

X_train = train[feature_cols].copy()
y_train = train[target]

X_test = test[feature_cols].copy()
y_test = test[target]



In [ ]:
non_informative_cols = []

for col in X_train.columns:
    if X_train[col].dtype == "object":
        # Skip if any row contains lists (unhashable)
        if X_train[col].apply(lambda x: isinstance(x, list)).any():
            continue
        # Drop if it's long text or URL
        if X_train[col].astype(str).str.len().mean() > 100 or \
           X_train[col].astype(str).str.contains("http|www|pdf|doc", case=False).mean() > 0.3:
            non_informative_cols.append(col)
        # Drop if almost every value is unique (like IDs)
        elif X_train[col].nunique(dropna=True) > 0.9 * len(X_train):
            non_informative_cols.append(col)

print("🔎 Non-informative columns to drop:\n", non_informative_cols)


In [ ]:
# Column	Why Drop?
# Family Summary	Long free-text with highly variable length — better for NLP than tabular
# Collection Description(s)	Often redundant and descriptive, not structured
# Document Content URL	Just a URL — no predictive value
# Internal Document ID	Unique ID — not informative, high-cardinality

In [ ]:

# === Drop non-use variables from both train and test ===
X_train.drop(columns=non_informative_cols, inplace=True, errors='ignore')
X_test.drop(columns=non_informative_cols, inplace=True, errors='ignore')


In [ ]:

# === (e) Convert object columns to numeric where possible ===
for df in [X_train, X_test]:
    for col in df.select_dtypes(include='object').columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')



In [ ]:

# === (f) One-hot encode selected categorical variables ===
categorical_vars = [
 "Document Type", "Topic/Response", "Sector", "Instrument",
    "Category", "Framework", "Hazard", "Author", "Author Type",
    "Geographies", "Source", "Geography ISOs"  # <-- New additions
]
categorical_vars = [col for col in categorical_vars if col in X_train.columns]
X_train = pd.get_dummies(X_train, columns=categorical_vars, dummy_na=True)
X_test = pd.get_dummies(X_test, columns=categorical_vars, dummy_na=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)


In [ ]:
# Step 1: Check remaining numeric columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
print(f"🔢 Total numeric columns: {len(numeric_cols)}")

# Step 2: Count missing values per numeric column
missing_train = X_train[numeric_cols].isna().sum()
missing_train = missing_train[missing_train > 0]

print("\n📌 Numeric columns with missing values in training set:")
print(missing_train)

# Step 3: Optional – show percentage missing
percent_missing = (missing_train / len(X_train) * 100).round(2)
print("\n📊 Percentage missing per column:")
print(percent_missing.sort_values(ascending=False))

# Step 4: Show sample rows with missing values in top offender column
if not missing_train.empty:
    top_col = missing_train.idxmax()
    print(f"\n🔎 Sample rows with missing in '{top_col}':")
    display(X_train[X_train[top_col].isna()].head())


✅ Summary of Missingness

| Column                          | Missing (%) | Reason or Type              | Action                                     |
|----------------------------------|--------------|------------------------------|---------------------------------------------|
| `Collection Title(s)`            | 100%         | Metadata, unused             | Drop                                     |
| `Collection Description(s)`      | 100%         | Metadata, unused             | Drop                                     |
| `Document Variant`               | 100%         | Metadata                     | Drop                                     |
| `Language`                       | 100%         | Non-informative              | Drop                                     |
| `First event in timeline`        | 100%         | Raw date field               | Drop (already extracted year)            |
| `Last event in timeline`         | 100%         | Raw date field               | Drop                                     |
| `Full timeline of events (dates)`| 100%         | Raw date field               | Drop                                     |
| `event_timeline`                 | 100%         | List of tuples               | Drop for ML; useful for NLP only         |
| `Date Added to System`           | 100%         | System metadata              | Drop                                     |
| `Last Modified on System`        | 100%         | System metadata              | Drop                                     |
| `Internal Family ID`             | 100%         | Internal ID                  | Drop                                     |
| `Internal Corpus ID`             | 100%         | Internal ID                  | Drop                                     |
| `Internal Collection ID(s)`      | 100%         | Internal ID                  | Drop                                     |
| `Document Role`                  | 100%         | Possibly encoded elsewhere   | Drop                                     |
| `Keyword`                        | 100%         | Free text                    | Drop                                     |


In [ ]:
cols_to_drop = [
    'Collection Title(s)', 'Collection Description(s)', 'Document Variant', 'Language',
    'First event in timeline', 'Last event in timeline', 'Full timeline of events (dates)',
    'event_timeline', 'Date Added to System', 'Last Modified on System',
    'Internal Family ID', 'Internal Corpus ID', 'Internal Collection ID(s)',
    'Document Role', 'Keyword'
]

X_train.drop(columns=[col for col in cols_to_drop if col in X_train.columns], inplace=True)
X_test.drop(columns=[col for col in cols_to_drop if col in X_test.columns], inplace=True)
print("✅ Dropped 100% missing / non-informative columns.")


In [ ]:
# === (g) Impute missing values safely ===

# Identify numeric columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Drop numeric columns with all NaNs in training set
all_nan_cols = [col for col in numeric_cols if X_train[col].isna().all()]
X_train.drop(columns=all_nan_cols, inplace=True)
X_test.drop(columns=all_nan_cols, errors='ignore', inplace=True)

# Re-identify numeric columns after dropping
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Impute if we still have numeric columns
if numeric_cols:
    imputer = SimpleImputer(strategy="mean")
    X_train[numeric_cols] = imputer.fit_transform(X_train[numeric_cols])
    X_test[numeric_cols] = imputer.transform(X_test[numeric_cols])
else:
    print("⚠️ No numeric columns left for imputation.")


# ✅ Confirm imputation worked
print("✅ Remaining numeric columns:", len(numeric_cols))
print("🔍 Any missing values left in training set?", X_train[numeric_cols].isna().any().any())
print("🔍 Any missing values left in test set?", X_test[numeric_cols].isna().any().any())

# Optional: Preview a few numeric columns
X_train[numeric_cols].head()


In [ ]:
# Drop datetime columns that is not useful
X_train.drop(columns=["first_passed_date", "last_event_date"], inplace=True, errors="ignore")
X_test.drop(columns=["first_passed_date", "last_event_date"], inplace=True, errors="ignore")

print("✅ Dropped datetime columns to fix XGBoost training issue.")


**[d] --- --- --- model**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, roc_auc_score, confusion_matrix,
    mean_squared_error, brier_score_loss
)

# === (h) Train Random Forest Model ===
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=6,
    random_state=42,
    class_weight='balanced',
    n_jobs=-1
)
rf_model.fit(X_train, y_train)
print("✅ Random Forest model trained.")


# === (i) Make Predictions ===
rf_pred = rf_model.predict(X_test)  # default threshold = 0.5
rf_proba = rf_model.predict_proba(X_test)[:, 1]  # probabilities


In [ ]:
print("🔍 Total length of y_test:", len(y_test))
print("❌ Missing values in y_test:", y_test.isna().sum())
# Show rows with NaN in y_test
missing_rows = X_test[y_test.isna()]
display(missing_rows.head())

# Optionally, show original y_test values
print("🧯 Missing y_test entries:")
print(y_test[y_test.isna()])


In [ ]:
# === (j) Filter out rows with NaN in y_test ===
valid_idx = ~y_test.isna()

y_test_clean = y_test[valid_idx]
y_pred_clean = rf_pred[valid_idx]
y_proba_clean = rf_proba[valid_idx]


In [ ]:

# Now run  evaluation
from sklearn.metrics import accuracy_score, roc_auc_score, brier_score_loss, mean_squared_error, confusion_matrix


# === (k) Evaluate ===
print(f"✅ Accuracy: {accuracy_score(y_test_clean, y_pred_clean):.3f}")
print(f"✅ AUC: {roc_auc_score(y_test_clean, y_proba_clean):.3f}")
print(f"✅ Brier Score: {brier_score_loss(y_test_clean, y_proba_clean):.4f}")
print(f"✅ MSE: {mean_squared_error(y_test_clean, y_proba_clean):.4f}")
print("✅ Confusion Matrix:\n", confusion_matrix(y_test_clean, y_pred_clean))



In [ ]:

# === (ii) Enhanced Confusion Matrix (Stata-style) ===
cm = confusion_matrix(y_test_clean, y_pred_clean, labels=[0, 1])
TN, FP, FN, TP = cm.ravel()

row_perc = cm / cm.sum(axis=1, keepdims=True) * 100
col_perc = cm / cm.sum(axis=0, keepdims=True) * 100
col_totals = cm.sum(axis=0)
grand_total = cm.sum()

print("\n+-------------------+")
print("| Key               |")
print("|-------------------|")
print("|     frequency     |")
print("|  row percentage   |")
print("| column percentage |")
print("+-------------------+\n")

print("leaver_binary |   pred_leave")
print("       ary     |     0        1     |   Total")
print("---------------+--------------------+----------")

labels = ["0", "1"]
for i, label in enumerate(labels):
    row = cm[i]
    row_pct = row_perc[i]
    col_pct = col_perc[i]
    row_total = row.sum()
    print(f"         {label} | {row[0]:10,} {row[1]:10,} | {row_total:8,}")
    print(f"           | {row_pct[0]:10.2f} {row_pct[1]:10.2f} |   100.00")
    print(f"           | {col_pct[0]:10.2f} {col_pct[1]:10.2f} |")
    print("---------------+--------------------+----------")

col_total_str = " ".join([f"{val:10,}" for val in col_totals])
col_perc_str = " ".join([f"{(val/grand_total*100):10.2f}" for val in col_totals])
print(f"     Total | {col_total_str} | {grand_total:8,}")
print(f"           | {col_perc_str} |   100.00")
print("           |    100.00     100.00 |   100.00")

# === (iii) Scalar-style metrics ===
Accuracy    = (TP + TN) / grand_total
Precision   = TP / (TP + FP) if (TP + FP) else 0.0
Recall      = TP / (TP + FN) if (TP + FN) else 0.0
F1          = 2 * (Precision * Recall) / (Precision + Recall) if (Precision + Recall) else 0.0
Specificity = TN / (TN + FP) if (TN + FP) else 0.0

print("\n📐 Scalar-style Evaluation Metrics:")
print(f"Accuracy:        {Accuracy:.8f}")
print(f"Precision:       {Precision:.8f}")
print(f"Recall:          {Recall:.8f}")
print(f"F1 Score:        {F1:.8f}")
print(f"Specificity:     {Specificity:.8f}")


In [ ]:
# === (l) Save Predictions ===
test_clean = test[valid_idx].copy()
test_clean["rf_pred"] = y_pred_clean
test_clean["rf_prob"] = y_proba_clean
test_clean.to_csv("randomforest_predictions_policyaction.csv", index=False)
print("📁 Exported predictions to: randomforest_predictions_policyaction.csv")


In [ ]:


print(len(test))           # Full test set
print(len(y_pred))         # Should match test if no filtering
print(len(y_pred_clean))   # Matches only valid_idx



**Python Code for ROC Threshold Evaluation and Export**

In [ ]:
import pandas as pd
import numpy as np

# === (1) Load final predictions ===
df = pd.read_csv("randomforest_predictions_policyaction.csv")

# === (2) Define column names
score_col = "rf_prob"         # Probabilities from XGBoost
actual_col = "policy_action"   # True label from gold test set

# === (3) Filter clean data ===
df = df[[score_col, actual_col]].dropna()
df['score_round'] = df[score_col].round(4)  # Round for consistency

# === (4) Prepare ROC table ===
thresholds = np.unique(df['score_round'])
results = []

P = (df[actual_col] == 1).sum()
N = (df[actual_col] == 0).sum()

for thresh in thresholds:
    pred = (df[score_col] > thresh).astype(int)
    TP = ((df[actual_col] == 1) & (pred == 1)).sum()
    FP = ((df[actual_col] == 0) & (pred == 1)).sum()
    FN = ((df[actual_col] == 1) & (pred == 0)).sum()
    TN = ((df[actual_col] == 0) & (pred == 0)).sum()

    TPR = TP / P if P > 0 else 0
    FPR = FP / N if N > 0 else 0
    distance = np.sqrt((1 - TPR) ** 2 + (FPR) ** 2)

    results.append({
        "cutoff": thresh,
        "TPR": TPR,
        "FPR": FPR,
        "distance": distance,
        "TP": TP,
        "FP": FP,
        "TN": TN,
        "FN": FN
    })

# === (5) Convert to DataFrame ===
roc_df = pd.DataFrame(results)

# === (6) Find best threshold ===
best_row = roc_df.loc[roc_df['distance'].idxmin()]
best_cutoff = best_row['cutoff']

print("\n🎯 Optimal Threshold Results")
print(f"Best cutoff       : {best_cutoff:.4f}")
print(f"True Positive Rate: {best_row['TPR']:.4f}")
print(f"False Positive Rate: {best_row['FPR']:.4f}")
print(f"Min Distance      : {best_row['distance']:.4f}")


In [ ]:

# === (7) Save all ROC points to CSV for review ===
roc_df.to_csv("rocpoints_randomforest.csv", index=False)
print("📁 All thresholds and ROC stats saved to: rocpoints_randomforest.csv")


In [ ]:
from sklearn.metrics import confusion_matrix

# === (1) Apply Best Cutoff ===
best_cutoff = 0.3528  # Replace with actual best threshold
df["final_pred"] = (df["rf_prob"] > best_cutoff).astype(int)

# Extract clean y_true and y_pred (already NaNs dropped earlier)
y_true = df["policy_action"]
y_pred = df["final_pred"]

# === (2) Compute Confusion Matrix ===
cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
TN, FP, FN, TP = cm.ravel()

row_perc = cm / cm.sum(axis=1, keepdims=True) * 100
col_perc = cm / cm.sum(axis=0, keepdims=True) * 100
col_totals = cm.sum(axis=0)
grand_total = cm.sum()

# === (3) Stata-style Table Output ===
print("\n+-------------------+")
print("| Key               |")
print("|-------------------|")
print("|     frequency     |")
print("|  row percentage   |")
print("| column percentage |")
print("+-------------------+\n")

print("policy_action |   predicted")
print("             |     0        1     |   Total")
print("---------------+--------------------+----------")

labels = ["0", "1"]
for i, label in enumerate(labels):
    row = cm[i]
    row_pct = row_perc[i]
    col_pct = col_perc[i]
    row_total = row.sum()
    print(f"         {label} | {row[0]:10,} {row[1]:10,} | {row_total:8,}")
    print(f"           | {row_pct[0]:10.2f} {row_pct[1]:10.2f} |   100.00")
    print(f"           | {col_pct[0]:10.2f} {col_pct[1]:10.2f} |")
    print("---------------+--------------------+----------")

col_total_str = " ".join([f"{val:10,}" for val in col_totals])
col_perc_str = " ".join([f"{(val/grand_total*100):10.2f}" for val in col_totals])
print(f"     Total | {col_total_str} | {grand_total:8,}")
print(f"           | {col_perc_str} |   100.00")
print("           |    100.00     100.00 |   100.00")

# === (4) Scalar Metrics ===
Accuracy    = (TP + TN) / grand_total
Precision   = TP / (TP + FP) if (TP + FP) else 0.0
Recall      = TP / (TP + FN) if (TP + FN) else 0.0
F1          = 2 * (Precision * Recall) / (Precision + Recall) if (Precision + Recall) else 0.0
Specificity = TN / (TN + FP) if (TN + FP) else 0.0

print("\n📐 Scalar-style Evaluation Metrics:")
print(f"Accuracy:        {Accuracy:.8f}")
print(f"Precision:       {Precision:.8f}")
print(f"Recall:          {Recall:.8f}")
print(f"F1 Score:        {F1:.8f}")
print(f"Specificity:     {Specificity:.8f}")


## 6.2. Xgboost

In [ ]:
# === (a) Import Libraries ===
import pandas as pd
import numpy as np
import xgboost as xgb
import re
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    accuracy_score, roc_auc_score, confusion_matrix,
    mean_squared_error, brier_score_loss
)

# === (b) Load Data ===
train = pd.read_csv("weak_labeled_policy_action_dataset_strict2.csv", encoding="ISO-8859-1")
test = pd.read_csv("classified_climate_policy_actions.csv", encoding="ISO-8859-1")


**[a] --- --- --- Dealing the time_event (single time: event_year_first and event_year_last)**

In [ ]:
from dateutil import parser

def safe_parse_year(text):
    if pd.isna(text):
        return np.nan
    try:
        return parser.parse(str(text), dayfirst=False).year
    except:
        try:
            return parser.parse(str(text), dayfirst=True).year
        except:
            return np.nan

train["event_year_first"] = train["First event in timeline"].apply(safe_parse_year)
train["event_year_last"] = train["Last event in timeline"].apply(safe_parse_year)

test["event_year_first"] = test["First event in timeline"].apply(safe_parse_year)
test["event_year_last"] = test["Last event in timeline"].apply(safe_parse_year)


In [ ]:
# === Check types and summary stats ===
print("🔍 Column types:\n", train[["event_year_first", "event_year_last"]].dtypes)

print("\n📊 Descriptive stats for train:")
print(train[["event_year_first", "event_year_last"]].describe())

print("\n📊 Descriptive stats for test:")
print(test[["event_year_first", "event_year_last"]].describe())

# === Check value counts (frequency) ===
print("\n🗓 Top 10 most frequent 'event_year_first':")
print(train["event_year_first"].value_counts(dropna=False).sort_index().tail(10))

print("\n🗓 Top 10 most frequent 'event_year_last':")
print(train["event_year_last"].value_counts(dropna=False).sort_index().tail(10))

# Show first 20 rows for event_year_first and event_year_last in training set
print("📄 First 20 rows (Training Data):")
print(train[["event_year_first", "event_year_last"]].head(20))

# Show first 20 rows for event_year_first and event_year_last in testing set
print("\n📄 First 20 rows (Testing Data):")
print(test[["event_year_first", "event_year_last"]].head(20))


**[b] --- --- --- Dealing with multiple time event within 1 variable**

In [ ]:
from dateutil import parser

def safe_parse(date_str):
    try:
        return parser.parse(date_str, dayfirst=False)
    except:
        try:
            return parser.parse(date_str, dayfirst=True)
        except:
            return None

def extract_sorted_events(row):
    types = row.get("Full timeline of events (types)")
    dates = row.get("Full timeline of events (dates)")
    
    if pd.isna(types) or pd.isna(dates):
        return None
    
    type_list = [t.strip() for t in str(types).split(";")]
    date_list = [d.strip() for d in str(dates).split(";")]
    
    if len(type_list) != len(date_list):
        return None  # mismatched length
    
    paired = [(t, safe_parse(d)) for t, d in zip(type_list, date_list)]
    paired = [x for x in paired if x[1] is not None]  # drop invalid
    return sorted(paired, key=lambda x: x[1])

train["event_timeline"] = train.apply(extract_sorted_events, axis=1)
test["event_timeline"] = test.apply(extract_sorted_events, axis=1)


In [ ]:
def order_after(df, move_col, after_col):
    cols = list(df.columns)
    cols.remove(move_col)
    insert_at = cols.index(after_col) + 1
    cols.insert(insert_at, move_col)
    return df[cols]

train = order_after(train, "event_timeline", "Full timeline of events (dates)")
test = order_after(test, "event_timeline", "Full timeline of events (dates)")


In [ ]:
import numpy as np

for df in [train, test]:
    # 1. Count of amendments
    df["num_amendments"] = df["event_timeline"].apply(
        lambda x: sum(1 for t, _ in x if t == "Amended") if x else 0
    )

    # 2. First "Passed/Approved" date
    df["first_passed_date"] = df["event_timeline"].apply(
        lambda x: next((d for t, d in x if t == "Passed/Approved"), None) if x else None
    )

    # 3. Last event date (regardless of type)
    df["last_event_date"] = df["event_timeline"].apply(
        lambda x: x[-1][1] if x else None
    )

    # 4. Duration in years
    df["policy_duration_years"] = df.apply(
        lambda row: (row["last_event_date"] - row["first_passed_date"]).days / 365.25
        if row["last_event_date"] and row["first_passed_date"] else np.nan,
        axis=1
    )

print(train[["first_passed_date", "last_event_date", "num_amendments", "policy_duration_years"]].head(10))


In [ ]:
# Convert to datetime if not already
for df in [train, test]:
    df["first_passed_date"] = pd.to_datetime(df["first_passed_date"], errors="coerce")
    df["last_event_date"] = pd.to_datetime(df["last_event_date"], errors="coerce")

    # Extract year after conversion
    df["event_year_passed"] = df["first_passed_date"].dt.year
    df["event_year_last_event"] = df["last_event_date"].dt.year

# ✅ Preview
print("🔍 Sample extracted years:")
print(train[["first_passed_date", "event_year_passed", "last_event_date", "event_year_last_event"]].dropna().head())


**[c] --- --- --- Extra cleaning**

In [ ]:

# === (d) Define target and features ===
target = "policy_action"
exclude_cols = [
    "Document ID", "Document Title", "Family ID", "Family Title",
    "Full timeline of events (types)", target
]
feature_cols = [col for col in train.columns if col not in exclude_cols]

X_train = train[feature_cols].copy()
y_train = train[target]

X_test = test[feature_cols].copy()
y_test = test[target]



In [ ]:
non_informative_cols = []

for col in X_train.columns:
    if X_train[col].dtype == "object":
        # Skip if any row contains lists (unhashable)
        if X_train[col].apply(lambda x: isinstance(x, list)).any():
            continue
        # Drop if it's long text or URL
        if X_train[col].astype(str).str.len().mean() > 100 or \
           X_train[col].astype(str).str.contains("http|www|pdf|doc", case=False).mean() > 0.3:
            non_informative_cols.append(col)
        # Drop if almost every value is unique (like IDs)
        elif X_train[col].nunique(dropna=True) > 0.9 * len(X_train):
            non_informative_cols.append(col)

print("🔎 Non-informative columns to drop:\n", non_informative_cols)


In [ ]:
# Column	Why Drop?
# Family Summary	Long free-text with highly variable length — better for NLP than tabular
# Collection Description(s)	Often redundant and descriptive, not structured
# Document Content URL	Just a URL — no predictive value
# Internal Document ID	Unique ID — not informative, high-cardinality

In [ ]:

# === Drop non-use variables from both train and test ===
X_train.drop(columns=non_informative_cols, inplace=True, errors='ignore')
X_test.drop(columns=non_informative_cols, inplace=True, errors='ignore')


In [ ]:

# === (e) Convert object columns to numeric where possible ===
for df in [X_train, X_test]:
    for col in df.select_dtypes(include='object').columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')



In [ ]:

# === (f) One-hot encode selected categorical variables ===
categorical_vars = [
 "Document Type", "Topic/Response", "Sector", "Instrument",
    "Category", "Framework", "Hazard", "Author", "Author Type",
    "Geographies", "Source", "Geography ISOs"  # <-- New additions
]
categorical_vars = [col for col in categorical_vars if col in X_train.columns]
X_train = pd.get_dummies(X_train, columns=categorical_vars, dummy_na=True)
X_test = pd.get_dummies(X_test, columns=categorical_vars, dummy_na=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)


In [ ]:
# Step 1: Check remaining numeric columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
print(f"🔢 Total numeric columns: {len(numeric_cols)}")

# Step 2: Count missing values per numeric column
missing_train = X_train[numeric_cols].isna().sum()
missing_train = missing_train[missing_train > 0]

print("\n📌 Numeric columns with missing values in training set:")
print(missing_train)

# Step 3: Optional – show percentage missing
percent_missing = (missing_train / len(X_train) * 100).round(2)
print("\n📊 Percentage missing per column:")
print(percent_missing.sort_values(ascending=False))

# Step 4: Show sample rows with missing values in top offender column
if not missing_train.empty:
    top_col = missing_train.idxmax()
    print(f"\n🔎 Sample rows with missing in '{top_col}':")
    display(X_train[X_train[top_col].isna()].head())


✅ Summary of Missingness

| Column                          | Missing (%) | Reason or Type              | Action                                     |
|----------------------------------|--------------|------------------------------|---------------------------------------------|
| `Collection Title(s)`            | 100%         | Metadata, unused             | Drop                                     |
| `Collection Description(s)`      | 100%         | Metadata, unused             | Drop                                     |
| `Document Variant`               | 100%         | Metadata                     | Drop                                     |
| `Language`                       | 100%         | Non-informative              | Drop                                     |
| `First event in timeline`        | 100%         | Raw date field               | Drop (already extracted year)            |
| `Last event in timeline`         | 100%         | Raw date field               | Drop                                     |
| `Full timeline of events (dates)`| 100%         | Raw date field               | Drop                                     |
| `event_timeline`                 | 100%         | List of tuples               | Drop for ML; useful for NLP only         |
| `Date Added to System`           | 100%         | System metadata              | Drop                                     |
| `Last Modified on System`        | 100%         | System metadata              | Drop                                     |
| `Internal Family ID`             | 100%         | Internal ID                  | Drop                                     |
| `Internal Corpus ID`             | 100%         | Internal ID                  | Drop                                     |
| `Internal Collection ID(s)`      | 100%         | Internal ID                  | Drop                                     |
| `Document Role`                  | 100%         | Possibly encoded elsewhere   | Drop                                     |
| `Keyword`                        | 100%         | Free text                    | Drop                                     |


In [ ]:
cols_to_drop = [
    'Collection Title(s)', 'Collection Description(s)', 'Document Variant', 'Language',
    'First event in timeline', 'Last event in timeline', 'Full timeline of events (dates)',
    'event_timeline', 'Date Added to System', 'Last Modified on System',
    'Internal Family ID', 'Internal Corpus ID', 'Internal Collection ID(s)',
    'Document Role', 'Keyword'
]

X_train.drop(columns=[col for col in cols_to_drop if col in X_train.columns], inplace=True)
X_test.drop(columns=[col for col in cols_to_drop if col in X_test.columns], inplace=True)
print("✅ Dropped 100% missing / non-informative columns.")


In [ ]:
# === (g) Impute missing values safely ===

# Identify numeric columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Drop numeric columns with all NaNs in training set
all_nan_cols = [col for col in numeric_cols if X_train[col].isna().all()]
X_train.drop(columns=all_nan_cols, inplace=True)
X_test.drop(columns=all_nan_cols, errors='ignore', inplace=True)

# Re-identify numeric columns after dropping
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Impute if we still have numeric columns
if numeric_cols:
    imputer = SimpleImputer(strategy="mean")
    X_train[numeric_cols] = imputer.fit_transform(X_train[numeric_cols])
    X_test[numeric_cols] = imputer.transform(X_test[numeric_cols])
else:
    print("⚠️ No numeric columns left for imputation.")


# ✅ Confirm imputation worked
print("✅ Remaining numeric columns:", len(numeric_cols))
print("🔍 Any missing values left in training set?", X_train[numeric_cols].isna().any().any())
print("🔍 Any missing values left in test set?", X_test[numeric_cols].isna().any().any())

# Optional: Preview a few numeric columns
X_train[numeric_cols].head()


In [ ]:
# Drop datetime columns that XGBoost cannot handle
X_train.drop(columns=["first_passed_date", "last_event_date"], inplace=True, errors="ignore")
X_test.drop(columns=["first_passed_date", "last_event_date"], inplace=True, errors="ignore")

print("✅ Dropped datetime columns to fix XGBoost training issue.")


**[d] --- --- --- model**

In [ ]:

# === (h) Train XGBoost Model ===
model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.2,
    n_jobs=-1,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)
model.fit(X_train, y_train)

# === (i) Predict and Evaluate ===
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

In [ ]:
print("🔍 Total length of y_test:", len(y_test))
print("❌ Missing values in y_test:", y_test.isna().sum())
# Show rows with NaN in y_test
missing_rows = X_test[y_test.isna()]
display(missing_rows.head())

# Optionally, show original y_test values
print("🧯 Missing y_test entries:")
print(y_test[y_test.isna()])


In [ ]:
# Filter out rows where y_test is NaN
valid_idx = ~y_test.isna()

# Filter both y_test and y_pred / y_proba accordingly
y_test_clean = y_test[valid_idx]
y_pred_clean = y_pred[valid_idx]
y_proba_clean = y_proba[valid_idx]


In [ ]:

# Now run evaluation
from sklearn.metrics import accuracy_score, roc_auc_score, brier_score_loss, mean_squared_error, confusion_matrix

print(f"✅ Accuracy: {accuracy_score(y_test_clean, y_pred_clean):.3f}")
print(f"✅ AUC: {roc_auc_score(y_test_clean, y_proba_clean):.3f}")
print(f"✅ Brier Score: {brier_score_loss(y_test_clean, y_proba_clean):.4f}")
print(f"✅ MSE: {mean_squared_error(y_test_clean, y_proba_clean):.4f}")
print("✅ Confusion Matrix:\n", confusion_matrix(y_test_clean, y_pred_clean))

In [ ]:

# === (ii) Enhanced Confusion Matrix (Stata-style) ===
cm = confusion_matrix(y_test_clean, y_pred_clean, labels=[0, 1])
TN, FP, FN, TP = cm.ravel()

row_perc = cm / cm.sum(axis=1, keepdims=True) * 100
col_perc = cm / cm.sum(axis=0, keepdims=True) * 100
col_totals = cm.sum(axis=0)
grand_total = cm.sum()

print("\n+-------------------+")
print("| Key               |")
print("|-------------------|")
print("|     frequency     |")
print("|  row percentage   |")
print("| column percentage |")
print("+-------------------+\n")

print("leaver_binary |   pred_leave")
print("       ary     |     0        1     |   Total")
print("---------------+--------------------+----------")

labels = ["0", "1"]
for i, label in enumerate(labels):
    row = cm[i]
    row_pct = row_perc[i]
    col_pct = col_perc[i]
    row_total = row.sum()
    print(f"         {label} | {row[0]:10,} {row[1]:10,} | {row_total:8,}")
    print(f"           | {row_pct[0]:10.2f} {row_pct[1]:10.2f} |   100.00")
    print(f"           | {col_pct[0]:10.2f} {col_pct[1]:10.2f} |")
    print("---------------+--------------------+----------")

col_total_str = " ".join([f"{val:10,}" for val in col_totals])
col_perc_str = " ".join([f"{(val/grand_total*100):10.2f}" for val in col_totals])
print(f"     Total | {col_total_str} | {grand_total:8,}")
print(f"           | {col_perc_str} |   100.00")
print("           |    100.00     100.00 |   100.00")

# === (iii) Scalar-style metrics ===
Accuracy    = (TP + TN) / grand_total
Precision   = TP / (TP + FP) if (TP + FP) else 0.0
Recall      = TP / (TP + FN) if (TP + FN) else 0.0
F1          = 2 * (Precision * Recall) / (Precision + Recall) if (Precision + Recall) else 0.0
Specificity = TN / (TN + FP) if (TN + FP) else 0.0

print("\n📐 Scalar-style Evaluation Metrics:")
print(f"Accuracy:        {Accuracy:.8f}")
print(f"Precision:       {Precision:.8f}")
print(f"Recall:          {Recall:.8f}")
print(f"F1 Score:        {F1:.8f}")
print(f"Specificity:     {Specificity:.8f}")


In [ ]:
# === (j) Save predictions ===
test_clean = test[valid_idx].copy()
test_clean["xgb_pred"] = y_pred_clean
test_clean["xgb_prob"] = y_proba_clean
test_clean.to_csv("xgboost_predictions_policyaction.csv", index=False)


In [ ]:


print(len(test))           # Full test set
print(len(y_pred))         # Should match test if no filtering
print(len(y_pred_clean))   # Matches only valid_idx



**Python Code for ROC Threshold Evaluation and Export**

In [ ]:
import pandas as pd
import numpy as np

# === (1) Load final predictions ===
df = pd.read_csv("xgboost_predictions_policyaction.csv")

# === (2) Define column names
score_col = "xgb_prob"         # Probabilities from XGBoost
actual_col = "policy_action"   # True label from gold test set

# === (3) Filter clean data ===
df = df[[score_col, actual_col]].dropna()
df['score_round'] = df[score_col].round(4)  # Round for consistency

# === (4) Prepare ROC table ===
thresholds = np.unique(df['score_round'])
results = []

P = (df[actual_col] == 1).sum()
N = (df[actual_col] == 0).sum()

for thresh in thresholds:
    pred = (df[score_col] > thresh).astype(int)
    TP = ((df[actual_col] == 1) & (pred == 1)).sum()
    FP = ((df[actual_col] == 0) & (pred == 1)).sum()
    FN = ((df[actual_col] == 1) & (pred == 0)).sum()
    TN = ((df[actual_col] == 0) & (pred == 0)).sum()

    TPR = TP / P if P > 0 else 0
    FPR = FP / N if N > 0 else 0
    distance = np.sqrt((1 - TPR) ** 2 + (FPR) ** 2)

    results.append({
        "cutoff": thresh,
        "TPR": TPR,
        "FPR": FPR,
        "distance": distance,
        "TP": TP,
        "FP": FP,
        "TN": TN,
        "FN": FN
    })

# === (5) Convert to DataFrame ===
roc_df = pd.DataFrame(results)

# === (6) Find best threshold ===
best_row = roc_df.loc[roc_df['distance'].idxmin()]
best_cutoff = best_row['cutoff']

print("\n🎯 Optimal Threshold Results")
print(f"Best cutoff       : {best_cutoff:.4f}")
print(f"True Positive Rate: {best_row['TPR']:.4f}")
print(f"False Positive Rate: {best_row['FPR']:.4f}")
print(f"Min Distance      : {best_row['distance']:.4f}")


In [ ]:

# === (7) Save all ROC points to CSV for review ===
roc_df.to_csv("rocpoints_xgboost.csv", index=False)
print("📁 All thresholds and ROC stats saved to: rocpoints_xgboost.csv")


In [ ]:
from sklearn.metrics import confusion_matrix

# === (1) Apply Best Cutoff ===
best_cutoff = 0.2107  # Replace with actual best threshold
df["final_pred"] = (df["xgb_prob"] > best_cutoff).astype(int)

# Extract clean y_true and y_pred (already NaNs dropped earlier)
y_true = df["policy_action"]
y_pred = df["final_pred"]

# === (2) Compute Confusion Matrix ===
cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
TN, FP, FN, TP = cm.ravel()

row_perc = cm / cm.sum(axis=1, keepdims=True) * 100
col_perc = cm / cm.sum(axis=0, keepdims=True) * 100
col_totals = cm.sum(axis=0)
grand_total = cm.sum()

# === (3) Stata-style Table Output ===
print("\n+-------------------+")
print("| Key               |")
print("|-------------------|")
print("|     frequency     |")
print("|  row percentage   |")
print("| column percentage |")
print("+-------------------+\n")

print("policy_action |   predicted")
print("             |     0        1     |   Total")
print("---------------+--------------------+----------")

labels = ["0", "1"]
for i, label in enumerate(labels):
    row = cm[i]
    row_pct = row_perc[i]
    col_pct = col_perc[i]
    row_total = row.sum()
    print(f"         {label} | {row[0]:10,} {row[1]:10,} | {row_total:8,}")
    print(f"           | {row_pct[0]:10.2f} {row_pct[1]:10.2f} |   100.00")
    print(f"           | {col_pct[0]:10.2f} {col_pct[1]:10.2f} |")
    print("---------------+--------------------+----------")

col_total_str = " ".join([f"{val:10,}" for val in col_totals])
col_perc_str = " ".join([f"{(val/grand_total*100):10.2f}" for val in col_totals])
print(f"     Total | {col_total_str} | {grand_total:8,}")
print(f"           | {col_perc_str} |   100.00")
print("           |    100.00     100.00 |   100.00")

# === (4) Scalar Metrics ===
Accuracy    = (TP + TN) / grand_total
Precision   = TP / (TP + FP) if (TP + FP) else 0.0
Recall      = TP / (TP + FN) if (TP + FN) else 0.0
F1          = 2 * (Precision * Recall) / (Precision + Recall) if (Precision + Recall) else 0.0
Specificity = TN / (TN + FP) if (TN + FP) else 0.0

print("\n📐 Scalar-style Evaluation Metrics:")
print(f"Accuracy:        {Accuracy:.8f}")
print(f"Precision:       {Precision:.8f}")
print(f"Recall:          {Recall:.8f}")
print(f"F1 Score:        {F1:.8f}")
print(f"Specificity:     {Specificity:.8f}")


In [ ]:
import pandas as pd

# Combine test metadata with model predictions
results_df = X_test.copy()
results_df['actual'] = y_test
results_df['predicted'] = y_pred

# Add metadata columns from original test set
results_df['Document ID'] = test['Document ID']
results_df['Family ID'] = test['Family ID']
results_df['Document Type'] = test['Document Type']

# Identify False Positives: predicted 1, actual 0
false_positives = results_df[(results_df['predicted'] == 1) & (results_df['actual'] == 0)]

# Identify False Negatives: predicted 0, actual 1
false_negatives = results_df[(results_df['predicted'] == 0) & (results_df['actual'] == 1)]

# Save results with identifying columns
false_positives.to_csv("false_positives_with_IDs.csv", index=False)
false_negatives.to_csv("false_negatives_with_IDs.csv", index=False)

# Print summaries
print("False Positives (Predicted 1, Actual 0):", false_positives.shape[0])
print("False Negatives (Predicted 0, Actual 1):", false_negatives.shape[0])


In [ ]:
false_positives.to_csv("false_positives.csv", index=False)
false_negatives.to_csv("false_negatives.csv", index=False)


## 6.3. Neural network

In [ ]:
# === (a) Import Libraries ===
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    accuracy_score, roc_auc_score, confusion_matrix,
    mean_squared_error, brier_score_loss
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler

# === (b) Load Data ===
train = pd.read_csv("weak_labeled_policy_action_dataset_strict2.csv", encoding="ISO-8859-1")
test = pd.read_csv("classified_climate_policy_actions.csv", encoding="ISO-8859-1")


**[a] --- --- --- Dealing the time_event (single time: event_year_first and event_year_last)**

In [ ]:
from dateutil import parser

def safe_parse_year(text):
    if pd.isna(text):
        return np.nan
    try:
        return parser.parse(str(text), dayfirst=False).year
    except:
        try:
            return parser.parse(str(text), dayfirst=True).year
        except:
            return np.nan

train["event_year_first"] = train["First event in timeline"].apply(safe_parse_year)
train["event_year_last"] = train["Last event in timeline"].apply(safe_parse_year)

test["event_year_first"] = test["First event in timeline"].apply(safe_parse_year)
test["event_year_last"] = test["Last event in timeline"].apply(safe_parse_year)


In [ ]:
# === Check types and summary stats ===
print("🔍 Column types:\n", train[["event_year_first", "event_year_last"]].dtypes)

print("\n📊 Descriptive stats for train:")
print(train[["event_year_first", "event_year_last"]].describe())

print("\n📊 Descriptive stats for test:")
print(test[["event_year_first", "event_year_last"]].describe())

# === Check value counts (frequency) ===
print("\n🗓 Top 10 most frequent 'event_year_first':")
print(train["event_year_first"].value_counts(dropna=False).sort_index().tail(10))

print("\n🗓 Top 10 most frequent 'event_year_last':")
print(train["event_year_last"].value_counts(dropna=False).sort_index().tail(10))

# Show first 20 rows for event_year_first and event_year_last in training set
print("📄 First 20 rows (Training Data):")
print(train[["event_year_first", "event_year_last"]].head(20))

# Show first 20 rows for event_year_first and event_year_last in testing set
print("\n📄 First 20 rows (Testing Data):")
print(test[["event_year_first", "event_year_last"]].head(20))


**[b] --- --- --- Dealing with multiple time event within 1 variable**

In [ ]:
from dateutil import parser

def safe_parse(date_str):
    try:
        return parser.parse(date_str, dayfirst=False)
    except:
        try:
            return parser.parse(date_str, dayfirst=True)
        except:
            return None

def extract_sorted_events(row):
    types = row.get("Full timeline of events (types)")
    dates = row.get("Full timeline of events (dates)")
    
    if pd.isna(types) or pd.isna(dates):
        return None
    
    type_list = [t.strip() for t in str(types).split(";")]
    date_list = [d.strip() for d in str(dates).split(";")]
    
    if len(type_list) != len(date_list):
        return None  # mismatched length
    
    paired = [(t, safe_parse(d)) for t, d in zip(type_list, date_list)]
    paired = [x for x in paired if x[1] is not None]  # drop invalid
    return sorted(paired, key=lambda x: x[1])

train["event_timeline"] = train.apply(extract_sorted_events, axis=1)
test["event_timeline"] = test.apply(extract_sorted_events, axis=1)


In [ ]:
def order_after(df, move_col, after_col):
    cols = list(df.columns)
    cols.remove(move_col)
    insert_at = cols.index(after_col) + 1
    cols.insert(insert_at, move_col)
    return df[cols]

train = order_after(train, "event_timeline", "Full timeline of events (dates)")
test = order_after(test, "event_timeline", "Full timeline of events (dates)")


In [ ]:
import numpy as np

for df in [train, test]:
    # 1. Count of amendments
    df["num_amendments"] = df["event_timeline"].apply(
        lambda x: sum(1 for t, _ in x if t == "Amended") if x else 0
    )

    # 2. First "Passed/Approved" date
    df["first_passed_date"] = df["event_timeline"].apply(
        lambda x: next((d for t, d in x if t == "Passed/Approved"), None) if x else None
    )

    # 3. Last event date (regardless of type)
    df["last_event_date"] = df["event_timeline"].apply(
        lambda x: x[-1][1] if x else None
    )

    # 4. Duration in years
    df["policy_duration_years"] = df.apply(
        lambda row: (row["last_event_date"] - row["first_passed_date"]).days / 365.25
        if row["last_event_date"] and row["first_passed_date"] else np.nan,
        axis=1
    )

print(train[["first_passed_date", "last_event_date", "num_amendments", "policy_duration_years"]].head(10))


In [ ]:
# Convert to datetime if not already
for df in [train, test]:
    df["first_passed_date"] = pd.to_datetime(df["first_passed_date"], errors="coerce")
    df["last_event_date"] = pd.to_datetime(df["last_event_date"], errors="coerce")

    # Extract year after conversion
    df["event_year_passed"] = df["first_passed_date"].dt.year
    df["event_year_last_event"] = df["last_event_date"].dt.year

# ✅ Preview
print("🔍 Sample extracted years:")
print(train[["first_passed_date", "event_year_passed", "last_event_date", "event_year_last_event"]].dropna().head())


**[c] --- --- --- Extra cleaning**

In [ ]:

# === (d) Define target and features ===
target = "policy_action"
exclude_cols = [
    "Document ID", "Document Title", "Family ID", "Family Title",
    "Full timeline of events (types)", target
]
feature_cols = [col for col in train.columns if col not in exclude_cols]

X_train = train[feature_cols].copy()
y_train = train[target]

X_test = test[feature_cols].copy()
y_test = test[target]



In [ ]:
non_informative_cols = []

for col in X_train.columns:
    if X_train[col].dtype == "object":
        # Skip if any row contains lists (unhashable)
        if X_train[col].apply(lambda x: isinstance(x, list)).any():
            continue
        # Drop if it's long text or URL
        if X_train[col].astype(str).str.len().mean() > 100 or \
           X_train[col].astype(str).str.contains("http|www|pdf|doc", case=False).mean() > 0.3:
            non_informative_cols.append(col)
        # Drop if almost every value is unique (like IDs)
        elif X_train[col].nunique(dropna=True) > 0.9 * len(X_train):
            non_informative_cols.append(col)

print("🔎 Non-informative columns to drop:\n", non_informative_cols)


In [ ]:
# Column	Why Drop?
# Family Summary	Long free-text with highly variable length — better for NLP than tabular
# Collection Description(s)	Often redundant and descriptive, not structured
# Document Content URL	Just a URL — no predictive value
# Internal Document ID	Unique ID — not informative, high-cardinality

In [ ]:

# === Drop non-use variables from both train and test ===
X_train.drop(columns=non_informative_cols, inplace=True, errors='ignore')
X_test.drop(columns=non_informative_cols, inplace=True, errors='ignore')


In [ ]:

# === (e) Convert object columns to numeric where possible ===
for df in [X_train, X_test]:
    for col in df.select_dtypes(include='object').columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')



In [ ]:

# === (f) One-hot encode selected categorical variables ===
categorical_vars = [
 "Document Type", "Topic/Response", "Sector", "Instrument",
    "Category", "Framework", "Hazard", "Author", "Author Type",
    "Geographies", "Source", "Geography ISOs"  # <-- New additions
]
categorical_vars = [col for col in categorical_vars if col in X_train.columns]
X_train = pd.get_dummies(X_train, columns=categorical_vars, dummy_na=True)
X_test = pd.get_dummies(X_test, columns=categorical_vars, dummy_na=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)


In [ ]:
# Step 1: Check remaining numeric columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
print(f"🔢 Total numeric columns: {len(numeric_cols)}")

# Step 2: Count missing values per numeric column
missing_train = X_train[numeric_cols].isna().sum()
missing_train = missing_train[missing_train > 0]

print("\n📌 Numeric columns with missing values in training set:")
print(missing_train)

# Step 3: Optional – show percentage missing
percent_missing = (missing_train / len(X_train) * 100).round(2)
print("\n📊 Percentage missing per column:")
print(percent_missing.sort_values(ascending=False))

# Step 4: Show sample rows with missing values in top offender column
if not missing_train.empty:
    top_col = missing_train.idxmax()
    print(f"\n🔎 Sample rows with missing in '{top_col}':")
    display(X_train[X_train[top_col].isna()].head())


✅ Summary of Missingness

| Column                          | Missing (%) | Reason or Type              | Action                                     |
|----------------------------------|--------------|------------------------------|---------------------------------------------|
| `Collection Title(s)`            | 100%         | Metadata, unused             | Drop                                     |
| `Collection Description(s)`      | 100%         | Metadata, unused             | Drop                                     |
| `Document Variant`               | 100%         | Metadata                     | Drop                                     |
| `Language`                       | 100%         | Non-informative              | Drop                                     |
| `First event in timeline`        | 100%         | Raw date field               | Drop (already extracted year)            |
| `Last event in timeline`         | 100%         | Raw date field               | Drop                                     |
| `Full timeline of events (dates)`| 100%         | Raw date field               | Drop                                     |
| `event_timeline`                 | 100%         | List of tuples               | Drop for ML; useful for NLP only         |
| `Date Added to System`           | 100%         | System metadata              | Drop                                     |
| `Last Modified on System`        | 100%         | System metadata              | Drop                                     |
| `Internal Family ID`             | 100%         | Internal ID                  | Drop                                     |
| `Internal Corpus ID`             | 100%         | Internal ID                  | Drop                                     |
| `Internal Collection ID(s)`      | 100%         | Internal ID                  | Drop                                     |
| `Document Role`                  | 100%         | Possibly encoded elsewhere   | Drop                                     |
| `Keyword`                        | 100%         | Free text                    | Drop                                     |


In [ ]:
cols_to_drop = [
    'Collection Title(s)', 'Collection Description(s)', 'Document Variant', 'Language',
    'First event in timeline', 'Last event in timeline', 'Full timeline of events (dates)',
    'event_timeline', 'Date Added to System', 'Last Modified on System',
    'Internal Family ID', 'Internal Corpus ID', 'Internal Collection ID(s)',
    'Document Role', 'Keyword'
]

X_train.drop(columns=[col for col in cols_to_drop if col in X_train.columns], inplace=True)
X_test.drop(columns=[col for col in cols_to_drop if col in X_test.columns], inplace=True)
print("✅ Dropped 100% missing / non-informative columns.")


In [ ]:
# === (g) Impute missing values safely ===

# Identify numeric columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Drop numeric columns with all NaNs in training set
all_nan_cols = [col for col in numeric_cols if X_train[col].isna().all()]
X_train.drop(columns=all_nan_cols, inplace=True)
X_test.drop(columns=all_nan_cols, errors='ignore', inplace=True)

# Re-identify numeric columns after dropping
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Impute if we still have numeric columns
if numeric_cols:
    imputer = SimpleImputer(strategy="mean")
    X_train[numeric_cols] = imputer.fit_transform(X_train[numeric_cols])
    X_test[numeric_cols] = imputer.transform(X_test[numeric_cols])
else:
    print("⚠️ No numeric columns left for imputation.")


# ✅ Confirm imputation worked
print("✅ Remaining numeric columns:", len(numeric_cols))
print("🔍 Any missing values left in training set?", X_train[numeric_cols].isna().any().any())
print("🔍 Any missing values left in test set?", X_test[numeric_cols].isna().any().any())

# Optional: Preview a few numeric columns
X_train[numeric_cols].head()


In [ ]:
# Drop datetime columns that is not useful
X_train.drop(columns=["first_passed_date", "last_event_date"], inplace=True, errors="ignore")
X_test.drop(columns=["first_passed_date", "last_event_date"], inplace=True, errors="ignore")

print("✅ Dropped datetime columns to fix XGBoost training issue.")


**[d] --- --- --- Extra cleaning before Neural network**


In [ ]:

# === (j) Normalize features ===
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

**[e] --- --- --- Model**

In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
# MLPClassifier - based pipeline (align with simple basic RF & xgboost) - dont use
from sklearn.neural_network import MLPClassifier

nn_model = MLPClassifier(
    hidden_layer_sizes=(100,),   # One hidden layer with 100 neurons
    activation='relu',           # ReLU activation
    solver='adam',               # Adaptive gradient descent
    alpha=0.0001,                # L2 regularization
    learning_rate='adaptive',    # Adjusts learning rate based on validation loss
    max_iter=500,                # Enough iterations to converge
    random_state=42
)


In [ ]:
# Keras deep neural network (DNN) 
# === (k) Define and train Neural Network ===
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),  
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=100, batch_size=64, callbacks=[early_stop], verbose=1)


In [ ]:
# === (l) Predict ===

# ✅ Step 1: Identify valid test indices (non-missing labels)
valid_idx = ~y_test.isna()

# ✅ Step 2: Filter out missing test labels
y_test_clean = y_test[valid_idx].astype(int)
X_test_clean = X_test_scaled[valid_idx]

# ✅ Step 3: Predict probabilities and class labels
y_proba = model.predict(X_test_clean, verbose=0).flatten()  # Get predicted probabilities
y_pred = (y_proba > 0.5).astype(int)                         # Default threshold: 0.5


In [ ]:
# ✅ Step 1: Diagnose test set
print("🔍 Total length of y_test:", len(y_test))
print("❌ Missing values in y_test:", y_test.isna().sum())

# Optional: show problematic test rows (i.e., where no label is available)
missing_rows = X_test[y_test.isna()]
display(missing_rows.head())

print("🧯 Missing y_test entries:")
print(y_test[y_test.isna()])



In [ ]:
# ✅ Step 2: Filter out NaN labels from y_test only
valid_idx = ~y_test.isna()

y_test_clean = y_test[valid_idx].astype(int)  # Ensure binary labels for evaluation
y_pred_clean = y_pred             # Already of length 200
y_proba_clean = y_proba           # Already of length 200


In [ ]:
# === (m) Evaluate Neural Network Model ===
print(f"✅ Accuracy:     {accuracy_score(y_test_clean, y_pred):.3f}")
print(f"✅ AUC:          {roc_auc_score(y_test_clean, y_proba):.3f}")
print(f"✅ Brier Score:  {brier_score_loss(y_test_clean, y_proba):.4f}")
print(f"✅ MSE:          {mean_squared_error(y_test_clean, y_proba):.4f}")

cm = confusion_matrix(y_test_clean, y_pred)
print("✅ Confusion Matrix:\n", cm)

# Optional: Unpack and display confusion matrix in labeled form
TN, FP, FN, TP = cm.ravel()
print(f"   TN: {TN}, FP: {FP}, FN: {FN}, TP: {TP}")



In [ ]:
# === (ii) Enhanced Confusion Matrix (Stata-style) ===
cm = confusion_matrix(y_test_clean, y_pred_clean, labels=[0, 1])
TN, FP, FN, TP = cm.ravel()

row_perc = cm / cm.sum(axis=1, keepdims=True) * 100
col_perc = cm / cm.sum(axis=0, keepdims=True) * 100
col_totals = cm.sum(axis=0)
grand_total = cm.sum()

print("\n+-------------------+")
print("| Key               |")
print("|-------------------|")
print("|     frequency     |")
print("|  row percentage   |")
print("| column percentage |")
print("+-------------------+\n")

print("policy_action |   predicted")
print("              |     0        1     |   Total")
print("--------------+--------------------+----------")

labels = ["0", "1"]
for i, label in enumerate(labels):
    row = cm[i]
    row_pct = row_perc[i]
    col_pct = col_perc[i]
    row_total = row.sum()
    print(f"         {label} | {row[0]:10,} {row[1]:10,} | {row_total:8,}")
    print(f"              | {row_pct[0]:10.2f} {row_pct[1]:10.2f} |   100.00")
    print(f"              | {col_pct[0]:10.2f} {col_pct[1]:10.2f} |")
    print("--------------+--------------------+----------")

col_total_str = " ".join([f"{val:10,}" for val in col_totals])
col_perc_str = " ".join([f"{(val/grand_total*100):10.2f}" for val in col_totals])
print(f"     Total   | {col_total_str} | {grand_total:8,}")
print(f"              | {col_perc_str} |   100.00")
print("              |    100.00     100.00 |   100.00")

# === (iii) Scalar-style metrics ===
Accuracy    = (TP + TN) / grand_total
Precision   = TP / (TP + FP) if (TP + FP) else 0.0
Recall      = TP / (TP + FN) if (TP + FN) else 0.0
F1          = 2 * (Precision * Recall) / (Precision + Recall) if (Precision + Recall) else 0.0
Specificity = TN / (TN + FP) if (TN + FP) else 0.0

print("\n📐 Scalar-style Evaluation Metrics:")
print(f"Accuracy:        {Accuracy:.8f}")
print(f"Precision:       {Precision:.8f}")
print(f"Recall:          {Recall:.8f}")
print(f"F1 Score:        {F1:.8f}")
print(f"Specificity:     {Specificity:.8f}")


In [ ]:
# === (n) Save Results ===
test_clean = test.loc[valid_idx].copy()
test_clean["nn_pred"] = y_pred
test_clean["nn_prob"] = y_proba
test_clean.to_csv("neuralnet_predictions_policyaction.csv", index=False)
print("📁 Predictions saved to neuralnet_predictions_policyaction.csv")


**Python Code for ROC Threshold Evaluation and Export**

In [ ]:
import pandas as pd
import numpy as np

# === (1) Load final predictions ===
df = pd.read_csv("neuralnet_predictions_policyaction.csv")

# === (2) Define column names
score_col = "nn_prob"         # Probabilities from XGBoost
actual_col = "policy_action"   # True label from gold test set

# === (3) Filter clean data ===
df = df[[score_col, actual_col]].dropna()
df['score_round'] = df[score_col].round(4)  # Round for consistency

# === (4) Prepare ROC table ===
thresholds = np.unique(df['score_round'])
results = []

P = (df[actual_col] == 1).sum()
N = (df[actual_col] == 0).sum()

for thresh in thresholds:
    pred = (df[score_col] > thresh).astype(int)
    TP = ((df[actual_col] == 1) & (pred == 1)).sum()
    FP = ((df[actual_col] == 0) & (pred == 1)).sum()
    FN = ((df[actual_col] == 1) & (pred == 0)).sum()
    TN = ((df[actual_col] == 0) & (pred == 0)).sum()

    TPR = TP / P if P > 0 else 0
    FPR = FP / N if N > 0 else 0
    distance = np.sqrt((1 - TPR) ** 2 + (FPR) ** 2)

    results.append({
        "cutoff": thresh,
        "TPR": TPR,
        "FPR": FPR,
        "distance": distance,
        "TP": TP,
        "FP": FP,
        "TN": TN,
        "FN": FN
    })

# === (5) Convert to DataFrame ===
roc_df = pd.DataFrame(results)

# === (6) Find best threshold ===
best_row = roc_df.loc[roc_df['distance'].idxmin()]
best_cutoff = best_row['cutoff']

print("\n🎯 Optimal Threshold Results")
print(f"Best cutoff       : {best_cutoff:.4f}")
print(f"True Positive Rate: {best_row['TPR']:.4f}")
print(f"False Positive Rate: {best_row['FPR']:.4f}")
print(f"Min Distance      : {best_row['distance']:.4f}")


In [ ]:

# === (7) Save all ROC points to CSV for review ===
roc_df.to_csv("rocpoints_neuralnet.csv", index=False)
print("📁 All thresholds and ROC stats saved to: rocpoints_neuralnet.csv")


In [ ]:
from sklearn.metrics import confusion_matrix

# === (1) Apply Best Cutoff ===
best_cutoff =  0.2518  # Replace with  actual best threshold
df["final_pred"] = (df["nn_prob"] > best_cutoff).astype(int)

# Extract clean y_true and y_pred (already NaNs dropped earlier)
y_true = df["policy_action"]
y_pred = df["final_pred"]

# === (2) Compute Confusion Matrix ===
cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
TN, FP, FN, TP = cm.ravel()

row_perc = cm / cm.sum(axis=1, keepdims=True) * 100
col_perc = cm / cm.sum(axis=0, keepdims=True) * 100
col_totals = cm.sum(axis=0)
grand_total = cm.sum()

# === (3) Stata-style Table Output ===
print("\n+-------------------+")
print("| Key               |")
print("|-------------------|")
print("|     frequency     |")
print("|  row percentage   |")
print("| column percentage |")
print("+-------------------+\n")

print("policy_action |   predicted")
print("             |     0        1     |   Total")
print("---------------+--------------------+----------")

labels = ["0", "1"]
for i, label in enumerate(labels):
    row = cm[i]
    row_pct = row_perc[i]
    col_pct = col_perc[i]
    row_total = row.sum()
    print(f"         {label} | {row[0]:10,} {row[1]:10,} | {row_total:8,}")
    print(f"           | {row_pct[0]:10.2f} {row_pct[1]:10.2f} |   100.00")
    print(f"           | {col_pct[0]:10.2f} {col_pct[1]:10.2f} |")
    print("---------------+--------------------+----------")

col_total_str = " ".join([f"{val:10,}" for val in col_totals])
col_perc_str = " ".join([f"{(val/grand_total*100):10.2f}" for val in col_totals])
print(f"     Total | {col_total_str} | {grand_total:8,}")
print(f"           | {col_perc_str} |   100.00")
print("           |    100.00     100.00 |   100.00")

# === (4) Scalar Metrics ===
Accuracy    = (TP + TN) / grand_total
Precision   = TP / (TP + FP) if (TP + FP) else 0.0
Recall      = TP / (TP + FN) if (TP + FN) else 0.0
F1          = 2 * (Precision * Recall) / (Precision + Recall) if (Precision + Recall) else 0.0
Specificity = TN / (TN + FP) if (TN + FP) else 0.0

print("\n📐 Scalar-style Evaluation Metrics:")
print(f"Accuracy:        {Accuracy:.8f}")
print(f"Precision:       {Precision:.8f}")
print(f"Recall:          {Recall:.8f}")
print(f"F1 Score:        {F1:.8f}")
print(f"Specificity:     {Specificity:.8f}")


## [7] Classification using Balance Machine learning

## 7.1. Balance Random forest

In [ ]:
# === (a) Import Libraries ===
import pandas as pd
import numpy as np
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, brier_score_loss


# === (b) Load Data ===
train = pd.read_csv("weak_labeled_policy_action_dataset_strict2.csv", encoding="ISO-8859-1")
test = pd.read_csv("classified_climate_policy_actions.csv", encoding="ISO-8859-1")


**[a] --- --- --- Dealing the time_event (single time: event_year_first and event_year_last)**

In [ ]:
from dateutil import parser

def safe_parse_year(text):
    if pd.isna(text):
        return np.nan
    try:
        return parser.parse(str(text), dayfirst=False).year
    except:
        try:
            return parser.parse(str(text), dayfirst=True).year
        except:
            return np.nan

train["event_year_first"] = train["First event in timeline"].apply(safe_parse_year)
train["event_year_last"] = train["Last event in timeline"].apply(safe_parse_year)

test["event_year_first"] = test["First event in timeline"].apply(safe_parse_year)
test["event_year_last"] = test["Last event in timeline"].apply(safe_parse_year)


In [ ]:
# === Check types and summary stats ===
print("🔍 Column types:\n", train[["event_year_first", "event_year_last"]].dtypes)

print("\n📊 Descriptive stats for train:")
print(train[["event_year_first", "event_year_last"]].describe())

print("\n📊 Descriptive stats for test:")
print(test[["event_year_first", "event_year_last"]].describe())

# === Check value counts (frequency) ===
print("\n🗓 Top 10 most frequent 'event_year_first':")
print(train["event_year_first"].value_counts(dropna=False).sort_index().tail(10))

print("\n🗓 Top 10 most frequent 'event_year_last':")
print(train["event_year_last"].value_counts(dropna=False).sort_index().tail(10))

# Show first 20 rows for event_year_first and event_year_last in training set
print("📄 First 20 rows (Training Data):")
print(train[["event_year_first", "event_year_last"]].head(20))

# Show first 20 rows for event_year_first and event_year_last in testing set
print("\n📄 First 20 rows (Testing Data):")
print(test[["event_year_first", "event_year_last"]].head(20))


**[b] --- --- --- Dealing with multiple time event within 1 variable**

In [ ]:
from dateutil import parser

def safe_parse(date_str):
    try:
        return parser.parse(date_str, dayfirst=False)
    except:
        try:
            return parser.parse(date_str, dayfirst=True)
        except:
            return None

def extract_sorted_events(row):
    types = row.get("Full timeline of events (types)")
    dates = row.get("Full timeline of events (dates)")
    
    if pd.isna(types) or pd.isna(dates):
        return None
    
    type_list = [t.strip() for t in str(types).split(";")]
    date_list = [d.strip() for d in str(dates).split(";")]
    
    if len(type_list) != len(date_list):
        return None  # mismatched length
    
    paired = [(t, safe_parse(d)) for t, d in zip(type_list, date_list)]
    paired = [x for x in paired if x[1] is not None]  # drop invalid
    return sorted(paired, key=lambda x: x[1])

train["event_timeline"] = train.apply(extract_sorted_events, axis=1)
test["event_timeline"] = test.apply(extract_sorted_events, axis=1)


In [ ]:
def order_after(df, move_col, after_col):
    cols = list(df.columns)
    cols.remove(move_col)
    insert_at = cols.index(after_col) + 1
    cols.insert(insert_at, move_col)
    return df[cols]

train = order_after(train, "event_timeline", "Full timeline of events (dates)")
test = order_after(test, "event_timeline", "Full timeline of events (dates)")


In [ ]:
import numpy as np

for df in [train, test]:
    # 1. Count of amendments
    df["num_amendments"] = df["event_timeline"].apply(
        lambda x: sum(1 for t, _ in x if t == "Amended") if x else 0
    )

    # 2. First "Passed/Approved" date
    df["first_passed_date"] = df["event_timeline"].apply(
        lambda x: next((d for t, d in x if t == "Passed/Approved"), None) if x else None
    )

    # 3. Last event date (regardless of type)
    df["last_event_date"] = df["event_timeline"].apply(
        lambda x: x[-1][1] if x else None
    )

    # 4. Duration in years
    df["policy_duration_years"] = df.apply(
        lambda row: (row["last_event_date"] - row["first_passed_date"]).days / 365.25
        if row["last_event_date"] and row["first_passed_date"] else np.nan,
        axis=1
    )

print(train[["first_passed_date", "last_event_date", "num_amendments", "policy_duration_years"]].head(10))


In [ ]:
# Convert to datetime if not already
for df in [train, test]:
    df["first_passed_date"] = pd.to_datetime(df["first_passed_date"], errors="coerce")
    df["last_event_date"] = pd.to_datetime(df["last_event_date"], errors="coerce")

    # Extract year after conversion
    df["event_year_passed"] = df["first_passed_date"].dt.year
    df["event_year_last_event"] = df["last_event_date"].dt.year

# ✅ Preview
print("🔍 Sample extracted years:")
print(train[["first_passed_date", "event_year_passed", "last_event_date", "event_year_last_event"]].dropna().head())


**[c] --- --- --- Extra cleaning**

In [ ]:

# === (d) Define target and features ===
target = "policy_action"
exclude_cols = [
    "Document ID", "Document Title", "Family ID", "Family Title",
    "Full timeline of events (types)", target
]
feature_cols = [col for col in train.columns if col not in exclude_cols]

X_train = train[feature_cols].copy()
y_train = train[target]

X_test = test[feature_cols].copy()
y_test = test[target]



In [ ]:
non_informative_cols = []

for col in X_train.columns:
    if X_train[col].dtype == "object":
        # Skip if any row contains lists (unhashable)
        if X_train[col].apply(lambda x: isinstance(x, list)).any():
            continue
        # Drop if it's long text or URL
        if X_train[col].astype(str).str.len().mean() > 100 or \
           X_train[col].astype(str).str.contains("http|www|pdf|doc", case=False).mean() > 0.3:
            non_informative_cols.append(col)
        # Drop if almost every value is unique (like IDs)
        elif X_train[col].nunique(dropna=True) > 0.9 * len(X_train):
            non_informative_cols.append(col)

print("🔎 Non-informative columns to drop:\n", non_informative_cols)


In [ ]:
# Column	Why Drop?
# Family Summary	Long free-text with highly variable length — better for NLP than tabular
# Collection Description(s)	Often redundant and descriptive, not structured
# Document Content URL	Just a URL — no predictive value
# Internal Document ID	Unique ID — not informative, high-cardinality

In [ ]:

# === Drop non-use variables from both train and test ===
X_train.drop(columns=non_informative_cols, inplace=True, errors='ignore')
X_test.drop(columns=non_informative_cols, inplace=True, errors='ignore')


In [ ]:

# === (e) Convert object columns to numeric where possible ===
for df in [X_train, X_test]:
    for col in df.select_dtypes(include='object').columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')



In [ ]:

# === (f) One-hot encode selected categorical variables ===
categorical_vars = [
 "Document Type", "Topic/Response", "Sector", "Instrument",
    "Category", "Framework", "Hazard", "Author", "Author Type",
    "Geographies", "Source", "Geography ISOs"  # <-- New additions
]
categorical_vars = [col for col in categorical_vars if col in X_train.columns]
X_train = pd.get_dummies(X_train, columns=categorical_vars, dummy_na=True)
X_test = pd.get_dummies(X_test, columns=categorical_vars, dummy_na=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)


In [ ]:
# Step 1: Check remaining numeric columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
print(f"🔢 Total numeric columns: {len(numeric_cols)}")

# Step 2: Count missing values per numeric column
missing_train = X_train[numeric_cols].isna().sum()
missing_train = missing_train[missing_train > 0]

print("\n📌 Numeric columns with missing values in training set:")
print(missing_train)

# Step 3: Optional – show percentage missing
percent_missing = (missing_train / len(X_train) * 100).round(2)
print("\n📊 Percentage missing per column:")
print(percent_missing.sort_values(ascending=False))

# Step 4: Show sample rows with missing values in top offender column
if not missing_train.empty:
    top_col = missing_train.idxmax()
    print(f"\n🔎 Sample rows with missing in '{top_col}':")
    display(X_train[X_train[top_col].isna()].head())


✅ Summary of Missingness

| Column                          | Missing (%) | Reason or Type              | Action                                     |
|----------------------------------|--------------|------------------------------|---------------------------------------------|
| `Collection Title(s)`            | 100%         | Metadata, unused             | Drop                                     |
| `Collection Description(s)`      | 100%         | Metadata, unused             | Drop                                     |
| `Document Variant`               | 100%         | Metadata                     | Drop                                     |
| `Language`                       | 100%         | Non-informative              | Drop                                     |
| `First event in timeline`        | 100%         | Raw date field               | Drop (already extracted year)            |
| `Last event in timeline`         | 100%         | Raw date field               | Drop                                     |
| `Full timeline of events (dates)`| 100%         | Raw date field               | Drop                                     |
| `event_timeline`                 | 100%         | List of tuples               | Drop for ML; useful for NLP only         |
| `Date Added to System`           | 100%         | System metadata              | Drop                                     |
| `Last Modified on System`        | 100%         | System metadata              | Drop                                     |
| `Internal Family ID`             | 100%         | Internal ID                  | Drop                                     |
| `Internal Corpus ID`             | 100%         | Internal ID                  | Drop                                     |
| `Internal Collection ID(s)`      | 100%         | Internal ID                  | Drop                                     |
| `Document Role`                  | 100%         | Possibly encoded elsewhere   | Drop                                     |
| `Keyword`                        | 100%         | Free text                    | Drop                                     |


In [ ]:
cols_to_drop = [
    'Collection Title(s)', 'Collection Description(s)', 'Document Variant', 'Language',
    'First event in timeline', 'Last event in timeline', 'Full timeline of events (dates)',
    'event_timeline', 'Date Added to System', 'Last Modified on System',
    'Internal Family ID', 'Internal Corpus ID', 'Internal Collection ID(s)',
    'Document Role', 'Keyword'
]

X_train.drop(columns=[col for col in cols_to_drop if col in X_train.columns], inplace=True)
X_test.drop(columns=[col for col in cols_to_drop if col in X_test.columns], inplace=True)
print("✅ Dropped 100% missing / non-informative columns.")


In [ ]:
# === (g) Impute missing values safely ===

# Identify numeric columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Drop numeric columns with all NaNs in training set
all_nan_cols = [col for col in numeric_cols if X_train[col].isna().all()]
X_train.drop(columns=all_nan_cols, inplace=True)
X_test.drop(columns=all_nan_cols, errors='ignore', inplace=True)

# Re-identify numeric columns after dropping
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Impute if we still have numeric columns
if numeric_cols:
    imputer = SimpleImputer(strategy="mean")
    X_train[numeric_cols] = imputer.fit_transform(X_train[numeric_cols])
    X_test[numeric_cols] = imputer.transform(X_test[numeric_cols])
else:
    print("⚠️ No numeric columns left for imputation.")


# ✅ Confirm imputation worked
print("✅ Remaining numeric columns:", len(numeric_cols))
print("🔍 Any missing values left in training set?", X_train[numeric_cols].isna().any().any())
print("🔍 Any missing values left in test set?", X_test[numeric_cols].isna().any().any())

# Optional: Preview a few numeric columns
X_train[numeric_cols].head()


In [ ]:
# Drop datetime columns that is not useful
X_train.drop(columns=["first_passed_date", "last_event_date"], inplace=True, errors="ignore")
X_test.drop(columns=["first_passed_date", "last_event_date"], inplace=True, errors="ignore")

print("✅ Dropped datetime columns to fix XGBoost training issue.")


**[d] --- --- --- model**

In [ ]:
from sklearn.metrics import (
    accuracy_score, roc_auc_score, confusion_matrix,
    mean_squared_error, brier_score_loss
)

# === (h) Train Balanced Random Forest Classifier ===
brf = BalancedRandomForestClassifier(
    n_estimators=200,
    max_depth=6,
    min_samples_split=4,
    sampling_strategy='auto',
    replacement=True,
    bootstrap=False,
    n_jobs=-1,
    random_state=42
)

brf.fit(X_train, y_train)
print("✅ Balanced Random Forest trained.")


# === (i) Make Predictions ===
y_pred = brf.predict(X_test)
y_prob = brf.predict_proba(X_test)[:, 1]


In [ ]:
print("🔍 Total length of y_test:", len(y_test))
print("❌ Missing values in y_test:", y_test.isna().sum())
# Show rows with NaN in y_test
missing_rows = X_test[y_test.isna()]
display(missing_rows.head())

# Optionally, show original y_test values
print("🧯 Missing y_test entries:")
print(y_test[y_test.isna()])


In [ ]:
# === (j) Filter out rows with NaN in y_test ===
valid_idx = ~y_test.isna()

y_test_clean = y_test[valid_idx]
y_pred_clean = y_pred[valid_idx]
y_proba_clean = y_prob[valid_idx]


In [ ]:

# Now run  evaluation
from sklearn.metrics import accuracy_score, roc_auc_score, brier_score_loss, mean_squared_error, confusion_matrix


# === (k) Evaluate ===
print(f"✅ Accuracy: {accuracy_score(y_test_clean, y_pred_clean):.3f}")
print(f"✅ AUC: {roc_auc_score(y_test_clean, y_proba_clean):.3f}")
print(f"✅ Brier Score: {brier_score_loss(y_test_clean, y_proba_clean):.4f}")
print(f"✅ MSE: {mean_squared_error(y_test_clean, y_proba_clean):.4f}")
print("✅ Confusion Matrix:\n", confusion_matrix(y_test_clean, y_pred_clean))



In [ ]:

# === (ii) Enhanced Confusion Matrix (Stata-style) ===
cm = confusion_matrix(y_test_clean, y_pred_clean, labels=[0, 1])
TN, FP, FN, TP = cm.ravel()

row_perc = cm / cm.sum(axis=1, keepdims=True) * 100
col_perc = cm / cm.sum(axis=0, keepdims=True) * 100
col_totals = cm.sum(axis=0)
grand_total = cm.sum()

print("\n+-------------------+")
print("| Key               |")
print("|-------------------|")
print("|     frequency     |")
print("|  row percentage   |")
print("| column percentage |")
print("+-------------------+\n")

print("leaver_binary |   pred_leave")
print("       ary     |     0        1     |   Total")
print("---------------+--------------------+----------")

labels = ["0", "1"]
for i, label in enumerate(labels):
    row = cm[i]
    row_pct = row_perc[i]
    col_pct = col_perc[i]
    row_total = row.sum()
    print(f"         {label} | {row[0]:10,} {row[1]:10,} | {row_total:8,}")
    print(f"           | {row_pct[0]:10.2f} {row_pct[1]:10.2f} |   100.00")
    print(f"           | {col_pct[0]:10.2f} {col_pct[1]:10.2f} |")
    print("---------------+--------------------+----------")

col_total_str = " ".join([f"{val:10,}" for val in col_totals])
col_perc_str = " ".join([f"{(val/grand_total*100):10.2f}" for val in col_totals])
print(f"     Total | {col_total_str} | {grand_total:8,}")
print(f"           | {col_perc_str} |   100.00")
print("           |    100.00     100.00 |   100.00")

# === (iii) Scalar-style metrics ===
Accuracy    = (TP + TN) / grand_total
Precision   = TP / (TP + FP) if (TP + FP) else 0.0
Recall      = TP / (TP + FN) if (TP + FN) else 0.0
F1          = 2 * (Precision * Recall) / (Precision + Recall) if (Precision + Recall) else 0.0
Specificity = TN / (TN + FP) if (TN + FP) else 0.0

print("\n📐 Scalar-style Evaluation Metrics:")
print(f"Accuracy:        {Accuracy:.8f}")
print(f"Precision:       {Precision:.8f}")
print(f"Recall:          {Recall:.8f}")
print(f"F1 Score:        {F1:.8f}")
print(f"Specificity:     {Specificity:.8f}")


In [ ]:
# === (l) Save Predictions ===
test_clean = test[valid_idx].copy()
test_clean["brf_pred"] = y_pred_clean
test_clean["brf_prob"] = y_proba_clean

test_clean.to_csv("balancedrf_predictions_policyaction.csv", index=False)
print("📁 Exported predictions to: balancedrf_predictions_policyaction.csv")


In [ ]:


print(len(test))           # Full test set
print(len(y_pred))         # Should match test if no filtering
print(len(y_pred_clean))   # Matches only valid_idx



**Python Code for ROC Threshold Evaluation and Export**

In [ ]:
import pandas as pd
import numpy as np

# === (1) Load final predictions ===
df = pd.read_csv("balancedrf_predictions_policyaction.csv")

# === (2) Define column names
score_col = "brf_prob"         # Probabilities from XGBoost
actual_col = "policy_action"   # True label from gold test set

# === (3) Filter clean data ===
df = df[[score_col, actual_col]].dropna()
df['score_round'] = df[score_col].round(4)  # Round for consistency

# === (4) Prepare ROC table ===
thresholds = np.unique(df['score_round'])
results = []

P = (df[actual_col] == 1).sum()
N = (df[actual_col] == 0).sum()

for thresh in thresholds:
    pred = (df[score_col] > thresh).astype(int)
    TP = ((df[actual_col] == 1) & (pred == 1)).sum()
    FP = ((df[actual_col] == 0) & (pred == 1)).sum()
    FN = ((df[actual_col] == 1) & (pred == 0)).sum()
    TN = ((df[actual_col] == 0) & (pred == 0)).sum()

    TPR = TP / P if P > 0 else 0
    FPR = FP / N if N > 0 else 0
    distance = np.sqrt((1 - TPR) ** 2 + (FPR) ** 2)

    results.append({
        "cutoff": thresh,
        "TPR": TPR,
        "FPR": FPR,
        "distance": distance,
        "TP": TP,
        "FP": FP,
        "TN": TN,
        "FN": FN
    })

# === (5) Convert to DataFrame ===
roc_df = pd.DataFrame(results)

# === (6) Find best threshold ===
best_row = roc_df.loc[roc_df['distance'].idxmin()]
best_cutoff = best_row['cutoff']

print("\n🎯 Optimal Threshold Results")
print(f"Best cutoff       : {best_cutoff:.4f}")
print(f"True Positive Rate: {best_row['TPR']:.4f}")
print(f"False Positive Rate: {best_row['FPR']:.4f}")
print(f"Min Distance      : {best_row['distance']:.4f}")


In [ ]:

# === (7) Save all ROC points to CSV for review ===
roc_df.to_csv("rocpoints_BRF.csv", index=False)
print("📁 All thresholds and ROC stats saved to: rocpoints_BRF.csv")


In [ ]:
from sklearn.metrics import confusion_matrix

# === (1) Apply Best Cutoff ===
best_cutoff = 0.3432  # Replace with the actual best threshold
df["final_pred"] = (df["brf_prob"] > best_cutoff).astype(int)

# Extract clean y_true and y_pred (already NaNs dropped earlier)
y_true = df["policy_action"]
y_pred = df["final_pred"]

# === (2) Compute Confusion Matrix ===
cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
TN, FP, FN, TP = cm.ravel()

row_perc = cm / cm.sum(axis=1, keepdims=True) * 100
col_perc = cm / cm.sum(axis=0, keepdims=True) * 100
col_totals = cm.sum(axis=0)
grand_total = cm.sum()

# === (3) Stata-style Table Output ===
print("\n+-------------------+")
print("| Key               |")
print("|-------------------|")
print("|     frequency     |")
print("|  row percentage   |")
print("| column percentage |")
print("+-------------------+\n")

print("policy_action |   predicted")
print("             |     0        1     |   Total")
print("---------------+--------------------+----------")

labels = ["0", "1"]
for i, label in enumerate(labels):
    row = cm[i]
    row_pct = row_perc[i]
    col_pct = col_perc[i]
    row_total = row.sum()
    print(f"         {label} | {row[0]:10,} {row[1]:10,} | {row_total:8,}")
    print(f"           | {row_pct[0]:10.2f} {row_pct[1]:10.2f} |   100.00")
    print(f"           | {col_pct[0]:10.2f} {col_pct[1]:10.2f} |")
    print("---------------+--------------------+----------")

col_total_str = " ".join([f"{val:10,}" for val in col_totals])
col_perc_str = " ".join([f"{(val/grand_total*100):10.2f}" for val in col_totals])
print(f"     Total | {col_total_str} | {grand_total:8,}")
print(f"           | {col_perc_str} |   100.00")
print("           |    100.00     100.00 |   100.00")

# === (4) Scalar Metrics ===
Accuracy    = (TP + TN) / grand_total
Precision   = TP / (TP + FP) if (TP + FP) else 0.0
Recall      = TP / (TP + FN) if (TP + FN) else 0.0
F1          = 2 * (Precision * Recall) / (Precision + Recall) if (Precision + Recall) else 0.0
Specificity = TN / (TN + FP) if (TN + FP) else 0.0

print("\n📐 Scalar-style Evaluation Metrics:")
print(f"Accuracy:        {Accuracy:.8f}")
print(f"Precision:       {Precision:.8f}")
print(f"Recall:          {Recall:.8f}")
print(f"F1 Score:        {F1:.8f}")
print(f"Specificity:     {Specificity:.8f}")


## 7.2. Balance Random forest with Cross-validation

In [ ]:
# === (a) Import Libraries ===
import pandas as pd
import numpy as np
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, brier_score_loss


# === (b) Load Data ===
train = pd.read_csv("weak_labeled_policy_action_dataset_strict2.csv", encoding="ISO-8859-1")
test = pd.read_csv("classified_climate_policy_actions.csv", encoding="ISO-8859-1")


**[a] --- --- --- Dealing the time_event (single time: event_year_first and event_year_last)**

In [ ]:
from dateutil import parser

def safe_parse_year(text):
    if pd.isna(text):
        return np.nan
    try:
        return parser.parse(str(text), dayfirst=False).year
    except:
        try:
            return parser.parse(str(text), dayfirst=True).year
        except:
            return np.nan

train["event_year_first"] = train["First event in timeline"].apply(safe_parse_year)
train["event_year_last"] = train["Last event in timeline"].apply(safe_parse_year)

test["event_year_first"] = test["First event in timeline"].apply(safe_parse_year)
test["event_year_last"] = test["Last event in timeline"].apply(safe_parse_year)


In [ ]:
# === Check types and summary stats ===
print("🔍 Column types:\n", train[["event_year_first", "event_year_last"]].dtypes)

print("\n📊 Descriptive stats for train:")
print(train[["event_year_first", "event_year_last"]].describe())

print("\n📊 Descriptive stats for test:")
print(test[["event_year_first", "event_year_last"]].describe())

# === Check value counts (frequency) ===
print("\n🗓 Top 10 most frequent 'event_year_first':")
print(train["event_year_first"].value_counts(dropna=False).sort_index().tail(10))

print("\n🗓 Top 10 most frequent 'event_year_last':")
print(train["event_year_last"].value_counts(dropna=False).sort_index().tail(10))

# Show first 20 rows for event_year_first and event_year_last in training set
print("📄 First 20 rows (Training Data):")
print(train[["event_year_first", "event_year_last"]].head(20))

# Show first 20 rows for event_year_first and event_year_last in testing set
print("\n📄 First 20 rows (Testing Data):")
print(test[["event_year_first", "event_year_last"]].head(20))


**[b] --- --- --- Dealing with multiple time event within 1 variable**

In [ ]:
from dateutil import parser

def safe_parse(date_str):
    try:
        return parser.parse(date_str, dayfirst=False)
    except:
        try:
            return parser.parse(date_str, dayfirst=True)
        except:
            return None

def extract_sorted_events(row):
    types = row.get("Full timeline of events (types)")
    dates = row.get("Full timeline of events (dates)")
    
    if pd.isna(types) or pd.isna(dates):
        return None
    
    type_list = [t.strip() for t in str(types).split(";")]
    date_list = [d.strip() for d in str(dates).split(";")]
    
    if len(type_list) != len(date_list):
        return None  # mismatched length
    
    paired = [(t, safe_parse(d)) for t, d in zip(type_list, date_list)]
    paired = [x for x in paired if x[1] is not None]  # drop invalid
    return sorted(paired, key=lambda x: x[1])

train["event_timeline"] = train.apply(extract_sorted_events, axis=1)
test["event_timeline"] = test.apply(extract_sorted_events, axis=1)


In [ ]:
def order_after(df, move_col, after_col):
    cols = list(df.columns)
    cols.remove(move_col)
    insert_at = cols.index(after_col) + 1
    cols.insert(insert_at, move_col)
    return df[cols]

train = order_after(train, "event_timeline", "Full timeline of events (dates)")
test = order_after(test, "event_timeline", "Full timeline of events (dates)")


In [ ]:
import numpy as np

for df in [train, test]:
    # 1. Count of amendments
    df["num_amendments"] = df["event_timeline"].apply(
        lambda x: sum(1 for t, _ in x if t == "Amended") if x else 0
    )

    # 2. First "Passed/Approved" date
    df["first_passed_date"] = df["event_timeline"].apply(
        lambda x: next((d for t, d in x if t == "Passed/Approved"), None) if x else None
    )

    # 3. Last event date (regardless of type)
    df["last_event_date"] = df["event_timeline"].apply(
        lambda x: x[-1][1] if x else None
    )

    # 4. Duration in years
    df["policy_duration_years"] = df.apply(
        lambda row: (row["last_event_date"] - row["first_passed_date"]).days / 365.25
        if row["last_event_date"] and row["first_passed_date"] else np.nan,
        axis=1
    )

print(train[["first_passed_date", "last_event_date", "num_amendments", "policy_duration_years"]].head(10))


In [ ]:
# Convert to datetime if not already
for df in [train, test]:
    df["first_passed_date"] = pd.to_datetime(df["first_passed_date"], errors="coerce")
    df["last_event_date"] = pd.to_datetime(df["last_event_date"], errors="coerce")

    # Extract year after conversion
    df["event_year_passed"] = df["first_passed_date"].dt.year
    df["event_year_last_event"] = df["last_event_date"].dt.year

# ✅ Preview
print("🔍 Sample extracted years:")
print(train[["first_passed_date", "event_year_passed", "last_event_date", "event_year_last_event"]].dropna().head())


**[c] --- --- --- Extra cleaning**

In [ ]:

# === (d) Define target and features ===
target = "policy_action"
exclude_cols = [
    "Document ID", "Document Title", "Family ID", "Family Title",
    "Full timeline of events (types)", target
]
feature_cols = [col for col in train.columns if col not in exclude_cols]

X_train = train[feature_cols].copy()
y_train = train[target]

X_test = test[feature_cols].copy()
y_test = test[target]



In [ ]:
non_informative_cols = []

for col in X_train.columns:
    if X_train[col].dtype == "object":
        # Skip if any row contains lists (unhashable)
        if X_train[col].apply(lambda x: isinstance(x, list)).any():
            continue
        # Drop if it's long text or URL
        if X_train[col].astype(str).str.len().mean() > 100 or \
           X_train[col].astype(str).str.contains("http|www|pdf|doc", case=False).mean() > 0.3:
            non_informative_cols.append(col)
        # Drop if almost every value is unique (like IDs)
        elif X_train[col].nunique(dropna=True) > 0.9 * len(X_train):
            non_informative_cols.append(col)

print("🔎 Non-informative columns to drop:\n", non_informative_cols)


In [ ]:
# Column	Why Drop?
# Family Summary	Long free-text with highly variable length — better for NLP than tabular
# Collection Description(s)	Often redundant and descriptive, not structured
# Document Content URL	Just a URL — no predictive value
# Internal Document ID	Unique ID — not informative, high-cardinality

In [ ]:

# === Drop non-use variables from both train and test ===
X_train.drop(columns=non_informative_cols, inplace=True, errors='ignore')
X_test.drop(columns=non_informative_cols, inplace=True, errors='ignore')


In [ ]:

# === (e) Convert object columns to numeric where possible ===
for df in [X_train, X_test]:
    for col in df.select_dtypes(include='object').columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')



In [ ]:

# === (f) One-hot encode selected categorical variables ===
categorical_vars = [
 "Document Type", "Topic/Response", "Sector", "Instrument",
    "Category", "Framework", "Hazard", "Author", "Author Type",
    "Geographies", "Source", "Geography ISOs"  # <-- New additions
]
categorical_vars = [col for col in categorical_vars if col in X_train.columns]
X_train = pd.get_dummies(X_train, columns=categorical_vars, dummy_na=True)
X_test = pd.get_dummies(X_test, columns=categorical_vars, dummy_na=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)


In [ ]:
# Step 1: Check remaining numeric columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
print(f"🔢 Total numeric columns: {len(numeric_cols)}")

# Step 2: Count missing values per numeric column
missing_train = X_train[numeric_cols].isna().sum()
missing_train = missing_train[missing_train > 0]

print("\n📌 Numeric columns with missing values in training set:")
print(missing_train)

# Step 3: Optional – show percentage missing
percent_missing = (missing_train / len(X_train) * 100).round(2)
print("\n📊 Percentage missing per column:")
print(percent_missing.sort_values(ascending=False))

# Step 4: Show sample rows with missing values in top offender column
if not missing_train.empty:
    top_col = missing_train.idxmax()
    print(f"\n🔎 Sample rows with missing in '{top_col}':")
    display(X_train[X_train[top_col].isna()].head())


✅ Summary of Missingness

| Column                          | Missing (%) | Reason or Type              | Action                                     |
|----------------------------------|--------------|------------------------------|---------------------------------------------|
| `Collection Title(s)`            | 100%         | Metadata, unused             | Drop                                     |
| `Collection Description(s)`      | 100%         | Metadata, unused             | Drop                                     |
| `Document Variant`               | 100%         | Metadata                     | Drop                                     |
| `Language`                       | 100%         | Non-informative              | Drop                                     |
| `First event in timeline`        | 100%         | Raw date field               | Drop (already extracted year)            |
| `Last event in timeline`         | 100%         | Raw date field               | Drop                                     |
| `Full timeline of events (dates)`| 100%         | Raw date field               | Drop                                     |
| `event_timeline`                 | 100%         | List of tuples               | Drop for ML; useful for NLP only         |
| `Date Added to System`           | 100%         | System metadata              | Drop                                     |
| `Last Modified on System`        | 100%         | System metadata              | Drop                                     |
| `Internal Family ID`             | 100%         | Internal ID                  | Drop                                     |
| `Internal Corpus ID`             | 100%         | Internal ID                  | Drop                                     |
| `Internal Collection ID(s)`      | 100%         | Internal ID                  | Drop                                     |
| `Document Role`                  | 100%         | Possibly encoded elsewhere   | Drop                                     |
| `Keyword`                        | 100%         | Free text                    | Drop                                     |


In [ ]:
cols_to_drop = [
    'Collection Title(s)', 'Collection Description(s)', 'Document Variant', 'Language',
    'First event in timeline', 'Last event in timeline', 'Full timeline of events (dates)',
    'event_timeline', 'Date Added to System', 'Last Modified on System',
    'Internal Family ID', 'Internal Corpus ID', 'Internal Collection ID(s)',
    'Document Role', 'Keyword'
]

X_train.drop(columns=[col for col in cols_to_drop if col in X_train.columns], inplace=True)
X_test.drop(columns=[col for col in cols_to_drop if col in X_test.columns], inplace=True)
print("✅ Dropped 100% missing / non-informative columns.")


In [ ]:
# === (g) Impute missing values safely ===

# Identify numeric columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Drop numeric columns with all NaNs in training set
all_nan_cols = [col for col in numeric_cols if X_train[col].isna().all()]
X_train.drop(columns=all_nan_cols, inplace=True)
X_test.drop(columns=all_nan_cols, errors='ignore', inplace=True)

# Re-identify numeric columns after dropping
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Impute if we still have numeric columns
if numeric_cols:
    imputer = SimpleImputer(strategy="mean")
    X_train[numeric_cols] = imputer.fit_transform(X_train[numeric_cols])
    X_test[numeric_cols] = imputer.transform(X_test[numeric_cols])
else:
    print("⚠️ No numeric columns left for imputation.")


# ✅ Confirm imputation worked
print("✅ Remaining numeric columns:", len(numeric_cols))
print("🔍 Any missing values left in training set?", X_train[numeric_cols].isna().any().any())
print("🔍 Any missing values left in test set?", X_test[numeric_cols].isna().any().any())

# Optional: Preview a few numeric columns
X_train[numeric_cols].head()


In [ ]:
# Drop datetime columns that is not useful
X_train.drop(columns=["first_passed_date", "last_event_date"], inplace=True, errors="ignore")
X_test.drop(columns=["first_passed_date", "last_event_date"], inplace=True, errors="ignore")

print("✅ Dropped datetime columns to fix not useful issue.")


**[d] --- --- --- model**

In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix, classification_report
)
import numpy as np

# === (a) Cross-validation Settings ===
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# === (b) Store results ===
metrics = {
    'accuracy': [], 'precision': [], 'recall': [],
    'f1': [], 'auc': [], 'specificity': []
}
all_y_true = []
all_y_pred = []

# === (c) Cross-validation loop ===
fold_num = 1
for train_idx, val_idx in cv.split(X_train, y_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    # Balanced Random Forest for this fold
    brf_cv = BalancedRandomForestClassifier(
        n_estimators=200,
        max_depth=7,
        min_samples_split=4,
        sampling_strategy='auto',
        replacement=True,
        bootstrap=False,
        n_jobs=-1,
        random_state=42
    )
    brf_cv.fit(X_tr, y_tr)

    # Predict using default threshold (0.5)
    y_proba = brf_cv.predict_proba(X_val)[:, 1]
    y_pred = brf_cv.predict(X_val)

    # Save out-of-fold predictions
    all_y_true.extend(y_val)
    all_y_pred.extend(y_pred)

    # Confusion matrix and metrics for this fold
    cm = confusion_matrix(y_val, y_pred)
    TN, FP, FN, TP = cm.ravel()

    metrics['accuracy'].append(accuracy_score(y_val, y_pred))
    metrics['precision'].append(precision_score(y_val, y_pred, zero_division=0))
    metrics['recall'].append(recall_score(y_val, y_pred))
    metrics['f1'].append(f1_score(y_val, y_pred))
    metrics['auc'].append(roc_auc_score(y_val, y_proba))
    metrics['specificity'].append(TN / (TN + FP))

    print(f"✅ Fold {fold_num} completed.")
    fold_num += 1

# === (d) Print average ± std of metrics ===
print("\n=== 📊 5-Fold CV Metrics (Default Cutoff = 0.5) ===")
for key in metrics:
    mean = np.mean(metrics[key])
    std = np.std(metrics[key])
    print(f"{key.capitalize():<12}: {mean:.4f} ± {std:.4f}")

# === (e) Final aggregated evaluation ===
print("\n=== 🔎 Final Aggregated Performance on All Out-of-Fold Predictions ===")
final_cm = confusion_matrix(all_y_true, all_y_pred)
TN, FP, FN, TP = final_cm.ravel()
print("Confusion Matrix:\n", final_cm)
print(f"Specificity: {TN / (TN + FP):.4f}")
print("\nClassification Report:")
print(classification_report(all_y_true, all_y_pred, digits=4))


## 7.3. Imbalance Weight Xgboost

In [ ]:
# === (a) Import Libraries ===
import pandas as pd
import numpy as np
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, brier_score_loss


# === (b) Load Data ===
train = pd.read_csv("weak_labeled_policy_action_dataset_strict2.csv", encoding="ISO-8859-1")
test = pd.read_csv("classified_climate_policy_actions.csv", encoding="ISO-8859-1")


**[a] --- --- --- Dealing the time_event (single time: event_year_first and event_year_last)**

In [ ]:
from dateutil import parser

def safe_parse_year(text):
    if pd.isna(text):
        return np.nan
    try:
        return parser.parse(str(text), dayfirst=False).year
    except:
        try:
            return parser.parse(str(text), dayfirst=True).year
        except:
            return np.nan

train["event_year_first"] = train["First event in timeline"].apply(safe_parse_year)
train["event_year_last"] = train["Last event in timeline"].apply(safe_parse_year)

test["event_year_first"] = test["First event in timeline"].apply(safe_parse_year)
test["event_year_last"] = test["Last event in timeline"].apply(safe_parse_year)


In [ ]:
# === Check types and summary stats ===
print("🔍 Column types:\n", train[["event_year_first", "event_year_last"]].dtypes)

print("\n📊 Descriptive stats for train:")
print(train[["event_year_first", "event_year_last"]].describe())

print("\n📊 Descriptive stats for test:")
print(test[["event_year_first", "event_year_last"]].describe())

# === Check value counts (frequency) ===
print("\n🗓 Top 10 most frequent 'event_year_first':")
print(train["event_year_first"].value_counts(dropna=False).sort_index().tail(10))

print("\n🗓 Top 10 most frequent 'event_year_last':")
print(train["event_year_last"].value_counts(dropna=False).sort_index().tail(10))

# Show first 20 rows for event_year_first and event_year_last in training set
print("📄 First 20 rows (Training Data):")
print(train[["event_year_first", "event_year_last"]].head(20))

# Show first 20 rows for event_year_first and event_year_last in testing set
print("\n📄 First 20 rows (Testing Data):")
print(test[["event_year_first", "event_year_last"]].head(20))


**[b] --- --- --- Dealing with multiple time event within 1 variable**

In [ ]:
from dateutil import parser

def safe_parse(date_str):
    try:
        return parser.parse(date_str, dayfirst=False)
    except:
        try:
            return parser.parse(date_str, dayfirst=True)
        except:
            return None

def extract_sorted_events(row):
    types = row.get("Full timeline of events (types)")
    dates = row.get("Full timeline of events (dates)")
    
    if pd.isna(types) or pd.isna(dates):
        return None
    
    type_list = [t.strip() for t in str(types).split(";")]
    date_list = [d.strip() for d in str(dates).split(";")]
    
    if len(type_list) != len(date_list):
        return None  # mismatched length
    
    paired = [(t, safe_parse(d)) for t, d in zip(type_list, date_list)]
    paired = [x for x in paired if x[1] is not None]  # drop invalid
    return sorted(paired, key=lambda x: x[1])

train["event_timeline"] = train.apply(extract_sorted_events, axis=1)
test["event_timeline"] = test.apply(extract_sorted_events, axis=1)


In [ ]:
def order_after(df, move_col, after_col):
    cols = list(df.columns)
    cols.remove(move_col)
    insert_at = cols.index(after_col) + 1
    cols.insert(insert_at, move_col)
    return df[cols]

train = order_after(train, "event_timeline", "Full timeline of events (dates)")
test = order_after(test, "event_timeline", "Full timeline of events (dates)")


In [ ]:
import numpy as np

for df in [train, test]:
    # 1. Count of amendments
    df["num_amendments"] = df["event_timeline"].apply(
        lambda x: sum(1 for t, _ in x if t == "Amended") if x else 0
    )

    # 2. First "Passed/Approved" date
    df["first_passed_date"] = df["event_timeline"].apply(
        lambda x: next((d for t, d in x if t == "Passed/Approved"), None) if x else None
    )

    # 3. Last event date (regardless of type)
    df["last_event_date"] = df["event_timeline"].apply(
        lambda x: x[-1][1] if x else None
    )

    # 4. Duration in years
    df["policy_duration_years"] = df.apply(
        lambda row: (row["last_event_date"] - row["first_passed_date"]).days / 365.25
        if row["last_event_date"] and row["first_passed_date"] else np.nan,
        axis=1
    )

print(train[["first_passed_date", "last_event_date", "num_amendments", "policy_duration_years"]].head(10))


In [ ]:
# Convert to datetime if not already
for df in [train, test]:
    df["first_passed_date"] = pd.to_datetime(df["first_passed_date"], errors="coerce")
    df["last_event_date"] = pd.to_datetime(df["last_event_date"], errors="coerce")

    # Extract year after conversion
    df["event_year_passed"] = df["first_passed_date"].dt.year
    df["event_year_last_event"] = df["last_event_date"].dt.year

# ✅ Preview
print("🔍 Sample extracted years:")
print(train[["first_passed_date", "event_year_passed", "last_event_date", "event_year_last_event"]].dropna().head())


**[c] --- --- --- Extra cleaning**

In [ ]:

# === (d) Define target and features ===
target = "policy_action"
exclude_cols = [
    "Document ID", "Document Title", "Family ID", "Family Title",
    "Full timeline of events (types)", target
]
feature_cols = [col for col in train.columns if col not in exclude_cols]

X_train = train[feature_cols].copy()
y_train = train[target]

X_test = test[feature_cols].copy()
y_test = test[target]



In [ ]:
non_informative_cols = []

for col in X_train.columns:
    if X_train[col].dtype == "object":
        # Skip if any row contains lists (unhashable)
        if X_train[col].apply(lambda x: isinstance(x, list)).any():
            continue
        # Drop if it's long text or URL
        if X_train[col].astype(str).str.len().mean() > 100 or \
           X_train[col].astype(str).str.contains("http|www|pdf|doc", case=False).mean() > 0.3:
            non_informative_cols.append(col)
        # Drop if almost every value is unique (like IDs)
        elif X_train[col].nunique(dropna=True) > 0.9 * len(X_train):
            non_informative_cols.append(col)

print("🔎 Non-informative columns to drop:\n", non_informative_cols)


In [ ]:
# Column	Why Drop?
# Family Summary	Long free-text with highly variable length — better for NLP than tabular
# Collection Description(s)	Often redundant and descriptive, not structured
# Document Content URL	Just a URL — no predictive value
# Internal Document ID	Unique ID — not informative, high-cardinality

In [ ]:

# === Drop non-use variables from both train and test ===
X_train.drop(columns=non_informative_cols, inplace=True, errors='ignore')
X_test.drop(columns=non_informative_cols, inplace=True, errors='ignore')


In [ ]:

# === (e) Convert object columns to numeric where possible ===
for df in [X_train, X_test]:
    for col in df.select_dtypes(include='object').columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')



In [ ]:

# === (f) One-hot encode selected categorical variables ===
categorical_vars = [
 "Document Type", "Topic/Response", "Sector", "Instrument",
    "Category", "Framework", "Hazard", "Author", "Author Type",
    "Geographies", "Source", "Geography ISOs"  # <-- New additions
]
categorical_vars = [col for col in categorical_vars if col in X_train.columns]
X_train = pd.get_dummies(X_train, columns=categorical_vars, dummy_na=True)
X_test = pd.get_dummies(X_test, columns=categorical_vars, dummy_na=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)


In [ ]:
# Step 1: Check remaining numeric columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
print(f"🔢 Total numeric columns: {len(numeric_cols)}")

# Step 2: Count missing values per numeric column
missing_train = X_train[numeric_cols].isna().sum()
missing_train = missing_train[missing_train > 0]

print("\n📌 Numeric columns with missing values in training set:")
print(missing_train)

# Step 3: Optional – show percentage missing
percent_missing = (missing_train / len(X_train) * 100).round(2)
print("\n📊 Percentage missing per column:")
print(percent_missing.sort_values(ascending=False))

# Step 4: Show sample rows with missing values in top offender column
if not missing_train.empty:
    top_col = missing_train.idxmax()
    print(f"\n🔎 Sample rows with missing in '{top_col}':")
    display(X_train[X_train[top_col].isna()].head())


✅ Summary of Missingness

| Column                          | Missing (%) | Reason or Type              | Action                                     |
|----------------------------------|--------------|------------------------------|---------------------------------------------|
| `Collection Title(s)`            | 100%         | Metadata, unused             | Drop                                     |
| `Collection Description(s)`      | 100%         | Metadata, unused             | Drop                                     |
| `Document Variant`               | 100%         | Metadata                     | Drop                                     |
| `Language`                       | 100%         | Non-informative              | Drop                                     |
| `First event in timeline`        | 100%         | Raw date field               | Drop (already extracted year)            |
| `Last event in timeline`         | 100%         | Raw date field               | Drop                                     |
| `Full timeline of events (dates)`| 100%         | Raw date field               | Drop                                     |
| `event_timeline`                 | 100%         | List of tuples               | Drop for ML; useful for NLP only         |
| `Date Added to System`           | 100%         | System metadata              | Drop                                     |
| `Last Modified on System`        | 100%         | System metadata              | Drop                                     |
| `Internal Family ID`             | 100%         | Internal ID                  | Drop                                     |
| `Internal Corpus ID`             | 100%         | Internal ID                  | Drop                                     |
| `Internal Collection ID(s)`      | 100%         | Internal ID                  | Drop                                     |
| `Document Role`                  | 100%         | Possibly encoded elsewhere   | Drop                                     |
| `Keyword`                        | 100%         | Free text                    | Drop                                     |


In [ ]:
cols_to_drop = [
    'Collection Title(s)', 'Collection Description(s)', 'Document Variant', 'Language',
    'First event in timeline', 'Last event in timeline', 'Full timeline of events (dates)',
    'event_timeline', 'Date Added to System', 'Last Modified on System',
    'Internal Family ID', 'Internal Corpus ID', 'Internal Collection ID(s)',
    'Document Role', 'Keyword'
]

X_train.drop(columns=[col for col in cols_to_drop if col in X_train.columns], inplace=True)
X_test.drop(columns=[col for col in cols_to_drop if col in X_test.columns], inplace=True)
print("✅ Dropped 100% missing / non-informative columns.")


In [ ]:
# === (g) Impute missing values safely ===

# Identify numeric columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Drop numeric columns with all NaNs in training set
all_nan_cols = [col for col in numeric_cols if X_train[col].isna().all()]
X_train.drop(columns=all_nan_cols, inplace=True)
X_test.drop(columns=all_nan_cols, errors='ignore', inplace=True)

# Re-identify numeric columns after dropping
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Impute if we still have numeric columns
if numeric_cols:
    imputer = SimpleImputer(strategy="mean")
    X_train[numeric_cols] = imputer.fit_transform(X_train[numeric_cols])
    X_test[numeric_cols] = imputer.transform(X_test[numeric_cols])
else:
    print("⚠️ No numeric columns left for imputation.")


# ✅ Confirm imputation worked
print("✅ Remaining numeric columns:", len(numeric_cols))
print("🔍 Any missing values left in training set?", X_train[numeric_cols].isna().any().any())
print("🔍 Any missing values left in test set?", X_test[numeric_cols].isna().any().any())

# Optional: Preview a few numeric columns
X_train[numeric_cols].head()


In [ ]:
# Drop datetime columns that is not useful
X_train.drop(columns=["first_passed_date", "last_event_date"], inplace=True, errors="ignore")
X_test.drop(columns=["first_passed_date", "last_event_date"], inplace=True, errors="ignore")

print("✅ Dropped datetime columns to fix XGBoost training issue.")


**[d] --- --- --- model**

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.impute import SimpleImputer

# === (h) Compute imbalance weight ===
n_pos = (y_train == 1).sum()
n_neg = (y_train == 0).sum()
scale_pos_weight = n_neg / n_pos
print(f"✅ Calculated scale_pos_weight: {scale_pos_weight:.4f}")

# === (i) Train Imbalanced XGBoost Model ===
model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.2,
    n_jobs=-1,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42,
    scale_pos_weight=scale_pos_weight
)
model.fit(X_train, y_train)
print("✅ Weighted XGBoost model trained.")


# === (i) Predict and Evaluate ===
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

In [ ]:
print("🔍 Total length of y_test:", len(y_test))
print("❌ Missing values in y_test:", y_test.isna().sum())
# Show rows with NaN in y_test
missing_rows = X_test[y_test.isna()]
display(missing_rows.head())

# Optionally, show original y_test values
print("🧯 Missing y_test entries:")
print(y_test[y_test.isna()])


In [ ]:
# === (j) Filter out rows with NaN in y_test ===
valid_idx = ~y_test.isna()

# Apply mask to clean targets and predictions
y_test_clean = y_test[valid_idx]
y_pred_clean = y_pred[valid_idx]
y_proba_clean = y_proba[valid_idx]

# Log results
print(f"✅ Valid y_test entries: {valid_idx.sum()} / {len(y_test)}")
print(f"✅ Cleaned predictions and probabilities aligned.")


In [ ]:

# Now run evaluation
from sklearn.metrics import (
    accuracy_score, roc_auc_score,
    brier_score_loss, mean_squared_error,
    confusion_matrix
)

# === (k) Evaluation Metrics ===
print("\n=== 📊 Evaluation Results (Default Threshold = 0.5) ===")
print(f"✅ Accuracy       : {accuracy_score(y_test_clean, y_pred_clean):.3f}")
print(f"✅ AUC            : {roc_auc_score(y_test_clean, y_proba_clean):.3f}")
print(f"✅ Brier Score    : {brier_score_loss(y_test_clean, y_proba_clean):.4f}")
print(f"✅ MSE            : {mean_squared_error(y_test_clean, y_proba_clean):.4f}")

# === (l) Confusion Matrix ===
print("\n✅ Confusion Matrix:")
print(confusion_matrix(y_test_clean, y_pred_clean, labels=[0, 1]))


In [ ]:

# === (ii) Enhanced Confusion Matrix (Stata-style) ===
cm = confusion_matrix(y_test_clean, y_pred_clean, labels=[0, 1])
TN, FP, FN, TP = cm.ravel()

row_perc = cm / cm.sum(axis=1, keepdims=True) * 100
col_perc = cm / cm.sum(axis=0, keepdims=True) * 100
col_totals = cm.sum(axis=0)
grand_total = cm.sum()

print("\n+-------------------+")
print("| Key               |")
print("|-------------------|")
print("|     frequency     |")
print("|  row percentage   |")
print("| column percentage |")
print("+-------------------+\n")

print("leaver_binary |   pred_leave")
print("       ary     |     0        1     |   Total")
print("---------------+--------------------+----------")

labels = ["0", "1"]
for i, label in enumerate(labels):
    row = cm[i]
    row_pct = row_perc[i]
    col_pct = col_perc[i]
    row_total = row.sum()
    print(f"         {label} | {row[0]:10,} {row[1]:10,} | {row_total:8,}")
    print(f"           | {row_pct[0]:10.2f} {row_pct[1]:10.2f} |   100.00")
    print(f"           | {col_pct[0]:10.2f} {col_pct[1]:10.2f} |")
    print("---------------+--------------------+----------")

col_total_str = " ".join([f"{val:10,}" for val in col_totals])
col_perc_str = " ".join([f"{(val/grand_total*100):10.2f}" for val in col_totals])
print(f"     Total | {col_total_str} | {grand_total:8,}")
print(f"           | {col_perc_str} |   100.00")
print("           |    100.00     100.00 |   100.00")

# === (iii) Scalar-style metrics ===
Accuracy    = (TP + TN) / grand_total
Precision   = TP / (TP + FP) if (TP + FP) else 0.0
Recall      = TP / (TP + FN) if (TP + FN) else 0.0
F1          = 2 * (Precision * Recall) / (Precision + Recall) if (Precision + Recall) else 0.0
Specificity = TN / (TN + FP) if (TN + FP) else 0.0

print("\n📐 Scalar-style Evaluation Metrics:")
print(f"Accuracy:        {Accuracy:.8f}")
print(f"Precision:       {Precision:.8f}")
print(f"Recall:          {Recall:.8f}")
print(f"F1 Score:        {F1:.8f}")
print(f"Specificity:     {Specificity:.8f}")


In [ ]:
# === (l) Save Predictions ===
test_clean = test[valid_idx].copy()
test_clean["xgb_pred"] = y_pred_clean
test_clean["xgb_prob"] = y_proba_clean

test_clean.to_csv("imbaxgboost_predictions_policyaction.csv", index=False)
print("📁 Exported predictions to: imbaxgboost_predictions_policyaction.csv")



**Python Code for ROC Threshold Evaluation and Export**

In [ ]:
import pandas as pd
import numpy as np

# === (1) Load final predictions ===
df = pd.read_csv("imbaxgboost_predictions_policyaction.csv")

# === (2) Define column names
score_col = "xgb_prob"         # Probabilities from XGBoost
actual_col = "policy_action"   # True label from gold test set

# === (3) Filter clean data ===
df = df[[score_col, actual_col]].dropna()
df['score_round'] = df[score_col].round(4)  # Round for consistency

# === (4) Prepare ROC table ===
thresholds = np.unique(df['score_round'])
results = []

P = (df[actual_col] == 1).sum()
N = (df[actual_col] == 0).sum()

for thresh in thresholds:
    pred = (df[score_col] > thresh).astype(int)
    TP = ((df[actual_col] == 1) & (pred == 1)).sum()
    FP = ((df[actual_col] == 0) & (pred == 1)).sum()
    FN = ((df[actual_col] == 1) & (pred == 0)).sum()
    TN = ((df[actual_col] == 0) & (pred == 0)).sum()

    TPR = TP / P if P > 0 else 0
    FPR = FP / N if N > 0 else 0
    distance = np.sqrt((1 - TPR) ** 2 + (FPR) ** 2)

    results.append({
        "cutoff": thresh,
        "TPR": TPR,
        "FPR": FPR,
        "distance": distance,
        "TP": TP,
        "FP": FP,
        "TN": TN,
        "FN": FN
    })

# === (5) Convert to DataFrame ===
roc_df = pd.DataFrame(results)

# === (6) Find best threshold ===
best_row = roc_df.loc[roc_df['distance'].idxmin()]
best_cutoff = best_row['cutoff']

print("\n🎯 Optimal Threshold Results")
print(f"Best cutoff       : {best_cutoff:.4f}")
print(f"True Positive Rate: {best_row['TPR']:.4f}")
print(f"False Positive Rate: {best_row['FPR']:.4f}")
print(f"Min Distance      : {best_row['distance']:.4f}")


In [ ]:

# === (7) Save all ROC points to CSV for review ===
roc_df.to_csv("rocpoints_ImbaXgboost.csv", index=False)
print("📁 All thresholds and ROC stats saved to: rocpoints_ImbaXgboost.csv")


In [ ]:
from sklearn.metrics import confusion_matrix

# === (1) Apply Best Cutoff ===
best_cutoff = 0.3746  # Replace with the actual best threshold
df["final_pred"] = (df["xgb_prob"] > best_cutoff).astype(int)

# Extract clean y_true and y_pred (already NaNs dropped earlier)
y_true = df["policy_action"]
y_pred = df["final_pred"]

# === (2) Compute Confusion Matrix ===
cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
TN, FP, FN, TP = cm.ravel()

row_perc = cm / cm.sum(axis=1, keepdims=True) * 100
col_perc = cm / cm.sum(axis=0, keepdims=True) * 100
col_totals = cm.sum(axis=0)
grand_total = cm.sum()

# === (3) Stata-style Table Output ===
print("\n+-------------------+")
print("| Key               |")
print("|-------------------|")
print("|     frequency     |")
print("|  row percentage   |")
print("| column percentage |")
print("+-------------------+\n")

print("policy_action |   predicted")
print("             |     0        1     |   Total")
print("---------------+--------------------+----------")

labels = ["0", "1"]
for i, label in enumerate(labels):
    row = cm[i]
    row_pct = row_perc[i]
    col_pct = col_perc[i]
    row_total = row.sum()
    print(f"         {label} | {row[0]:10,} {row[1]:10,} | {row_total:8,}")
    print(f"           | {row_pct[0]:10.2f} {row_pct[1]:10.2f} |   100.00")
    print(f"           | {col_pct[0]:10.2f} {col_pct[1]:10.2f} |")
    print("---------------+--------------------+----------")

col_total_str = " ".join([f"{val:10,}" for val in col_totals])
col_perc_str = " ".join([f"{(val/grand_total*100):10.2f}" for val in col_totals])
print(f"     Total | {col_total_str} | {grand_total:8,}")
print(f"           | {col_perc_str} |   100.00")
print("           |    100.00     100.00 |   100.00")

# === (4) Scalar Metrics ===
Accuracy    = (TP + TN) / grand_total
Precision   = TP / (TP + FP) if (TP + FP) else 0.0
Recall      = TP / (TP + FN) if (TP + FN) else 0.0
F1          = 2 * (Precision * Recall) / (Precision + Recall) if (Precision + Recall) else 0.0
Specificity = TN / (TN + FP) if (TN + FP) else 0.0

print("\n📐 Scalar-style Evaluation Metrics:")
print(f"Accuracy:        {Accuracy:.8f}")
print(f"Precision:       {Precision:.8f}")
print(f"Recall:          {Recall:.8f}")
print(f"F1 Score:        {F1:.8f}")
print(f"Specificity:     {Specificity:.8f}")


## 7.4. Imbalance Xgboost with cross-validation

In [ ]:
# === (a) Import Libraries ===
import pandas as pd
import numpy as np
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, brier_score_loss


# === (b) Load Data ===
train = pd.read_csv("weak_labeled_policy_action_dataset_strict2.csv", encoding="ISO-8859-1")
test = pd.read_csv("classified_climate_policy_actions.csv", encoding="ISO-8859-1")


**[a] --- --- --- Dealing the time_event (single time: event_year_first and event_year_last)**

In [ ]:
from dateutil import parser

def safe_parse_year(text):
    if pd.isna(text):
        return np.nan
    try:
        return parser.parse(str(text), dayfirst=False).year
    except:
        try:
            return parser.parse(str(text), dayfirst=True).year
        except:
            return np.nan

train["event_year_first"] = train["First event in timeline"].apply(safe_parse_year)
train["event_year_last"] = train["Last event in timeline"].apply(safe_parse_year)

test["event_year_first"] = test["First event in timeline"].apply(safe_parse_year)
test["event_year_last"] = test["Last event in timeline"].apply(safe_parse_year)


In [ ]:
# === Check types and summary stats ===
print("🔍 Column types:\n", train[["event_year_first", "event_year_last"]].dtypes)

print("\n📊 Descriptive stats for train:")
print(train[["event_year_first", "event_year_last"]].describe())

print("\n📊 Descriptive stats for test:")
print(test[["event_year_first", "event_year_last"]].describe())

# === Check value counts (frequency) ===
print("\n🗓 Top 10 most frequent 'event_year_first':")
print(train["event_year_first"].value_counts(dropna=False).sort_index().tail(10))

print("\n🗓 Top 10 most frequent 'event_year_last':")
print(train["event_year_last"].value_counts(dropna=False).sort_index().tail(10))

# Show first 20 rows for event_year_first and event_year_last in training set
print("📄 First 20 rows (Training Data):")
print(train[["event_year_first", "event_year_last"]].head(20))

# Show first 20 rows for event_year_first and event_year_last in testing set
print("\n📄 First 20 rows (Testing Data):")
print(test[["event_year_first", "event_year_last"]].head(20))


**[b] --- --- --- Dealing with multiple time event within 1 variable**

In [ ]:
from dateutil import parser

def safe_parse(date_str):
    try:
        return parser.parse(date_str, dayfirst=False)
    except:
        try:
            return parser.parse(date_str, dayfirst=True)
        except:
            return None

def extract_sorted_events(row):
    types = row.get("Full timeline of events (types)")
    dates = row.get("Full timeline of events (dates)")
    
    if pd.isna(types) or pd.isna(dates):
        return None
    
    type_list = [t.strip() for t in str(types).split(";")]
    date_list = [d.strip() for d in str(dates).split(";")]
    
    if len(type_list) != len(date_list):
        return None  # mismatched length
    
    paired = [(t, safe_parse(d)) for t, d in zip(type_list, date_list)]
    paired = [x for x in paired if x[1] is not None]  # drop invalid
    return sorted(paired, key=lambda x: x[1])

train["event_timeline"] = train.apply(extract_sorted_events, axis=1)
test["event_timeline"] = test.apply(extract_sorted_events, axis=1)


In [ ]:
def order_after(df, move_col, after_col):
    cols = list(df.columns)
    cols.remove(move_col)
    insert_at = cols.index(after_col) + 1
    cols.insert(insert_at, move_col)
    return df[cols]

train = order_after(train, "event_timeline", "Full timeline of events (dates)")
test = order_after(test, "event_timeline", "Full timeline of events (dates)")


In [ ]:
import numpy as np

for df in [train, test]:
    # 1. Count of amendments
    df["num_amendments"] = df["event_timeline"].apply(
        lambda x: sum(1 for t, _ in x if t == "Amended") if x else 0
    )

    # 2. First "Passed/Approved" date
    df["first_passed_date"] = df["event_timeline"].apply(
        lambda x: next((d for t, d in x if t == "Passed/Approved"), None) if x else None
    )

    # 3. Last event date (regardless of type)
    df["last_event_date"] = df["event_timeline"].apply(
        lambda x: x[-1][1] if x else None
    )

    # 4. Duration in years
    df["policy_duration_years"] = df.apply(
        lambda row: (row["last_event_date"] - row["first_passed_date"]).days / 365.25
        if row["last_event_date"] and row["first_passed_date"] else np.nan,
        axis=1
    )

print(train[["first_passed_date", "last_event_date", "num_amendments", "policy_duration_years"]].head(10))


In [ ]:
# Convert to datetime if not already
for df in [train, test]:
    df["first_passed_date"] = pd.to_datetime(df["first_passed_date"], errors="coerce")
    df["last_event_date"] = pd.to_datetime(df["last_event_date"], errors="coerce")

    # Extract year after conversion
    df["event_year_passed"] = df["first_passed_date"].dt.year
    df["event_year_last_event"] = df["last_event_date"].dt.year

# ✅ Preview
print("🔍 Sample extracted years:")
print(train[["first_passed_date", "event_year_passed", "last_event_date", "event_year_last_event"]].dropna().head())


**[c] --- --- --- Extra cleaning**

In [ ]:

# === (d) Define target and features ===
target = "policy_action"
exclude_cols = [
    "Document ID", "Document Title", "Family ID", "Family Title",
    "Full timeline of events (types)", target
]
feature_cols = [col for col in train.columns if col not in exclude_cols]

X_train = train[feature_cols].copy()
y_train = train[target]

X_test = test[feature_cols].copy()
y_test = test[target]



In [ ]:
non_informative_cols = []

for col in X_train.columns:
    if X_train[col].dtype == "object":
        # Skip if any row contains lists (unhashable)
        if X_train[col].apply(lambda x: isinstance(x, list)).any():
            continue
        # Drop if it's long text or URL
        if X_train[col].astype(str).str.len().mean() > 100 or \
           X_train[col].astype(str).str.contains("http|www|pdf|doc", case=False).mean() > 0.3:
            non_informative_cols.append(col)
        # Drop if almost every value is unique (like IDs)
        elif X_train[col].nunique(dropna=True) > 0.9 * len(X_train):
            non_informative_cols.append(col)

print("🔎 Non-informative columns to drop:\n", non_informative_cols)


In [ ]:
# Column	Why Drop?
# Family Summary	Long free-text with highly variable length — better for NLP than tabular
# Collection Description(s)	Often redundant and descriptive, not structured
# Document Content URL	Just a URL — no predictive value
# Internal Document ID	Unique ID — not informative, high-cardinality

In [ ]:

# === Drop non-use variables from both train and test ===
X_train.drop(columns=non_informative_cols, inplace=True, errors='ignore')
X_test.drop(columns=non_informative_cols, inplace=True, errors='ignore')


In [ ]:

# === (e) Convert object columns to numeric where possible ===
for df in [X_train, X_test]:
    for col in df.select_dtypes(include='object').columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')



In [ ]:

# === (f) One-hot encode selected categorical variables ===
categorical_vars = [
 "Document Type", "Topic/Response", "Sector", "Instrument",
    "Category", "Framework", "Hazard", "Author", "Author Type",
    "Geographies", "Source", "Geography ISOs"  # <-- New additions
]
categorical_vars = [col for col in categorical_vars if col in X_train.columns]
X_train = pd.get_dummies(X_train, columns=categorical_vars, dummy_na=True)
X_test = pd.get_dummies(X_test, columns=categorical_vars, dummy_na=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)


In [ ]:
# Step 1: Check remaining numeric columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
print(f"🔢 Total numeric columns: {len(numeric_cols)}")

# Step 2: Count missing values per numeric column
missing_train = X_train[numeric_cols].isna().sum()
missing_train = missing_train[missing_train > 0]

print("\n📌 Numeric columns with missing values in training set:")
print(missing_train)

# Step 3: Optional – show percentage missing
percent_missing = (missing_train / len(X_train) * 100).round(2)
print("\n📊 Percentage missing per column:")
print(percent_missing.sort_values(ascending=False))

# Step 4: Show sample rows with missing values in top offender column
if not missing_train.empty:
    top_col = missing_train.idxmax()
    print(f"\n🔎 Sample rows with missing in '{top_col}':")
    display(X_train[X_train[top_col].isna()].head())


✅ Summary of Missingness

| Column                          | Missing (%) | Reason or Type              | Action                                     |
|----------------------------------|--------------|------------------------------|---------------------------------------------|
| `Collection Title(s)`            | 100%         | Metadata, unused             | Drop                                     |
| `Collection Description(s)`      | 100%         | Metadata, unused             | Drop                                     |
| `Document Variant`               | 100%         | Metadata                     | Drop                                     |
| `Language`                       | 100%         | Non-informative              | Drop                                     |
| `First event in timeline`        | 100%         | Raw date field               | Drop (already extracted year)            |
| `Last event in timeline`         | 100%         | Raw date field               | Drop                                     |
| `Full timeline of events (dates)`| 100%         | Raw date field               | Drop                                     |
| `event_timeline`                 | 100%         | List of tuples               | Drop for ML; useful for NLP only         |
| `Date Added to System`           | 100%         | System metadata              | Drop                                     |
| `Last Modified on System`        | 100%         | System metadata              | Drop                                     |
| `Internal Family ID`             | 100%         | Internal ID                  | Drop                                     |
| `Internal Corpus ID`             | 100%         | Internal ID                  | Drop                                     |
| `Internal Collection ID(s)`      | 100%         | Internal ID                  | Drop                                     |
| `Document Role`                  | 100%         | Possibly encoded elsewhere   | Drop                                     |
| `Keyword`                        | 100%         | Free text                    | Drop                                     |


In [ ]:
cols_to_drop = [
    'Collection Title(s)', 'Collection Description(s)', 'Document Variant', 'Language',
    'First event in timeline', 'Last event in timeline', 'Full timeline of events (dates)',
    'event_timeline', 'Date Added to System', 'Last Modified on System',
    'Internal Family ID', 'Internal Corpus ID', 'Internal Collection ID(s)',
    'Document Role', 'Keyword'
]

X_train.drop(columns=[col for col in cols_to_drop if col in X_train.columns], inplace=True)
X_test.drop(columns=[col for col in cols_to_drop if col in X_test.columns], inplace=True)
print("✅ Dropped 100% missing / non-informative columns.")


In [ ]:
# === (g) Impute missing values safely ===

# Identify numeric columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Drop numeric columns with all NaNs in training set
all_nan_cols = [col for col in numeric_cols if X_train[col].isna().all()]
X_train.drop(columns=all_nan_cols, inplace=True)
X_test.drop(columns=all_nan_cols, errors='ignore', inplace=True)

# Re-identify numeric columns after dropping
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Impute if we still have numeric columns
if numeric_cols:
    imputer = SimpleImputer(strategy="mean")
    X_train[numeric_cols] = imputer.fit_transform(X_train[numeric_cols])
    X_test[numeric_cols] = imputer.transform(X_test[numeric_cols])
else:
    print("⚠️ No numeric columns left for imputation.")


# ✅ Confirm imputation worked
print("✅ Remaining numeric columns:", len(numeric_cols))
print("🔍 Any missing values left in training set?", X_train[numeric_cols].isna().any().any())
print("🔍 Any missing values left in test set?", X_test[numeric_cols].isna().any().any())

# Optional: Preview a few numeric columns
X_train[numeric_cols].head()


In [ ]:
# Drop datetime columns that is not useful
X_train.drop(columns=["first_passed_date", "last_event_date"], inplace=True, errors="ignore")
X_test.drop(columns=["first_passed_date", "last_event_date"], inplace=True, errors="ignore")

print("✅ Dropped datetime columns to fix not useful issue.")


**[d] --- --- --- model**

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix
)
import numpy as np
import xgboost as xgb

# === (a) Cross-validation setup ===
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f"✅ Calculated scale_pos_weight: {scale_pos_weight:.4f}")

# === (b) Storage for metrics ===
metrics = {
    'accuracy': [], 'precision': [], 'recall': [],
    'f1': [], 'auc': [], 'specificity': []
}
all_y_true = []
all_y_pred = []

# === (c) Cross-validation loop ===
fold_num = 1
for train_idx, val_idx in cv.split(X_train, y_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    model_cv = xgb.XGBClassifier(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.2,
        n_jobs=-1,
        eval_metric='logloss',
        scale_pos_weight=scale_pos_weight,
        use_label_encoder=False,
        random_state=42
    )
    model_cv.fit(X_tr, y_tr)

    y_proba = model_cv.predict_proba(X_val)[:, 1]
    y_pred = (y_proba >= 0.5).astype(int)

    # Save out-of-fold predictions
    all_y_true.extend(y_val)
    all_y_pred.extend(y_pred)

    # Confusion matrix
    cm = confusion_matrix(y_val, y_pred)
    TN, FP, FN, TP = cm.ravel()

    metrics['accuracy'].append(accuracy_score(y_val, y_pred))
    metrics['precision'].append(precision_score(y_val, y_pred, zero_division=0))
    metrics['recall'].append(recall_score(y_val, y_pred))
    metrics['f1'].append(f1_score(y_val, y_pred))
    metrics['auc'].append(roc_auc_score(y_val, y_proba))
    metrics['specificity'].append(TN / (TN + FP))

    print(f"✅ Fold {fold_num} completed.")
    fold_num += 1

# === (d) Display CV metrics ===
print("\n=== 📊 5-Fold CV Metrics (Default Cutoff = 0.5) ===")
for key in metrics:
    mean = np.mean(metrics[key])
    std = np.std(metrics[key])
    print(f"{key.capitalize():<12}: {mean:.4f} ± {std:.4f}")

# === (e) Final aggregated performance ===
final_cm = confusion_matrix(all_y_true, all_y_pred)
TN, FP, FN, TP = final_cm.ravel()
print("\n=== 🔎 Final Aggregated Performance on All Out-of-Fold Predictions ===")
print("Confusion Matrix:\n", final_cm)
print(f"Specificity: {TN / (TN + FP):.4f}")


## 7.5. Neural network with imbalance control

In [ ]:
# === (a) Import Libraries ===
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    accuracy_score, roc_auc_score, confusion_matrix,
    mean_squared_error, brier_score_loss
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler

# === (b) Load Data ===
train = pd.read_csv("weak_labeled_policy_action_dataset_strict2.csv", encoding="ISO-8859-1")
test = pd.read_csv("classified_climate_policy_actions.csv", encoding="ISO-8859-1")


**[a] --- --- --- Dealing the time_event (single time: event_year_first and event_year_last)**

In [ ]:
from dateutil import parser

def safe_parse_year(text):
    if pd.isna(text):
        return np.nan
    try:
        return parser.parse(str(text), dayfirst=False).year
    except:
        try:
            return parser.parse(str(text), dayfirst=True).year
        except:
            return np.nan

train["event_year_first"] = train["First event in timeline"].apply(safe_parse_year)
train["event_year_last"] = train["Last event in timeline"].apply(safe_parse_year)

test["event_year_first"] = test["First event in timeline"].apply(safe_parse_year)
test["event_year_last"] = test["Last event in timeline"].apply(safe_parse_year)


In [ ]:
# === Check types and summary stats ===
print("🔍 Column types:\n", train[["event_year_first", "event_year_last"]].dtypes)

print("\n📊 Descriptive stats for train:")
print(train[["event_year_first", "event_year_last"]].describe())

print("\n📊 Descriptive stats for test:")
print(test[["event_year_first", "event_year_last"]].describe())

# === Check value counts (frequency) ===
print("\n🗓 Top 10 most frequent 'event_year_first':")
print(train["event_year_first"].value_counts(dropna=False).sort_index().tail(10))

print("\n🗓 Top 10 most frequent 'event_year_last':")
print(train["event_year_last"].value_counts(dropna=False).sort_index().tail(10))

# Show first 20 rows for event_year_first and event_year_last in training set
print("📄 First 20 rows (Training Data):")
print(train[["event_year_first", "event_year_last"]].head(20))

# Show first 20 rows for event_year_first and event_year_last in testing set
print("\n📄 First 20 rows (Testing Data):")
print(test[["event_year_first", "event_year_last"]].head(20))


**[b] --- --- --- Dealing with multiple time event within 1 variable**

In [ ]:
from dateutil import parser

def safe_parse(date_str):
    try:
        return parser.parse(date_str, dayfirst=False)
    except:
        try:
            return parser.parse(date_str, dayfirst=True)
        except:
            return None

def extract_sorted_events(row):
    types = row.get("Full timeline of events (types)")
    dates = row.get("Full timeline of events (dates)")
    
    if pd.isna(types) or pd.isna(dates):
        return None
    
    type_list = [t.strip() for t in str(types).split(";")]
    date_list = [d.strip() for d in str(dates).split(";")]
    
    if len(type_list) != len(date_list):
        return None  # mismatched length
    
    paired = [(t, safe_parse(d)) for t, d in zip(type_list, date_list)]
    paired = [x for x in paired if x[1] is not None]  # drop invalid
    return sorted(paired, key=lambda x: x[1])

train["event_timeline"] = train.apply(extract_sorted_events, axis=1)
test["event_timeline"] = test.apply(extract_sorted_events, axis=1)


In [ ]:
def order_after(df, move_col, after_col):
    cols = list(df.columns)
    cols.remove(move_col)
    insert_at = cols.index(after_col) + 1
    cols.insert(insert_at, move_col)
    return df[cols]

train = order_after(train, "event_timeline", "Full timeline of events (dates)")
test = order_after(test, "event_timeline", "Full timeline of events (dates)")


In [ ]:
import numpy as np

for df in [train, test]:
    # 1. Count of amendments
    df["num_amendments"] = df["event_timeline"].apply(
        lambda x: sum(1 for t, _ in x if t == "Amended") if x else 0
    )

    # 2. First "Passed/Approved" date
    df["first_passed_date"] = df["event_timeline"].apply(
        lambda x: next((d for t, d in x if t == "Passed/Approved"), None) if x else None
    )

    # 3. Last event date (regardless of type)
    df["last_event_date"] = df["event_timeline"].apply(
        lambda x: x[-1][1] if x else None
    )

    # 4. Duration in years
    df["policy_duration_years"] = df.apply(
        lambda row: (row["last_event_date"] - row["first_passed_date"]).days / 365.25
        if row["last_event_date"] and row["first_passed_date"] else np.nan,
        axis=1
    )

print(train[["first_passed_date", "last_event_date", "num_amendments", "policy_duration_years"]].head(10))


In [ ]:
# Convert to datetime if not already
for df in [train, test]:
    df["first_passed_date"] = pd.to_datetime(df["first_passed_date"], errors="coerce")
    df["last_event_date"] = pd.to_datetime(df["last_event_date"], errors="coerce")

    # Extract year after conversion
    df["event_year_passed"] = df["first_passed_date"].dt.year
    df["event_year_last_event"] = df["last_event_date"].dt.year

# ✅ Preview
print("🔍 Sample extracted years:")
print(train[["first_passed_date", "event_year_passed", "last_event_date", "event_year_last_event"]].dropna().head())


**[c] --- --- --- Extra cleaning**

In [ ]:

# === (d) Define target and features ===
target = "policy_action"
exclude_cols = [
    "Document ID", "Document Title", "Family ID", "Family Title",
    "Full timeline of events (types)", target
]
feature_cols = [col for col in train.columns if col not in exclude_cols]

X_train = train[feature_cols].copy()
y_train = train[target]

X_test = test[feature_cols].copy()
y_test = test[target]



In [ ]:
non_informative_cols = []

for col in X_train.columns:
    if X_train[col].dtype == "object":
        # Skip if any row contains lists (unhashable)
        if X_train[col].apply(lambda x: isinstance(x, list)).any():
            continue
        # Drop if it's long text or URL
        if X_train[col].astype(str).str.len().mean() > 100 or \
           X_train[col].astype(str).str.contains("http|www|pdf|doc", case=False).mean() > 0.3:
            non_informative_cols.append(col)
        # Drop if almost every value is unique (like IDs)
        elif X_train[col].nunique(dropna=True) > 0.9 * len(X_train):
            non_informative_cols.append(col)

print("🔎 Non-informative columns to drop:\n", non_informative_cols)


In [ ]:
# Column	Why Drop?
# Family Summary	Long free-text with highly variable length — better for NLP than tabular
# Collection Description(s)	Often redundant and descriptive, not structured
# Document Content URL	Just a URL — no predictive value
# Internal Document ID	Unique ID — not informative, high-cardinality

In [ ]:

# === Drop non-use variables from both train and test ===
X_train.drop(columns=non_informative_cols, inplace=True, errors='ignore')
X_test.drop(columns=non_informative_cols, inplace=True, errors='ignore')


In [ ]:

# === (e) Convert object columns to numeric where possible ===
for df in [X_train, X_test]:
    for col in df.select_dtypes(include='object').columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')



In [ ]:

# === (f) One-hot encode selected categorical variables ===
categorical_vars = [
 "Document Type", "Topic/Response", "Sector", "Instrument",
    "Category", "Framework", "Hazard", "Author", "Author Type",
    "Geographies", "Source", "Geography ISOs"  # <-- New additions
]
categorical_vars = [col for col in categorical_vars if col in X_train.columns]
X_train = pd.get_dummies(X_train, columns=categorical_vars, dummy_na=True)
X_test = pd.get_dummies(X_test, columns=categorical_vars, dummy_na=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)


In [ ]:
# Step 1: Check remaining numeric columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
print(f"🔢 Total numeric columns: {len(numeric_cols)}")

# Step 2: Count missing values per numeric column
missing_train = X_train[numeric_cols].isna().sum()
missing_train = missing_train[missing_train > 0]

print("\n📌 Numeric columns with missing values in training set:")
print(missing_train)

# Step 3: Optional – show percentage missing
percent_missing = (missing_train / len(X_train) * 100).round(2)
print("\n📊 Percentage missing per column:")
print(percent_missing.sort_values(ascending=False))

# Step 4: Show sample rows with missing values in top offender column
if not missing_train.empty:
    top_col = missing_train.idxmax()
    print(f"\n🔎 Sample rows with missing in '{top_col}':")
    display(X_train[X_train[top_col].isna()].head())


✅ Summary of Missingness

| Column                          | Missing (%) | Reason or Type              | Action                                     |
|----------------------------------|--------------|------------------------------|---------------------------------------------|
| `Collection Title(s)`            | 100%         | Metadata, unused             | Drop                                     |
| `Collection Description(s)`      | 100%         | Metadata, unused             | Drop                                     |
| `Document Variant`               | 100%         | Metadata                     | Drop                                     |
| `Language`                       | 100%         | Non-informative              | Drop                                     |
| `First event in timeline`        | 100%         | Raw date field               | Drop (already extracted year)            |
| `Last event in timeline`         | 100%         | Raw date field               | Drop                                     |
| `Full timeline of events (dates)`| 100%         | Raw date field               | Drop                                     |
| `event_timeline`                 | 100%         | List of tuples               | Drop for ML; useful for NLP only         |
| `Date Added to System`           | 100%         | System metadata              | Drop                                     |
| `Last Modified on System`        | 100%         | System metadata              | Drop                                     |
| `Internal Family ID`             | 100%         | Internal ID                  | Drop                                     |
| `Internal Corpus ID`             | 100%         | Internal ID                  | Drop                                     |
| `Internal Collection ID(s)`      | 100%         | Internal ID                  | Drop                                     |
| `Document Role`                  | 100%         | Possibly encoded elsewhere   | Drop                                     |
| `Keyword`                        | 100%         | Free text                    | Drop                                     |


In [ ]:
cols_to_drop = [
    'Collection Title(s)', 'Collection Description(s)', 'Document Variant', 'Language',
    'First event in timeline', 'Last event in timeline', 'Full timeline of events (dates)',
    'event_timeline', 'Date Added to System', 'Last Modified on System',
    'Internal Family ID', 'Internal Corpus ID', 'Internal Collection ID(s)',
    'Document Role', 'Keyword'
]

X_train.drop(columns=[col for col in cols_to_drop if col in X_train.columns], inplace=True)
X_test.drop(columns=[col for col in cols_to_drop if col in X_test.columns], inplace=True)
print("✅ Dropped 100% missing / non-informative columns.")


In [ ]:
# === (g) Impute missing values safely ===

# Identify numeric columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Drop numeric columns with all NaNs in training set
all_nan_cols = [col for col in numeric_cols if X_train[col].isna().all()]
X_train.drop(columns=all_nan_cols, inplace=True)
X_test.drop(columns=all_nan_cols, errors='ignore', inplace=True)

# Re-identify numeric columns after dropping
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Impute if we still have numeric columns
if numeric_cols:
    imputer = SimpleImputer(strategy="mean")
    X_train[numeric_cols] = imputer.fit_transform(X_train[numeric_cols])
    X_test[numeric_cols] = imputer.transform(X_test[numeric_cols])
else:
    print("⚠️ No numeric columns left for imputation.")


# ✅ Confirm imputation worked
print("✅ Remaining numeric columns:", len(numeric_cols))
print("🔍 Any missing values left in training set?", X_train[numeric_cols].isna().any().any())
print("🔍 Any missing values left in test set?", X_test[numeric_cols].isna().any().any())

# Optional: Preview a few numeric columns
X_train[numeric_cols].head()


In [ ]:
# Drop datetime columns that is not useful
X_train.drop(columns=["first_passed_date", "last_event_date"], inplace=True, errors="ignore")
X_test.drop(columns=["first_passed_date", "last_event_date"], inplace=True, errors="ignore")

print("✅ Dropped datetime columns to fix XGBoost training issue.")


**[d] --- --- --- Extra cleaning before Neural network**


In [ ]:

# === (j) Normalize features ===
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

**[e] --- --- --- Model**

In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# === (a) Compute Class Weights ===
classes = np.unique(y_train)
weights = compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))
print("✅ Computed class weights:", class_weights)

# === (b) Define Neural Network ===
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# === (c) Compile Model ===
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# === (d) Fit Model with Early Stopping and Class Weights ===
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=64,
    callbacks=[early_stop],
    class_weight=class_weights,
    verbose=1
)


In [ ]:
# === (l) Predict ===

# ✅ Step 1: Identify valid test indices (non-missing labels)
valid_idx = ~y_test.isna()

# ✅ Step 2: Filter out missing test labels
y_test_clean = y_test[valid_idx].astype(int)
X_test_clean = X_test_scaled[valid_idx]

# ✅ Step 3: Predict probabilities and class labels
y_proba = model.predict(X_test_clean, verbose=0).flatten()  # Get predicted probabilities
y_pred = (y_proba > 0.5).astype(int)                         # Default threshold: 0.5


In [ ]:
# ✅ Step 1: Diagnose test set
print("🔍 Total length of y_test:", len(y_test))
print("❌ Missing values in y_test:", y_test.isna().sum())

# Optional: show problematic test rows (i.e., where no label is available)
missing_rows = X_test[y_test.isna()]
display(missing_rows.head())

print("🧯 Missing y_test entries:")
print(y_test[y_test.isna()])



In [ ]:
# ✅ Step 2: Filter out NaN labels from y_test only
valid_idx = ~y_test.isna()

y_test_clean = y_test[valid_idx].astype(int)  # Ensure binary labels for evaluation
y_pred_clean = y_pred             # Already of length 200
y_proba_clean = y_proba           # Already of length 200


In [ ]:
# === (m) Evaluate Neural Network Model ===
print(f"✅ Accuracy:     {accuracy_score(y_test_clean, y_pred):.3f}")
print(f"✅ AUC:          {roc_auc_score(y_test_clean, y_proba):.3f}")
print(f"✅ Brier Score:  {brier_score_loss(y_test_clean, y_proba):.4f}")
print(f"✅ MSE:          {mean_squared_error(y_test_clean, y_proba):.4f}")

cm = confusion_matrix(y_test_clean, y_pred)
print("✅ Confusion Matrix:\n", cm)

# Optional: Unpack and display confusion matrix in labeled form
TN, FP, FN, TP = cm.ravel()
print(f"   TN: {TN}, FP: {FP}, FN: {FN}, TP: {TP}")



In [ ]:
# === (ii) Enhanced Confusion Matrix (Stata-style) ===
cm = confusion_matrix(y_test_clean, y_pred_clean, labels=[0, 1])
TN, FP, FN, TP = cm.ravel()

row_perc = cm / cm.sum(axis=1, keepdims=True) * 100
col_perc = cm / cm.sum(axis=0, keepdims=True) * 100
col_totals = cm.sum(axis=0)
grand_total = cm.sum()

print("\n+-------------------+")
print("| Key               |")
print("|-------------------|")
print("|     frequency     |")
print("|  row percentage   |")
print("| column percentage |")
print("+-------------------+\n")

print("policy_action |   predicted")
print("              |     0        1     |   Total")
print("--------------+--------------------+----------")

labels = ["0", "1"]
for i, label in enumerate(labels):
    row = cm[i]
    row_pct = row_perc[i]
    col_pct = col_perc[i]
    row_total = row.sum()
    print(f"         {label} | {row[0]:10,} {row[1]:10,} | {row_total:8,}")
    print(f"              | {row_pct[0]:10.2f} {row_pct[1]:10.2f} |   100.00")
    print(f"              | {col_pct[0]:10.2f} {col_pct[1]:10.2f} |")
    print("--------------+--------------------+----------")

col_total_str = " ".join([f"{val:10,}" for val in col_totals])
col_perc_str = " ".join([f"{(val/grand_total*100):10.2f}" for val in col_totals])
print(f"     Total   | {col_total_str} | {grand_total:8,}")
print(f"              | {col_perc_str} |   100.00")
print("              |    100.00     100.00 |   100.00")

# === (iii) Scalar-style metrics ===
Accuracy    = (TP + TN) / grand_total
Precision   = TP / (TP + FP) if (TP + FP) else 0.0
Recall      = TP / (TP + FN) if (TP + FN) else 0.0
F1          = 2 * (Precision * Recall) / (Precision + Recall) if (Precision + Recall) else 0.0
Specificity = TN / (TN + FP) if (TN + FP) else 0.0

print("\n📐 Scalar-style Evaluation Metrics:")
print(f"Accuracy:        {Accuracy:.8f}")
print(f"Precision:       {Precision:.8f}")
print(f"Recall:          {Recall:.8f}")
print(f"F1 Score:        {F1:.8f}")
print(f"Specificity:     {Specificity:.8f}")


In [ ]:
# === (n) Save Results ===
test_clean = test.loc[valid_idx].copy()
test_clean["nn_pred"] = y_pred
test_clean["nn_prob"] = y_proba
test_clean.to_csv("imbalneuralnet_predictions_policyaction.csv", index=False)
print("📁 Predictions saved to imbalneuralnet_predictions_policyaction.csv")


**Python Code for ROC Threshold Evaluation and Export**

In [ ]:
import pandas as pd
import numpy as np

# === (1) Load final predictions ===
df = pd.read_csv("imbalneuralnet_predictions_policyaction.csv")

# === (2) Define column names
score_col = "nn_prob"         # Probabilities from XGBoost
actual_col = "policy_action"   # True label from gold test set

# === (3) Filter clean data ===
df = df[[score_col, actual_col]].dropna()
df['score_round'] = df[score_col].round(4)  # Round for consistency

# === (4) Prepare ROC table ===
thresholds = np.unique(df['score_round'])
results = []

P = (df[actual_col] == 1).sum()
N = (df[actual_col] == 0).sum()

for thresh in thresholds:
    pred = (df[score_col] > thresh).astype(int)
    TP = ((df[actual_col] == 1) & (pred == 1)).sum()
    FP = ((df[actual_col] == 0) & (pred == 1)).sum()
    FN = ((df[actual_col] == 1) & (pred == 0)).sum()
    TN = ((df[actual_col] == 0) & (pred == 0)).sum()

    TPR = TP / P if P > 0 else 0
    FPR = FP / N if N > 0 else 0
    distance = np.sqrt((1 - TPR) ** 2 + (FPR) ** 2)

    results.append({
        "cutoff": thresh,
        "TPR": TPR,
        "FPR": FPR,
        "distance": distance,
        "TP": TP,
        "FP": FP,
        "TN": TN,
        "FN": FN
    })

# === (5) Convert to DataFrame ===
roc_df = pd.DataFrame(results)

# === (6) Find best threshold ===
best_row = roc_df.loc[roc_df['distance'].idxmin()]
best_cutoff = best_row['cutoff']

print("\n🎯 Optimal Threshold Results")
print(f"Best cutoff       : {best_cutoff:.4f}")
print(f"True Positive Rate: {best_row['TPR']:.4f}")
print(f"False Positive Rate: {best_row['FPR']:.4f}")
print(f"Min Distance      : {best_row['distance']:.4f}")


In [ ]:

# === (7) Save all ROC points to CSV for review ===
roc_df.to_csv("rocpoints_imbalneuralnet.csv", index=False)
print("📁 All thresholds and ROC stats saved to: rocpoints_imbalneuralnet.csv")


In [ ]:
from sklearn.metrics import confusion_matrix

# === (1) Apply Best Cutoff ===
best_cutoff =  0.3612  # Replace with actual best threshold
df["final_pred"] = (df["nn_prob"] > best_cutoff).astype(int)

# Extract clean y_true and y_pred (already NaNs dropped earlier)
y_true = df["policy_action"]
y_pred = df["final_pred"]

# === (2) Compute Confusion Matrix ===
cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
TN, FP, FN, TP = cm.ravel()

row_perc = cm / cm.sum(axis=1, keepdims=True) * 100
col_perc = cm / cm.sum(axis=0, keepdims=True) * 100
col_totals = cm.sum(axis=0)
grand_total = cm.sum()

# === (3) Stata-style Table Output ===
print("\n+-------------------+")
print("| Key               |")
print("|-------------------|")
print("|     frequency     |")
print("|  row percentage   |")
print("| column percentage |")
print("+-------------------+\n")

print("policy_action |   predicted")
print("             |     0        1     |   Total")
print("---------------+--------------------+----------")

labels = ["0", "1"]
for i, label in enumerate(labels):
    row = cm[i]
    row_pct = row_perc[i]
    col_pct = col_perc[i]
    row_total = row.sum()
    print(f"         {label} | {row[0]:10,} {row[1]:10,} | {row_total:8,}")
    print(f"           | {row_pct[0]:10.2f} {row_pct[1]:10.2f} |   100.00")
    print(f"           | {col_pct[0]:10.2f} {col_pct[1]:10.2f} |")
    print("---------------+--------------------+----------")

col_total_str = " ".join([f"{val:10,}" for val in col_totals])
col_perc_str = " ".join([f"{(val/grand_total*100):10.2f}" for val in col_totals])
print(f"     Total | {col_total_str} | {grand_total:8,}")
print(f"           | {col_perc_str} |   100.00")
print("           |    100.00     100.00 |   100.00")

# === (4) Scalar Metrics ===
Accuracy    = (TP + TN) / grand_total
Precision   = TP / (TP + FP) if (TP + FP) else 0.0
Recall      = TP / (TP + FN) if (TP + FN) else 0.0
F1          = 2 * (Precision * Recall) / (Precision + Recall) if (Precision + Recall) else 0.0
Specificity = TN / (TN + FP) if (TN + FP) else 0.0

print("\n📐 Scalar-style Evaluation Metrics:")
print(f"Accuracy:        {Accuracy:.8f}")
print(f"Precision:       {Precision:.8f}")
print(f"Recall:          {Recall:.8f}")
print(f"F1 Score:        {F1:.8f}")
print(f"Specificity:     {Specificity:.8f}")


## 7.6. Neural network with imbalance control and cross-validation

In [ ]:
# === (a) Import Libraries ===
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    accuracy_score, roc_auc_score, confusion_matrix,
    mean_squared_error, brier_score_loss
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler

# === (b) Load Data ===
train = pd.read_csv("weak_labeled_policy_action_dataset_strict2.csv", encoding="ISO-8859-1")
test = pd.read_csv("classified_climate_policy_actions.csv", encoding="ISO-8859-1")


**[a] --- --- --- Dealing the time_event (single time: event_year_first and event_year_last)**

In [ ]:
from dateutil import parser

def safe_parse_year(text):
    if pd.isna(text):
        return np.nan
    try:
        return parser.parse(str(text), dayfirst=False).year
    except:
        try:
            return parser.parse(str(text), dayfirst=True).year
        except:
            return np.nan

train["event_year_first"] = train["First event in timeline"].apply(safe_parse_year)
train["event_year_last"] = train["Last event in timeline"].apply(safe_parse_year)

test["event_year_first"] = test["First event in timeline"].apply(safe_parse_year)
test["event_year_last"] = test["Last event in timeline"].apply(safe_parse_year)


In [ ]:
# === Check types and summary stats ===
print("🔍 Column types:\n", train[["event_year_first", "event_year_last"]].dtypes)

print("\n📊 Descriptive stats for train:")
print(train[["event_year_first", "event_year_last"]].describe())

print("\n📊 Descriptive stats for test:")
print(test[["event_year_first", "event_year_last"]].describe())

# === Check value counts (frequency) ===
print("\n🗓 Top 10 most frequent 'event_year_first':")
print(train["event_year_first"].value_counts(dropna=False).sort_index().tail(10))

print("\n🗓 Top 10 most frequent 'event_year_last':")
print(train["event_year_last"].value_counts(dropna=False).sort_index().tail(10))

# Show first 20 rows for event_year_first and event_year_last in training set
print("📄 First 20 rows (Training Data):")
print(train[["event_year_first", "event_year_last"]].head(20))

# Show first 20 rows for event_year_first and event_year_last in testing set
print("\n📄 First 20 rows (Testing Data):")
print(test[["event_year_first", "event_year_last"]].head(20))


**[b] --- --- --- Dealing with multiple time event within 1 variable**

In [ ]:
from dateutil import parser

def safe_parse(date_str):
    try:
        return parser.parse(date_str, dayfirst=False)
    except:
        try:
            return parser.parse(date_str, dayfirst=True)
        except:
            return None

def extract_sorted_events(row):
    types = row.get("Full timeline of events (types)")
    dates = row.get("Full timeline of events (dates)")
    
    if pd.isna(types) or pd.isna(dates):
        return None
    
    type_list = [t.strip() for t in str(types).split(";")]
    date_list = [d.strip() for d in str(dates).split(";")]
    
    if len(type_list) != len(date_list):
        return None  # mismatched length
    
    paired = [(t, safe_parse(d)) for t, d in zip(type_list, date_list)]
    paired = [x for x in paired if x[1] is not None]  # drop invalid
    return sorted(paired, key=lambda x: x[1])

train["event_timeline"] = train.apply(extract_sorted_events, axis=1)
test["event_timeline"] = test.apply(extract_sorted_events, axis=1)


In [ ]:
def order_after(df, move_col, after_col):
    cols = list(df.columns)
    cols.remove(move_col)
    insert_at = cols.index(after_col) + 1
    cols.insert(insert_at, move_col)
    return df[cols]

train = order_after(train, "event_timeline", "Full timeline of events (dates)")
test = order_after(test, "event_timeline", "Full timeline of events (dates)")


In [ ]:
import numpy as np

for df in [train, test]:
    # 1. Count of amendments
    df["num_amendments"] = df["event_timeline"].apply(
        lambda x: sum(1 for t, _ in x if t == "Amended") if x else 0
    )

    # 2. First "Passed/Approved" date
    df["first_passed_date"] = df["event_timeline"].apply(
        lambda x: next((d for t, d in x if t == "Passed/Approved"), None) if x else None
    )

    # 3. Last event date (regardless of type)
    df["last_event_date"] = df["event_timeline"].apply(
        lambda x: x[-1][1] if x else None
    )

    # 4. Duration in years
    df["policy_duration_years"] = df.apply(
        lambda row: (row["last_event_date"] - row["first_passed_date"]).days / 365.25
        if row["last_event_date"] and row["first_passed_date"] else np.nan,
        axis=1
    )

print(train[["first_passed_date", "last_event_date", "num_amendments", "policy_duration_years"]].head(10))


In [ ]:
# Convert to datetime if not already
for df in [train, test]:
    df["first_passed_date"] = pd.to_datetime(df["first_passed_date"], errors="coerce")
    df["last_event_date"] = pd.to_datetime(df["last_event_date"], errors="coerce")

    # Extract year after conversion
    df["event_year_passed"] = df["first_passed_date"].dt.year
    df["event_year_last_event"] = df["last_event_date"].dt.year

# ✅ Preview
print("🔍 Sample extracted years:")
print(train[["first_passed_date", "event_year_passed", "last_event_date", "event_year_last_event"]].dropna().head())


**[c] --- --- --- Extra cleaning**

In [ ]:

# === (d) Define target and features ===
target = "policy_action"
exclude_cols = [
    "Document ID", "Document Title", "Family ID", "Family Title",
    "Full timeline of events (types)", target
]
feature_cols = [col for col in train.columns if col not in exclude_cols]

X_train = train[feature_cols].copy()
y_train = train[target]

X_test = test[feature_cols].copy()
y_test = test[target]



In [ ]:
non_informative_cols = []

for col in X_train.columns:
    if X_train[col].dtype == "object":
        # Skip if any row contains lists (unhashable)
        if X_train[col].apply(lambda x: isinstance(x, list)).any():
            continue
        # Drop if it's long text or URL
        if X_train[col].astype(str).str.len().mean() > 100 or \
           X_train[col].astype(str).str.contains("http|www|pdf|doc", case=False).mean() > 0.3:
            non_informative_cols.append(col)
        # Drop if almost every value is unique (like IDs)
        elif X_train[col].nunique(dropna=True) > 0.9 * len(X_train):
            non_informative_cols.append(col)

print("🔎 Non-informative columns to drop:\n", non_informative_cols)


In [ ]:
# Column	Why Drop?
# Family Summary	Long free-text with highly variable length — better for NLP than tabular
# Collection Description(s)	Often redundant and descriptive, not structured
# Document Content URL	Just a URL — no predictive value
# Internal Document ID	Unique ID — not informative, high-cardinality

In [ ]:

# === Drop non-use variables from both train and test ===
X_train.drop(columns=non_informative_cols, inplace=True, errors='ignore')
X_test.drop(columns=non_informative_cols, inplace=True, errors='ignore')


In [ ]:

# === (e) Convert object columns to numeric where possible ===
for df in [X_train, X_test]:
    for col in df.select_dtypes(include='object').columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')



In [ ]:

# === (f) One-hot encode selected categorical variables ===
categorical_vars = [
 "Document Type", "Topic/Response", "Sector", "Instrument",
    "Category", "Framework", "Hazard", "Author", "Author Type",
    "Geographies", "Source", "Geography ISOs"  # <-- New additions
]
categorical_vars = [col for col in categorical_vars if col in X_train.columns]
X_train = pd.get_dummies(X_train, columns=categorical_vars, dummy_na=True)
X_test = pd.get_dummies(X_test, columns=categorical_vars, dummy_na=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)


In [ ]:
# Step 1: Check remaining numeric columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
print(f"🔢 Total numeric columns: {len(numeric_cols)}")

# Step 2: Count missing values per numeric column
missing_train = X_train[numeric_cols].isna().sum()
missing_train = missing_train[missing_train > 0]

print("\n📌 Numeric columns with missing values in training set:")
print(missing_train)

# Step 3: Optional – show percentage missing
percent_missing = (missing_train / len(X_train) * 100).round(2)
print("\n📊 Percentage missing per column:")
print(percent_missing.sort_values(ascending=False))

# Step 4: Show sample rows with missing values in top offender column
if not missing_train.empty:
    top_col = missing_train.idxmax()
    print(f"\n🔎 Sample rows with missing in '{top_col}':")
    display(X_train[X_train[top_col].isna()].head())


✅ Summary of Missingness

| Column                          | Missing (%) | Reason or Type              | Action                                     |
|----------------------------------|--------------|------------------------------|---------------------------------------------|
| `Collection Title(s)`            | 100%         | Metadata, unused             | Drop                                     |
| `Collection Description(s)`      | 100%         | Metadata, unused             | Drop                                     |
| `Document Variant`               | 100%         | Metadata                     | Drop                                     |
| `Language`                       | 100%         | Non-informative              | Drop                                     |
| `First event in timeline`        | 100%         | Raw date field               | Drop (already extracted year)            |
| `Last event in timeline`         | 100%         | Raw date field               | Drop                                     |
| `Full timeline of events (dates)`| 100%         | Raw date field               | Drop                                     |
| `event_timeline`                 | 100%         | List of tuples               | Drop for ML; useful for NLP only         |
| `Date Added to System`           | 100%         | System metadata              | Drop                                     |
| `Last Modified on System`        | 100%         | System metadata              | Drop                                     |
| `Internal Family ID`             | 100%         | Internal ID                  | Drop                                     |
| `Internal Corpus ID`             | 100%         | Internal ID                  | Drop                                     |
| `Internal Collection ID(s)`      | 100%         | Internal ID                  | Drop                                     |
| `Document Role`                  | 100%         | Possibly encoded elsewhere   | Drop                                     |
| `Keyword`                        | 100%         | Free text                    | Drop                                     |


In [ ]:
cols_to_drop = [
    'Collection Title(s)', 'Collection Description(s)', 'Document Variant', 'Language',
    'First event in timeline', 'Last event in timeline', 'Full timeline of events (dates)',
    'event_timeline', 'Date Added to System', 'Last Modified on System',
    'Internal Family ID', 'Internal Corpus ID', 'Internal Collection ID(s)',
    'Document Role', 'Keyword'
]

X_train.drop(columns=[col for col in cols_to_drop if col in X_train.columns], inplace=True)
X_test.drop(columns=[col for col in cols_to_drop if col in X_test.columns], inplace=True)
print("✅ Dropped 100% missing / non-informative columns.")


In [ ]:
# === (g) Impute missing values safely ===

# Identify numeric columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Drop numeric columns with all NaNs in training set
all_nan_cols = [col for col in numeric_cols if X_train[col].isna().all()]
X_train.drop(columns=all_nan_cols, inplace=True)
X_test.drop(columns=all_nan_cols, errors='ignore', inplace=True)

# Re-identify numeric columns after dropping
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Impute if we still have numeric columns
if numeric_cols:
    imputer = SimpleImputer(strategy="mean")
    X_train[numeric_cols] = imputer.fit_transform(X_train[numeric_cols])
    X_test[numeric_cols] = imputer.transform(X_test[numeric_cols])
else:
    print("⚠️ No numeric columns left for imputation.")


# ✅ Confirm imputation worked
print("✅ Remaining numeric columns:", len(numeric_cols))
print("🔍 Any missing values left in training set?", X_train[numeric_cols].isna().any().any())
print("🔍 Any missing values left in test set?", X_test[numeric_cols].isna().any().any())

# Optional: Preview a few numeric columns
X_train[numeric_cols].head()


In [ ]:
# Drop datetime columns that is not useful
X_train.drop(columns=["first_passed_date", "last_event_date"], inplace=True, errors="ignore")
X_test.drop(columns=["first_passed_date", "last_event_date"], inplace=True, errors="ignore")

print("✅ Dropped datetime columns to fix XGBoost training issue.")


**[d] --- --- --- Extra cleaning before Neural network**


In [ ]:

# === (j) Normalize features ===
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

**[e] --- --- --- Model**

In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix, classification_report
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

# === (a) Cross-validation setup ===
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

metrics = {
    'accuracy': [], 'precision': [], 'recall': [],
    'f1': [], 'auc': [], 'specificity': []
}
all_y_true = []
all_y_pred = []

# === (b) CV loop ===
fold_num = 1
for train_idx, val_idx in cv.split(X_train_scaled, y_train):
    X_tr, X_val = X_train_scaled[train_idx], X_train_scaled[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    # Compute class weights for this fold
    classes = np.unique(y_tr)
    weights = compute_class_weight(class_weight="balanced", classes=classes, y=y_tr)
    class_weights = dict(zip(classes, weights))

    # Build model
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_tr.shape[1],)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train
    model.fit(
        X_tr, y_tr,
        validation_data=(X_val, y_val),
        epochs=100,
        batch_size=64,
        class_weight=class_weights,
        callbacks=[early_stop],
        verbose=0
    )

    # Predict
    y_proba = model.predict(X_val).flatten()
    y_pred = (y_proba >= 0.5).astype(int)

    # Save results
    all_y_true.extend(y_val)
    all_y_pred.extend(y_pred)

    cm = confusion_matrix(y_val, y_pred)
    TN, FP, FN, TP = cm.ravel()

    metrics['accuracy'].append(accuracy_score(y_val, y_pred))
    metrics['precision'].append(precision_score(y_val, y_pred, zero_division=0))
    metrics['recall'].append(recall_score(y_val, y_pred))
    metrics['f1'].append(f1_score(y_val, y_pred))
    metrics['auc'].append(roc_auc_score(y_val, y_proba))
    metrics['specificity'].append(TN / (TN + FP))

    print(f"✅ Fold {fold_num} completed.")
    fold_num += 1

# === (c) Report average ± std
print("\n=== 📊 5-Fold CV Metrics (Neural Net, Default Cutoff = 0.5) ===")
for key in metrics:
    mean = np.mean(metrics[key])
    std = np.std(metrics[key])
    print(f"{key.capitalize():<12}: {mean:.4f} ± {std:.4f}")

# === (d) Final classification report
print("\n=== 🔎 Final Aggregated Evaluation ===")
print(confusion_matrix(all_y_true, all_y_pred))
print("\nClassification Report:")
print(classification_report(all_y_true, all_y_pred, digits=4))


## [8]. [Policy Action] Employ Imbalance Weight Xgboost (for final sample)

In [ ]:
# === (a) Import Libraries ===
import pandas as pd
import numpy as np
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, brier_score_loss


# === (b) Load Data ===
train = pd.read_csv("weak_labeled_policy_action_dataset_strict2.csv", encoding="ISO-8859-1")
test = pd.read_csv("classified_climate_policy_actions.csv", encoding="ISO-8859-1")


**[a] --- --- --- Dealing the time_event (single time: event_year_first and event_year_last)**

In [ ]:
from dateutil import parser

def safe_parse_year(text):
    if pd.isna(text):
        return np.nan
    try:
        return parser.parse(str(text), dayfirst=False).year
    except:
        try:
            return parser.parse(str(text), dayfirst=True).year
        except:
            return np.nan

train["event_year_first"] = train["First event in timeline"].apply(safe_parse_year)
train["event_year_last"] = train["Last event in timeline"].apply(safe_parse_year)

test["event_year_first"] = test["First event in timeline"].apply(safe_parse_year)
test["event_year_last"] = test["Last event in timeline"].apply(safe_parse_year)


In [ ]:
# === Check types and summary stats ===
print("🔍 Column types:\n", train[["event_year_first", "event_year_last"]].dtypes)

print("\n📊 Descriptive stats for train:")
print(train[["event_year_first", "event_year_last"]].describe())

print("\n📊 Descriptive stats for test:")
print(test[["event_year_first", "event_year_last"]].describe())

# === Check value counts (frequency) ===
print("\n🗓 Top 10 most frequent 'event_year_first':")
print(train["event_year_first"].value_counts(dropna=False).sort_index().tail(10))

print("\n🗓 Top 10 most frequent 'event_year_last':")
print(train["event_year_last"].value_counts(dropna=False).sort_index().tail(10))

# Show first 20 rows for event_year_first and event_year_last in training set
print("📄 First 20 rows (Training Data):")
print(train[["event_year_first", "event_year_last"]].head(20))

# Show first 20 rows for event_year_first and event_year_last in testing set
print("\n📄 First 20 rows (Testing Data):")
print(test[["event_year_first", "event_year_last"]].head(20))


**[b] --- --- --- Dealing with multiple time event within 1 variable**

In [ ]:
from dateutil import parser

def safe_parse(date_str):
    try:
        return parser.parse(date_str, dayfirst=False)
    except:
        try:
            return parser.parse(date_str, dayfirst=True)
        except:
            return None

def extract_sorted_events(row):
    types = row.get("Full timeline of events (types)")
    dates = row.get("Full timeline of events (dates)")
    
    if pd.isna(types) or pd.isna(dates):
        return None
    
    type_list = [t.strip() for t in str(types).split(";")]
    date_list = [d.strip() for d in str(dates).split(";")]
    
    if len(type_list) != len(date_list):
        return None  # mismatched length
    
    paired = [(t, safe_parse(d)) for t, d in zip(type_list, date_list)]
    paired = [x for x in paired if x[1] is not None]  # drop invalid
    return sorted(paired, key=lambda x: x[1])

train["event_timeline"] = train.apply(extract_sorted_events, axis=1)
test["event_timeline"] = test.apply(extract_sorted_events, axis=1)


In [ ]:
def order_after(df, move_col, after_col):
    cols = list(df.columns)
    cols.remove(move_col)
    insert_at = cols.index(after_col) + 1
    cols.insert(insert_at, move_col)
    return df[cols]

train = order_after(train, "event_timeline", "Full timeline of events (dates)")
test = order_after(test, "event_timeline", "Full timeline of events (dates)")


In [ ]:
import numpy as np

for df in [train, test]:
    # 1. Count of amendments
    df["num_amendments"] = df["event_timeline"].apply(
        lambda x: sum(1 for t, _ in x if t == "Amended") if x else 0
    )

    # 2. First "Passed/Approved" date
    df["first_passed_date"] = df["event_timeline"].apply(
        lambda x: next((d for t, d in x if t == "Passed/Approved"), None) if x else None
    )

    # 3. Last event date (regardless of type)
    df["last_event_date"] = df["event_timeline"].apply(
        lambda x: x[-1][1] if x else None
    )

    # 4. Duration in years
    df["policy_duration_years"] = df.apply(
        lambda row: (row["last_event_date"] - row["first_passed_date"]).days / 365.25
        if row["last_event_date"] and row["first_passed_date"] else np.nan,
        axis=1
    )

print(train[["first_passed_date", "last_event_date", "num_amendments", "policy_duration_years"]].head(10))


In [ ]:
# Convert to datetime if not already
for df in [train, test]:
    df["first_passed_date"] = pd.to_datetime(df["first_passed_date"], errors="coerce")
    df["last_event_date"] = pd.to_datetime(df["last_event_date"], errors="coerce")

    # Extract year after conversion
    df["event_year_passed"] = df["first_passed_date"].dt.year
    df["event_year_last_event"] = df["last_event_date"].dt.year

# ✅ Preview
print("🔍 Sample extracted years:")
print(train[["first_passed_date", "event_year_passed", "last_event_date", "event_year_last_event"]].dropna().head())


**[c] --- --- --- Extra cleaning**

In [ ]:

# === (d) Define target and features ===
target = "policy_action"
exclude_cols = [
    "Document ID", "Document Title", "Family ID", "Family Title",
    "Full timeline of events (types)", target
]
feature_cols = [col for col in train.columns if col not in exclude_cols]

X_train = train[feature_cols].copy()
y_train = train[target]

X_test = test[feature_cols].copy()
y_test = test[target]



In [ ]:
non_informative_cols = []

for col in X_train.columns:
    if X_train[col].dtype == "object":
        # Skip if any row contains lists (unhashable)
        if X_train[col].apply(lambda x: isinstance(x, list)).any():
            continue
        # Drop if it's long text or URL
        if X_train[col].astype(str).str.len().mean() > 100 or \
           X_train[col].astype(str).str.contains("http|www|pdf|doc", case=False).mean() > 0.3:
            non_informative_cols.append(col)
        # Drop if almost every value is unique (like IDs)
        elif X_train[col].nunique(dropna=True) > 0.9 * len(X_train):
            non_informative_cols.append(col)

print("🔎 Non-informative columns to drop:\n", non_informative_cols)


In [ ]:
# Column	Why Drop?
# Family Summary	Long free-text with highly variable length — better for NLP than tabular
# Collection Description(s)	Often redundant and descriptive, not structured
# Document Content URL	Just a URL — no predictive value
# Internal Document ID	Unique ID — not informative, high-cardinality

In [ ]:

# === Drop non-use variables from both train and test ===
X_train.drop(columns=non_informative_cols, inplace=True, errors='ignore')
X_test.drop(columns=non_informative_cols, inplace=True, errors='ignore')


In [ ]:

# === (e) Convert object columns to numeric where possible ===
for df in [X_train, X_test]:
    for col in df.select_dtypes(include='object').columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')



In [ ]:

# === (f) One-hot encode selected categorical variables ===
categorical_vars = [
 "Document Type", "Topic/Response", "Sector", "Instrument",
    "Category", "Framework", "Hazard", "Author", "Author Type",
    "Geographies", "Source", "Geography ISOs"  # <-- New additions
]
categorical_vars = [col for col in categorical_vars if col in X_train.columns]
X_train = pd.get_dummies(X_train, columns=categorical_vars, dummy_na=True)
X_test = pd.get_dummies(X_test, columns=categorical_vars, dummy_na=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)


In [ ]:
# Step 1: Check remaining numeric columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
print(f"🔢 Total numeric columns: {len(numeric_cols)}")

# Step 2: Count missing values per numeric column
missing_train = X_train[numeric_cols].isna().sum()
missing_train = missing_train[missing_train > 0]

print("\n📌 Numeric columns with missing values in training set:")
print(missing_train)

# Step 3: Optional – show percentage missing
percent_missing = (missing_train / len(X_train) * 100).round(2)
print("\n📊 Percentage missing per column:")
print(percent_missing.sort_values(ascending=False))

# Step 4: Show sample rows with missing values in top offender column
if not missing_train.empty:
    top_col = missing_train.idxmax()
    print(f"\n🔎 Sample rows with missing in '{top_col}':")
    display(X_train[X_train[top_col].isna()].head())


✅ Summary of Missingness

| Column                          | Missing (%) | Reason or Type              | Action                                     |
|----------------------------------|--------------|------------------------------|---------------------------------------------|
| `Collection Title(s)`            | 100%         | Metadata, unused             | Drop                                     |
| `Collection Description(s)`      | 100%         | Metadata, unused             | Drop                                     |
| `Document Variant`               | 100%         | Metadata                     | Drop                                     |
| `Language`                       | 100%         | Non-informative              | Drop                                     |
| `First event in timeline`        | 100%         | Raw date field               | Drop (already extracted year)            |
| `Last event in timeline`         | 100%         | Raw date field               | Drop                                     |
| `Full timeline of events (dates)`| 100%         | Raw date field               | Drop                                     |
| `event_timeline`                 | 100%         | List of tuples               | Drop for ML; useful for NLP only         |
| `Date Added to System`           | 100%         | System metadata              | Drop                                     |
| `Last Modified on System`        | 100%         | System metadata              | Drop                                     |
| `Internal Family ID`             | 100%         | Internal ID                  | Drop                                     |
| `Internal Corpus ID`             | 100%         | Internal ID                  | Drop                                     |
| `Internal Collection ID(s)`      | 100%         | Internal ID                  | Drop                                     |
| `Document Role`                  | 100%         | Possibly encoded elsewhere   | Drop                                     |
| `Keyword`                        | 100%         | Free text                    | Drop                                     |


In [ ]:
cols_to_drop = [
    'Collection Title(s)', 'Collection Description(s)', 'Document Variant', 'Language',
    'First event in timeline', 'Last event in timeline', 'Full timeline of events (dates)',
    'event_timeline', 'Date Added to System', 'Last Modified on System',
    'Internal Family ID', 'Internal Corpus ID', 'Internal Collection ID(s)',
    'Document Role', 'Keyword'
]

X_train.drop(columns=[col for col in cols_to_drop if col in X_train.columns], inplace=True)
X_test.drop(columns=[col for col in cols_to_drop if col in X_test.columns], inplace=True)
print("✅ Dropped 100% missing / non-informative columns.")


In [ ]:
# === (g) Impute missing values safely ===

# Identify numeric columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Drop numeric columns with all NaNs in training set
all_nan_cols = [col for col in numeric_cols if X_train[col].isna().all()]
X_train.drop(columns=all_nan_cols, inplace=True)
X_test.drop(columns=all_nan_cols, errors='ignore', inplace=True)

# Re-identify numeric columns after dropping
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Impute if we still have numeric columns
if numeric_cols:
    imputer = SimpleImputer(strategy="mean")
    X_train[numeric_cols] = imputer.fit_transform(X_train[numeric_cols])
    X_test[numeric_cols] = imputer.transform(X_test[numeric_cols])
else:
    print("⚠️ No numeric columns left for imputation.")


# ✅ Confirm imputation worked
print("✅ Remaining numeric columns:", len(numeric_cols))
print("🔍 Any missing values left in training set?", X_train[numeric_cols].isna().any().any())
print("🔍 Any missing values left in test set?", X_test[numeric_cols].isna().any().any())

# Optional: Preview a few numeric columns
X_train[numeric_cols].head()


In [ ]:
# Drop datetime columns that is not useful
X_train.drop(columns=["first_passed_date", "last_event_date"], inplace=True, errors="ignore")
X_test.drop(columns=["first_passed_date", "last_event_date"], inplace=True, errors="ignore")

print("✅ Dropped datetime columns to fix XGBoost training issue.")


**[d] --- --- --- model**

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.impute import SimpleImputer

# === (h) Compute imbalance weight ===
n_pos = (y_train == 1).sum()
n_neg = (y_train == 0).sum()
scale_pos_weight = n_neg / n_pos
print(f"✅ Calculated scale_pos_weight: {scale_pos_weight:.4f}")

# === (i) Train Imbalanced XGBoost Model ===
model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.2,
    n_jobs=-1,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42,
    scale_pos_weight=scale_pos_weight
)
model.fit(X_train, y_train)
print("✅ Weighted XGBoost model trained.")


In [ ]:
#  Apply Model to Full Sample
X_full = pd.concat([X_train, X_test], ignore_index=True)

# === Predict on full sample ===
xgb_prob = model.predict_proba(X_full)[:, 1]
xgb_pred = (xgb_prob > 0.3734).astype(int)


In [ ]:
# === Attach predictions back to original data ===
df_full = pd.concat([train, test], ignore_index=True).copy()
df_full["xgb_prob"] = xgb_prob
df_full["xgb_pred"] = xgb_pred

# === Save final predictions ===
df_full.to_csv("final_climate_policy_predictions_xgboost.csv", index=False)
print("📁 Final predictions saved to: final_climate_policy_predictions_xgboost.csv")
print("✅ Includes: xgb_prob (probability), xgb_pred (final 1/0 label)")


**[e] --- --- --- Aggregate to country-year data**

In [ ]:
import pandas as pd

# === (a) Load Final Prediction Data ===
file_path = "final_climate_policy_predictions_xgboost.csv"
data = pd.read_csv(file_path)

# === (b) Step 1: Extract year from "First event in timeline"
data["document_year"] = pd.to_datetime(data["First event in timeline"], errors='coerce').dt.year

# === (c) Step 2: Drop rows with missing or invalid year or country
data = data.dropna(subset=["document_year", "Geographies"])
data["document_year"] = data["document_year"].astype(int)

# === (d) Aggregation to Country-Year Panel ===
policy_panel = (
    data
    .groupby(["Geographies", "document_year"])
    .agg(
        num_docs=("Document ID", "count"),
        avg_xgb_prob=("xgb_prob", "mean"),
        share_xgb_pred_1=("xgb_pred", "mean"),
        sum_xgb_pred_1=("xgb_pred", "sum")
    )
    .reset_index()
    .sort_values(["Geographies", "document_year"])
)

# === (e) Round for clarity
policy_panel[["avg_xgb_prob", "share_xgb_pred_1"]] = \
    policy_panel[["avg_xgb_prob", "share_xgb_pred_1"]].round(3)

# === (f) Preview and Save
print(policy_panel.head(10))
policy_panel.to_csv("aggregated_policyaction_panel.csv", index=False)
print("✅ Aggregated country-year panel saved to: aggregated_policyaction_panel.csv")


In [ ]:
### refine version include iso code
import pandas as pd

# === (a) Load Final Prediction Data ===
file_path = "final_climate_policy_predictions_xgboost.csv"
data = pd.read_csv(file_path)

# === (b) Extract document year from "First event in timeline"
data["document_year"] = pd.to_datetime(data["First event in timeline"], errors='coerce').dt.year

# === (c) Drop rows with missing year or ISO code
data = data.dropna(subset=["document_year", "Geography ISOs"])
data["document_year"] = data["document_year"].astype(int)

# === (d) Group and Aggregate to ISO3-Year Level ===
panel = (
    data
    .groupby(["Geography ISOs", "document_year"])
    .agg(
        num_docs=("Document ID", "count"),
        avg_xgb_prob=("xgb_prob", "mean"),
        share_xgb_pred_1=("xgb_pred", "mean"),
        sum_xgb_pred_1=("xgb_pred", "sum")
    )
    .reset_index()
    .sort_values(["Geography ISOs", "document_year"])
)

# === (e) Rename columns
panel.rename(columns={
    "Geography ISOs": "iso3",
    "document_year": "year"
}, inplace=True)

# === (f) Round selected columns
panel[["avg_xgb_prob", "share_xgb_pred_1"]] = panel[["avg_xgb_prob", "share_xgb_pred_1"]].round(3)

# === (g) Save to file
panel.to_csv("aggregated_policyaction_panel_iso3.csv", index=False)
print("✅ Saved panel to: aggregated_policyaction_panel_iso3.csv")

# === (h) Preview
print(panel.head(10))
